In [ ]:
#ติดตั้ง Libraries
!pip install "numpy>=1.23.5,<1.27.0"
!pip install "pandas>=1.5.0,<2.3.0"
!pip install "pandas_ta>=0.3.14b0"
!pip install "langgraph>=0.0.67" "pydantic-ai>=0.2.6" "pydantic>=2.5.0" python-dotenv nest-asyncio "httpx>=0.25.0" "yfinance>=0.2.30" newsapi-python scikit-learn -q


In [ ]:
# -*- coding: utf-8 -*-
"""
An Agentic AI Framework for Weekly Stock Portfolio Selection in the U.S. Tech Sector

Kunakorn Pruksakorn and Ekarat Rattagan

Graduate School of Applied Statistics, National Institute of Development Administration, 148 Seri Thai
Road, Klong-Chan, Bangkapi, Bangkok THAILAND 10240
===========================================================================

Specialized AI-driven tech stock analysis for 5-day trading cycles (Monday buy, Friday sell)
- Fixed all datetime comparison issues with proper timezone handling
- Enhanced data source tracking with actual URLs and API endpoints
- Comprehensive error handling and detailed output formatting
- Advanced price predictions with confidence intervals
- Complete API source tracking for full transparency
- Professional-grade analysis with detailed recommendations

Focus: Technology sector stocks with high liquidity
Strategy: Buy Monday, Sell Friday (5-day holding period)
Optimized for production use with complete analysis pipeline
"""

# === COMPREHENSIVE IMPORTS ===
import asyncio
import os
import random
import json
import time
import requests
import math
import warnings
from datetime import datetime, timedelta, timezone
from typing import List, Dict, Any, TypedDict, Optional, Tuple, Union
from dataclasses import dataclass
import pytz

import pandas as pd
import yfinance as yf
import pandas_ta as ta
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from httpx import AsyncClient, HTTPStatusError, TimeoutException
from pydantic import BaseModel, Field, field_validator
from urllib.parse import urlencode
import httpx
from httpx import AsyncClient, HTTPStatusError, TimeoutException
from IPython.display import HTML
display(HTML("<style>div.output_area pre{max-height: none !important; max-width: none !important;}</style>"))
warnings.filterwarnings('ignore')

# === API KEYS CONFIGURATION ===
IS_COLAB = False
GROQ_API_KEY = None
ALPHA_VANTAGE_API_KEY = None
GOOGLE_API_KEY = None
GOOGLE_CSE_ID = None
FINANCIAL_MODELING_PREP_API_KEY = None
POLYGON_API_KEY = None
FRED_API_KEY = None
NEWS_API_KEY = None

try:
   from google.colab import userdata
   IS_COLAB = True
   GROQ_API_KEY = userdata.get('GROQ_API_KEY')
   ALPHA_VANTAGE_API_KEY = userdata.get('ALPHA_VANTAGE_API_KEY')
   GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
   GOOGLE_CSE_ID = userdata.get('GOOGLE_CSE_ID')
   FINANCIAL_MODELING_PREP_API_KEY = userdata.get('FINANCIAL_MODELING_PREP_API_KEY')
   POLYGON_API_KEY = userdata.get('POLYGON_API_KEY')
   FRED_API_KEY = userdata.get('FRED_API_KEY')
   NEWS_API_KEY = userdata.get('NEWS_API_KEY')
   print("🔑 Loaded API keys from Google Colab userdata.")

   if GROQ_API_KEY:
       os.environ["GROQ_API_KEY"] = GROQ_API_KEY
       print("   ✓ Set GROQ_API_KEY in environment.")
   else:
       print("   ❌ GROQ_API_KEY not found in Colab userdata.")

except ImportError:
   try:
       from dotenv import load_dotenv
       print("🔧 Not running in Colab. Loading keys from .env file.")
       load_dotenv()
       GROQ_API_KEY = os.getenv('GROQ_API_KEY')
       ALPHA_VANTAGE_API_KEY = os.getenv('ALPHA_VANTAGE_API_KEY')
       GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
       GOOGLE_CSE_ID = os.getenv('GOOGLE_CSE_ID')
       FINANCIAL_MODELING_PREP_API_KEY = os.getenv('FINANCIAL_MODELING_PREP_API_KEY')
       POLYGON_API_KEY = os.getenv('POLYGON_API_KEY')
       FRED_API_KEY = os.getenv('FRED_API_KEY')
       NEWS_API_KEY = os.getenv('NEWS_API_KEY')
   except ImportError:
       print("⚠️ No .env file found. Using environment variables.")

# === API KEYS STATUS CHECK ===
print("\n🔍 API Keys Status:")
api_status = {
   "GROQ_API_KEY": "✓" if GROQ_API_KEY else "❌ CRITICAL",
   "ALPHA_VANTAGE_API_KEY": "✓" if ALPHA_VANTAGE_API_KEY else "⚠️ Important",
   "NEWS_API_KEY": "✓" if NEWS_API_KEY else "⚠️ Important",
   "POLYGON_API_KEY": "✓" if POLYGON_API_KEY else "⚠️ Optional",
}

for key, status in api_status.items():
   print(f"   {key}: {status}")

# === MOCK LIBRARIES FOR COMPATIBILITY ===
try:
    from pydantic_ai import Agent, RunContext
    print("✅ PydanticAI loaded successfully")
    raise ImportError("Force using custom Agent for better control") # บังคับให้ไปใช้ except

except ImportError:
    print("⚠️ PydanticAI not available, using direct API calls")

    class Agent:
        def __init__(self, model, system_prompt="", **kwargs):
            self.model = model
            self.system_prompt = system_prompt
            self.kwargs = kwargs

        async def run(self, prompt, deps=None):
            """เรียก AI API จริงแทนการ simulate"""
            return await self._call_real_ai_api(prompt)


        async def _call_real_ai_api(self, prompt):
            """เรียก GROQ API จริงเท่านั้น - แก้ไขให้ทนทานต่อการตอบสนองทุกรูปแบบ"""
            if not GROQ_API_KEY:
                raise Exception("❌ CRITICAL: GROQ API key required - ระบบต้องใช้ AI จริงเท่านั้น")

            try:
                import httpx
                import re # <-- เพิ่มการ import re
                import json

                headers = {
                    "Authorization": f"Bearer {GROQ_API_KEY}",
                    "Content-Type": "application/json"
                }

                data = {
                    "model": "llama-3.3-70b-versatile",
                    "messages": [
                        {"role": "system", "content": self.system_prompt},
                        {"role": "user", "content": prompt}
                    ],
                    "temperature": 0.2,
                    "max_tokens": 2000,
                    "response_format": {"type": "json_object"}
                }

                async with httpx.AsyncClient(timeout=60.0) as client:
                    response = await client.post(
                        "https://api.groq.com/openai/v1/chat/completions",
                        headers=headers,
                        json=data
                    )

                    if response.status_code == 200:
                        result = response.json()
                        content = result["choices"][0]["message"]["content"]

                        # --- ส่วนแก้ไขที่สำคัญที่สุด ---
                        # เพิ่ม Log เพื่อดูการตอบกลับดิบๆ ไว้ดีบักเสมอ
                        # print(f"--- RAW AI RESPONSE ---\n{content}\n-----------------------")

                        try:
                            # 1. ลองหา JSON block จากในข้อความทั้งหมดก่อน
                            json_match = re.search(r'\{.*\}', content, re.DOTALL)

                            if json_match:
                                json_string = json_match.group(0)
                                ai_response = json.loads(json_string)
                                print("✅ Successfully extracted JSON from mixed text.")
                                return ai_response
                            else:
                                # ถ้าหาไม่เจอ แสดงว่าการตอบกลับอาจมีปัญหา
                                raise json.JSONDecodeError("No JSON object found in response", content, 0)

                        except (json.JSONDecodeError, AttributeError):
                            print(f"⚠️ JSON parsing failed, falling back to text extraction.")
                            return self._extract_score_from_text(content)
                    else:
                        raise Exception(f"API Error {response.status_code}: {response.text}")

            except Exception as e:
                print(f"❌ CRITICAL AI API ERROR: {e}")
                raise Exception(f"AI analysis failed - no fallback available: {e}")

        def _extract_score_from_text(self, text):
            """Extract score from AI text response"""
            import re

            score_match = re.search(r'"?score"?\s*:?\s*([0-9.]+)', text)
            confidence_match = re.search(r'"?confidence"?\s*:?\s*([0-9.]+)', text)

            score = float(score_match.group(1)) if score_match else 6.0
            confidence = float(confidence_match.group(1)) if confidence_match else 0.7

            return {
                "score": min(10.0, max(0.0, score)),
                "confidence": min(1.0, max(0.0, confidence)),
                "recommendation": "moderate_buy_monday",
                "source": "ai_text_extraction"
            }

        # def _fallback_data_driven_analysis(self, prompt):
        #     """Data-driven analysis แทนการสุ่ม"""
        #     # Extract symbol and agent type
        #     symbol = "UNKNOWN"
        #     if "SYMBOL:" in prompt:
        #         try:
        #             symbol = prompt.split("SYMBOL:")[1].split()[0]
        #         except:
        #             pass

        #     agent_type = "general"
        #     if "momentum" in prompt.lower():
        #         agent_type = "momentum"
        #     elif "catalyst" in prompt.lower():
        #         agent_type = "catalyst"
        #     elif "technical" in prompt.lower():
        #         agent_type = "technical"
        #     elif "liquidity" in prompt.lower():
        #         agent_type = "liquidity"
        #     elif "risk" in prompt.lower():
        #         agent_type = "risk"
        #     elif "timing" in prompt.lower():
        #         agent_type = "timing"

        #     # วิเคราะห์จาก context data จริง
        #     return self._analyze_from_real_data(agent_type, symbol, prompt)

        # def _extract_context_from_prompt(self, prompt):
        #     """Extract context data from prompt"""
        #     import json
        #     import re

        #     try:
        #         # หาส่วน DETAILED CONTEXT DATA หรือ ANALYSIS CONTEXT
        #         context_match = re.search(r'DETAILED CONTEXT DATA:\s*\n(.*?)\n\n', prompt, re.DOTALL)
        #         if not context_match:
        #             context_match = re.search(r'ANALYSIS CONTEXT:\s*\n(.*?)\n\n', prompt, re.DOTALL)

        #         if context_match:
        #             context_str = context_match.group(1)
        #             return json.loads(context_str)
        #         else:
        #             return {}
        #     except:
        #         return {}

        # def _analyze_from_real_data(self, agent_type, symbol, prompt):
        #     """วิเคราะห์จากข้อมูลจริงแทนการสุ่ม"""
        #     context_data = self._extract_context_from_prompt(prompt)

        #     if agent_type == "momentum":
        #         return self._analyze_momentum_agent(context_data, symbol)
        #     elif agent_type == "catalyst":
        #         return self._analyze_catalyst_agent(context_data, symbol)
        #     elif agent_type == "technical":
        #         return self._analyze_technical_agent(context_data, symbol)
        #     elif agent_type == "liquidity":
        #         return self._analyze_liquidity_agent(context_data, symbol)
        #     elif agent_type == "risk":
        #         return self._analyze_risk_agent(context_data, symbol)
        #     elif agent_type == "timing":
        #         return self._analyze_timing_agent(context_data, symbol)
        #     else:
        #         return {"score": 6.0, "confidence": 0.6, "recommendation": "hold"}

        # def _analyze_momentum_agent(self, context_data, symbol):
        #     """Momentum Agent - วิเคราะห์จากข้อมูล momentum จริง"""
        #     base_score = 5.0

        #     # วิเคราะห์จาก predicted return
        #     predicted_return = context_data.get('predicted_5day_return', 0)
        #     if predicted_return > 5:
        #         base_score += 3.0
        #     elif predicted_return > 3:
        #         base_score += 2.0
        #     elif predicted_return > 1:
        #         base_score += 1.0
        #     elif predicted_return < -2:
        #         base_score -= 1.5

        #     # วิเคราะห์จาก momentum trend
        #     momentum_trend = context_data.get('daily_momentum', {}).get('momentum_trend', 'neutral')
        #     if momentum_trend == 'positive':
        #         base_score += 1.0
        #     elif momentum_trend == 'negative':
        #         base_score -= 1.0

        #     # วิเคราะห์จาก confidence
        #     prediction_confidence = context_data.get('prediction_confidence', 0.5)
        #     base_score += (prediction_confidence - 0.5) * 2

        #     final_score = max(1.0, min(10.0, base_score))

        #     if final_score >= 8.5:
        #         recommendation = "strong_buy_monday"
        #     elif final_score >= 7.0:
        #         recommendation = "buy_monday"
        #     elif final_score >= 5.5:
        #         recommendation = "moderate_buy_monday"
        #     else:
        #         recommendation = "hold"

        #     confidence = min(0.95, max(0.5, prediction_confidence + 0.2))

        #     return {
        #         "score": round(final_score, 1),
        #         "confidence": round(confidence, 2),
        #         "recommendation": recommendation,
        #         "key_insights": [
        #             f"Predicted 5-day return: {predicted_return:+.1f}%",
        #             f"Momentum trend: {momentum_trend}",
        #             f"Confidence level: {prediction_confidence:.1%}"
        #         ],
        #         "rationale": f"Data-driven momentum analysis: {predicted_return:+.1f}% expected return with {momentum_trend} trend",
        #         "agent_type": "momentum",
        #         "data_driven": True
        #     }

        # def _analyze_catalyst_agent(self, context_data, symbol):
        #     """Catalyst Agent - วิเคราะห์จากข้อมูลข่าวจริง"""
        #     base_score = 5.0

        #     catalyst_strength = context_data.get('catalyst_strength', 'none')
        #     if catalyst_strength == 'strong':
        #         base_score += 3.0
        #     elif catalyst_strength == 'moderate':
        #         base_score += 1.5
        #     elif catalyst_strength == 'weak':
        #         base_score += 0.5

        #     news_count = len(context_data.get('breaking_news', []))
        #     if news_count >= 5:
        #         base_score += 1.5
        #     elif news_count >= 3:
        #         base_score += 1.0
        #     elif news_count >= 1:
        #         base_score += 0.5

        #     final_score = max(1.0, min(10.0, base_score))

        #     if final_score >= 8.0:
        #         recommendation = "strong_buy_monday"
        #     elif final_score >= 7.0:
        #         recommendation = "buy_monday"
        #     elif final_score >= 5.5:
        #         recommendation = "moderate_buy_monday"
        #     else:
        #         recommendation = "hold"

        #     return {
        #         "score": round(final_score, 1),
        #         "confidence": 0.8 if catalyst_strength == 'strong' else 0.6,
        #         "recommendation": recommendation,
        #         "key_insights": [
        #             f"Catalyst strength: {catalyst_strength}",
        #             f"Breaking news articles: {news_count}",
        #             "Data-driven news analysis"
        #         ],
        #         "rationale": f"News catalyst analysis: {catalyst_strength} strength with {news_count} articles",
        #         "agent_type": "catalyst",
        #         "data_driven": True
        #     }

        # def _analyze_technical_agent(self, context_data, symbol):
        #     """Technical Agent"""
        #     base_score = 5.0

        #     momentum_strength = context_data.get('momentum_strength', 'neutral')
        #     if momentum_strength == 'strong_bullish':
        #         base_score += 3.0
        #     elif momentum_strength == 'bullish':
        #         base_score += 2.0
        #     elif momentum_strength == 'strong_bearish':
        #         base_score -= 2.0
        #     elif momentum_strength == 'bearish':
        #         base_score -= 1.0

        #     volume_confirmation = context_data.get('volume_confirmation', False)
        #     if volume_confirmation:
        #         base_score += 1.0

        #     final_score = max(1.0, min(10.0, base_score))

        #     return {
        #         "score": round(final_score, 1),
        #         "confidence": 0.75,
        #         "recommendation": "buy_monday" if final_score >= 7 else "hold",
        #         "key_insights": [f"Technical momentum: {momentum_strength}"],
        #         "rationale": f"Technical analysis: {momentum_strength} momentum",
        #         "agent_type": "technical",
        #         "data_driven": True
        #     }

        # def _analyze_liquidity_agent(self, context_data, symbol):
        #     """Liquidity Agent"""
        #     base_score = 5.0

        #     liquidity_score = context_data.get('liquidity_score', 5.0)
        #     base_score += (liquidity_score - 5.0) * 0.5

        #     avg_volume = context_data.get('average_daily_volume', 0)
        #     if avg_volume > 50_000_000:
        #         base_score += 2.0
        #     elif avg_volume > 20_000_000:
        #         base_score += 1.0

        #     final_score = max(1.0, min(10.0, base_score))

        #     return {
        #         "score": round(final_score, 1),
        #         "confidence": 0.8,
        #         "recommendation": "buy_monday" if final_score >= 7 else "hold",
        #         "key_insights": [f"Liquidity score: {liquidity_score:.1f}/10"],
        #         "rationale": f"Liquidity analysis: {liquidity_score:.1f}/10 score",
        #         "agent_type": "liquidity",
        #         "data_driven": True
        #     }

        # def _analyze_risk_agent(self, context_data, symbol):
        #     """Risk Agent"""
        #     base_score = 7.0  # เริ่มสูงแล้วลดตาม risk

        #     five_day_var = context_data.get('five_day_var', -3.0)
        #     if five_day_var and abs(five_day_var) > 8:
        #         base_score -= 2.0
        #     elif five_day_var and abs(five_day_var) > 5:
        #         base_score -= 1.0

        #     risk_reward = context_data.get('risk_reward_ratio', 1.0)
        #     if risk_reward > 2.0:
        #         base_score += 1.0
        #     elif risk_reward < 0.8:
        #         base_score -= 1.0

        #     final_score = max(1.0, min(10.0, base_score))

        #     return {
        #         "score": round(final_score, 1),
        #         "confidence": 0.7,
        #         "recommendation": "buy_monday" if final_score >= 6.5 else "hold",
        #         "key_insights": [f"Risk-reward ratio: {risk_reward:.2f}"],
        #         "rationale": f"Risk analysis: {risk_reward:.2f} risk-reward ratio",
        #         "agent_type": "risk",
        #         "data_driven": True
        #     }

        # def _analyze_timing_agent(self, context_data, symbol):
        #     """Timing Agent"""
        #     base_score = 6.0

        #     success_probability = context_data.get('success_probability', 0.5)
        #     base_score += (success_probability - 0.5) * 4

        #     best_entry = context_data.get('best_entry_time', 'monday_open')
        #     if 'monday' in best_entry:
        #         base_score += 1.0

        #     final_score = max(1.0, min(10.0, base_score))

        #     return {
        #         "score": round(final_score, 1),
        #         "confidence": 0.75,
        #         "recommendation": "buy_monday" if final_score >= 7 else "hold",
        #         "key_insights": [f"Success probability: {success_probability:.1%}"],
        #         "rationale": f"Timing analysis: {success_probability:.1%} success rate",
        #         "agent_type": "timing",
        #         "data_driven": True
        #     }
        ##    }

    print("✅ PydanticAI mock loaded successfully")
except Exception as e:
    print(f"❌ PydanticAI mock setup error: {e}")

try:
   class StateGraph:
       def __init__(self, state_type):
           self.state_type = state_type
           self.nodes = {}
           self.edges = {}
           self.entry_point = None

       def add_node(self, name, func):
           self.nodes[name] = func

       def add_edge(self, from_node, to_node):
           if from_node not in self.edges:
               self.edges[from_node] = []
           self.edges[from_node].append(to_node)

       def add_conditional_edges(self, from_node, condition_func, mapping):
           pass

       def set_entry_point(self, node_name):
           self.entry_point = node_name

       def compile(self):
           return WorkflowApp(self)

   class WorkflowApp:
       def __init__(self, graph):
           self.graph = graph

       async def ainvoke(self, initial_state):
           state = initial_state

           if "initialize" in self.graph.nodes:
               state.update(await self.graph.nodes["initialize"](state))

           while not state.get("batch_completed", False):
               if "batch_processing" in self.graph.nodes:
                   result = await self.graph.nodes["batch_processing"](state)
                   state.update(result)
               else:
                   break

           if "portfolio_selector" in self.graph.nodes:
               result = await self.graph.nodes["portfolio_selector"](state)
               state.update(result)

           return state

   END = "END"
   print("✅ LangGraph mock loaded successfully")
except Exception as e:
   print(f"❌ LangGraph mock setup error: {e}")

# # === CONFIGURATION ===
# MAX_TECH_STOCKS_TO_ANALYZE = 15
# MAX_STOCKS_TO_SELECT = 5
# MIN_WEEKLY_SCORE = 6.0
# BATCH_SIZE = 5

ACTIVE_GROQ_MODEL = "groq:llama-3.3-70b-versatile"
TEMPERATURE_SETTING = 0.2 #0.1

# === TECHNOLOGY STOCKS UNIVERSE ===
# TECH_STOCKS_UNIVERSE = [
#    # FAANG + Tech Giants (Market Cap > $500B)
#    "AAPL", "MSFT", "GOOGL", "GOOG", "AMZN", "META", "TSLA", "NFLX",

#    # AI & Semiconductors Leaders ($100B+)
#    "NVDA", "AMD", "INTC", "AVGO", "QCOM", "MU", "TSM", "ASML",

#    # Cloud & Enterprise Software ($50B+)
#    "CRM", "ORCL", "ADBE", "INTU", "NOW", "SNOW", "WDAY", "DDOG", "CRWD",

#    # E-commerce & Digital Platforms ($20B+)
#    "SHOP", "UBER", "ABNB", "DASH", "PYPL", "SQ", "COIN", "HOOD",

#    # Semiconductor Equipment & Materials
#    "LRCX", "AMAT", "KLAC", "TER", "MPWR", "MRVL", "ON", "ADI", "TXN", "NXPI", "MCHP", "WOLF",

#    # Software & SaaS (Growth Companies)
#    "ZM", "OKTA", "ZS", "TTD", "DOCU", "TWLO", "TEAM", "MDB", "NET", "FSLY", "GTLB", "MNDY",
#    "ZI", "BILL", "PATH", "VEEV", "PANW", "FTNT", "S", "CYBR", "TENB",

#    # Gaming & Entertainment Tech
#    "EA", "ATVI", "TTWO", "RBLX", "U", "UNITY", "ROKU", "SPOT", "PINS", "SNAP",

#    # Hardware & Infrastructure
#    "DELL", "HPQ", "CSCO", "ANET", "SMCI", "WDC", "STX", "NTAP", "PURE",

#    # Telecom & 5G Infrastructure
#    "CHTR", "CMCSA", "T", "VZ", "TMUS", "AMT", "CCI", "SBAC",

#    # Biotech & HealthTech with AI Focus
#    "MRNA", "BNTX", "TDOC", "VEEV", "ISRG", "DXCM", "ILMN", "REGN", "GILD", "BIIB",

#    # Fintech & Blockchain
#    "COIN", "MSTR", "RIOT", "MARA", "CLSK", "HUT", "BITF", "CAN", "EBON",

#    # Electric Vehicles & Clean Energy Tech
#    "RIVN", "LCID", "NIO", "LI", "XPEV", "ENPH", "SEDG", "FSLR", "PLUG", "BE",

#    # Emerging Tech & Space
#    "RKLB", "SPCE", "JOBY", "LILM", "ACHR", "EVTL", "UAL", "LUV",

#    # Additional High-Growth Tech
#    "CRWD", "ZS", "OKTA", "DDOG", "NET", "FSLY", "TEAM", "MDB", "SNOW", "PLTR",
#    "PALANTIR", "SOFI", "UPST", "AFRM", "PTON", "ZOOM", "DOCU", "TWLO"
# ]

TECH_STOCKS_UNIVERSE = [
    # 🔹 FAANG + Megacap Tech + AI Leaders
    "AAPL", "MSFT", "GOOGL", "AMZN", "META", "TSLA", "NVDA",

    # 🔹 AI & Semiconductors (Top Performers)
    "AMD", "AVGO", "MU", "TSM", "ASML", "SMCI", "LRCX", "MRVL", "INTC",
    "AMAT", "KLAC", "MPWR", "ON", "NXPI", "SNPS",

    # 🔹 Cloud / SaaS / AI Enterprise
    "CRM", "NOW", "SNOW", "DDOG", "MDB", "TEAM", "ADBE", "INTU", "WDAY", "VEEV", "ESTC",

    # 🔹 Cybersecurity
    "CRWD", "ZS", "PANW", "FTNT", "OKTA", "CYBR", "TENB", "AKAM",

    # 🔹 AI-Growth & Momentum Tech
    "PLTR", "PATH", "MSTR", "BILL", "UPST", "AFRM", "SOFI", "NTNX",

    # 🔹 Fintech & Digital Platforms
    "SHOP", "COIN", "HOOD", "PYPL", "ABNB", "UBER", "NU",

    # 🔹 Digital Media / Analytics / Edge
    "NET", "FSLY", "TTD", "GTLB", "MNDY",

    # 🔹 Consumer Tech / Gaming
    "RBLX", "U", "SPOT", "EA", "DELL", "ANET", "CSCO", "WDC", "ZM", "RNG", "COMM",

    # 🔹 Biotech & HealthTech (AI-Enhanced)
    "ISRG", "REGN", "DXCM", "TDOC", "BNTX",

    # 🔹 Electric Vehicles & Clean Energy
    "NIO", "RIVN", "LI", "XPEV", "FSLR", "ENPH", "SEDG", "CHPT",

    # 🔹 E-commerce & Retail Tech
    "ETSY", "W", "CHWY",

    # 🔹 Swing Picks / Miscellaneous
    "DOCU", "TWLO"
]


# Remove duplicates and sort - should give us 80+ unique symbols
TECH_STOCKS_UNIVERSE = sorted(list(set(TECH_STOCKS_UNIVERSE)))
print(f"🏭 Total Tech Stocks: {len(TECH_STOCKS_UNIVERSE)} symbols")

MAX_TECH_STOCKS_TO_ANALYZE = len(TECH_STOCKS_UNIVERSE)
MAX_STOCKS_TO_SELECT = 5
MONEY = 200
MIN_WEEKLY_SCORE = 6.0            # ลดจาก 7.0 เป็น 6.0
MIN_POSITIVE_RETURN = 0.2       # ลดจาก 0.0 เป็น -1.0
MIN_CONFIDENCE_LEVEL = 0.25      # ลดจาก 0.20 เป็น 0.15
MIN_SUCCESS_PROBABILITY = 0.25     # คงเดิม
MIN_LIQUIDITY_THRESHOLD = 3.0     # เพิ่มตัวแปรใหม่
MIN_RISK_REWARD_RATIO = 0.3       # เพิ่มตัวแปรใหม่
BATCH_SIZE = 10

print(f"🎯 Configuration Updated:")
print(f"📊 Will analyze ALL {MAX_TECH_STOCKS_TO_ANALYZE} tech stocks")
print(f"🔥 Portfolio size: {MAX_STOCKS_TO_SELECT} best stocks")
print(f"💯 Minimum score: {MIN_WEEKLY_SCORE}/10 (only the best!)")
print(f"⚡ Batch size: {BATCH_SIZE} stocks per batch")

# === API DELAYS ===
API_CALL_DELAY_YFINANCE = 0.2
API_CALL_DELAY_ALPHA_VANTAGE = 12.0
API_CALL_DELAY_NEWS = 0.8
API_CALL_DELAY_POLYGON = 0.3

# === ANALYSIS PERIODS ===
SHORT_TERM_ANALYSIS_PERIOD = "1mo"
INTRADAY_PERIOD = "5d"
NEWS_ARTICLES_TARGET = 20
TECHNICAL_ANALYSIS_PERIOD = "1mo"
VOLUME_ANALYSIS_PERIOD = "1mo"

# === TOKEN MANAGER ===
class ShortTermTokenManager:
   def __init__(self, max_tokens_per_minute=8000):
       self.max_tokens = max_tokens_per_minute
       self.used_tokens = 0
       self.reset_time = time.time() + 60
       self.request_history = []

   async def can_make_request(self, estimated_tokens=400):
       current_time = time.time()

       if current_time >= self.reset_time:
           self.used_tokens = 0
           self.reset_time = current_time + 60
           self.request_history.clear()

       if self.used_tokens + estimated_tokens > self.max_tokens:
           wait_time = self.reset_time - current_time
           print(f"⏳ Rate limit: waiting {wait_time:.1f}s")
           await asyncio.sleep(wait_time + 1)
           return await self.can_make_request(estimated_tokens)

       self.used_tokens += estimated_tokens
       return True

   async def wait_if_needed(self, estimated_tokens=400):
       await self.can_make_request(estimated_tokens)

token_manager = ShortTermTokenManager()

def create_enhanced_agent_prompt(agent_name: str, context_data: Dict, symbol: str) -> str:
    """สร้าง prompt ที่ละเอียดสำหรับแต่ละ agent"""

    base_context = f"""
═══════════════════════════════════════════════════════════════════════════════
🎯 PROFESSIONAL AI STOCK ANALYSIS - {agent_name.upper()} SPECIALIST
UNIQUE_ANALYSIS_ID: {symbol}_{agent_name}_{hash(str(context_data)) % 10000}
═══════════════════════════════════════════════════════════════════════════════

CRITICAL: Each stock requires UNIQUE analysis. Do NOT use generic responses.
SYMBOL-SPECIFIC FOCUS: Analyze {symbol} specifically, not generic patterns.

TARGET: {symbol} (Technology Sector)
STRATEGY: 5-Day Momentum Swing Trading (Monday Buy → Friday Sell)
ANALYSIS_DATE: {datetime.now().strftime('%Y-%m-%d %H:%M:%S UTC')}

COMPREHENSIVE DATA PROVIDED:
{json.dumps(context_data, default=str, indent=2)}

"""

    if agent_name == "momentum":
        specific_prompt = """
        PERSONA: You are a Senior Quantitative Analyst at a momentum-focused hedge fund. Your analysis must be data-driven, skeptical, and focused on the sustainability and quality of the momentum, not just its direction.

        MISSION: Conduct a rigorous analysis of the provided data to determine if there is a high-quality, sustainable momentum signal for a 5-day swing trade. Your primary goal is to differentiate between genuine, powerful trends and short-lived, low-quality price movements.

        ANALYTICAL FRAMEWORK (Follow these steps):
        1.  **Trend Quality & Confirmation:** Is the price trend confirmed by a corresponding increase in volume from the `COMPREHENSIVE DATA PROVIDED`? High-volume up-days are a strong sign.
        2.  **Momentum Dynamics:** Analyze momentum indicators (like RSI, MACD histogram) to detect signs of exhaustion or divergence. A rising price with falling momentum is a major red flag.
        3.  **Sustainability Assessment:** Based on the above, assess the probability that this momentum can be sustained for the next 5 trading days.

        - Use ONLY the FACTS/NEWS provided above. Do NOT invent numbers or events.
        - STRICT JSON output only. No text outside JSON.
        - If evidence/clarity is insufficient → set "expected_5day_return": null and lower "confidence".

        REQUIRED JSON OUTPUT (Strictly adhere to this format):
        {
            "score": [0.0-10.0, precise to one decimal, based on the OVERALL QUALITY of the momentum setup],
            "confidence": [0.0-1.0 confidence level, your conviction in the sustainability of the trend],
            "recommendation": "[strong_buy_monday|buy_monday|hold|avoid]",
            "expected_5day_return": [Your statistically-driven return prediction % for the next 5 days],
            "momentum_quality": "[Excellent|Good|Average|Poor|Exhausted]",
            "key_findings": {
                "strongest_bullish_evidence": "Cite the single most compelling piece of data supporting sustained momentum (e.g., 'Volume on the last up-day was 150% of the 20-day average').",
                "most_significant_risk": "Cite the primary risk that could cause this momentum to fail (e.g., 'Clear bearish divergence on the 14-day RSI')."
            },
            "rationale": "Provide a concise, professional summary synthesizing your analysis of trend quality, dynamics, and sustainability."
        }
        """

    elif agent_name == "catalyst":
        specific_prompt = """
        PERSONA: You are a sharp, skeptical analyst at a global macro hedge fund specializing in catalyst-driven events in the tech sector. Your job is to find alpha by identifying market mispricing around news events. You ignore hype and focus on second-order effects.

        MISSION: Analyze the provided news catalysts to determine if they create a tangible, mispriced trading opportunity over a 5-day horizon. Your analysis MUST be grounded in the `breaking_news` data provided.

        ANALYTICAL FRAMEWORK (Follow these steps):
        1.  **"Is It Priced In?":** This is your most important question. How much of this news was already anticipated by the market? An expected product launch or an in-line earnings report is NOT a catalyst. Look for genuine surprise.
        2.  **Magnitude & Certainty:** How significant is the impact of the news on the company's fundamentals (e.g., revenue, margins)? A signed major contract is more certain than a potential partnership.
        3.  **"Sell the News" Risk:** Assess the probability that this is a "buy the rumor, sell the news" event. Has the stock already run up significantly in anticipation of this news?

        - Use ONLY the FACTS/NEWS provided above. Do NOT invent numbers or events.
        - STRICT JSON output only. No text outside JSON.
        - If evidence/clarity is insufficient → set "expected_5day_return": null and lower "confidence".
        - evidence must be URLs from NEWS; if none → cap "score" ≤ 6.0 and "expected_5day_return": null

        REQUIRED JSON OUTPUT (Strictly adhere to this format):
        {
            "score": [0.0-10.0, precise to one decimal, based on the alpha-generating potential of the catalyst],
            "confidence": [0.0-1.0 confidence level, your conviction that the news is not fully priced in],
            "recommendation": "[strong_buy_monday|buy_monday|hold|avoid]",
            "expected_5day_return": [Your predicted return % derived from the market's potential re-pricing due to the catalyst],
            "catalyst_edge": "[High Alpha Potential|Moderate Edge|Fully Priced In|Narrative Trap|Negative Surprise]",
            "key_findings": {
                "untapped_potential": "Explain why you believe the market is under-reacting to this news.",
                "priced_in_risk": "Explain why the market may have already fully anticipated this news, making it a non-event or a 'sell the news' candidate."
            },
            "rationale": "Provide a concise, professional summary assessing if an actionable edge exists from this news within the next 5 days."
        }
        """

    elif agent_name == "technical":
        specific_prompt = """
        PERSONA: You are the Head of Technical Analysis at a proprietary trading firm. You only approve trades with high-probability setups confirmed by multiple, non-correlated indicators. You reject ambiguous or conflicting chart signals.

        MISSION: Conduct a thorough technical analysis of the `swing_signals` and `chart_patterns` provided to identify a high-conviction 5-day swing trading setup. Your goal is to find a confluence of bullish signals.

        ANALYTICAL FRAMEWORK (Follow these steps):
        1.  **Trend Confirmation:** Does the `ma_trend` signal a bullish short-term trend? Is it supported by the longer-term picture?
        2.  **Signal Convergence:** Look for a "convergence" of bullish signals across `rsi_signal`, `macd_signal`, and `bb_signal`. A setup is strongest when at least 3 indicators agree.
        3.  **Risk/Reward Mapping:** Analyze the `suggested_entry_level`, `suggested_exit_level`, and `stop_loss_level`. Is the potential reward significantly greater than the potential loss?

        - Use ONLY the FACTS/NEWS provided above. Do NOT invent numbers or events.
        - STRICT JSON output only. No text outside JSON.
        - If evidence/clarity is insufficient → set "expected_5day_return": null and lower "confidence".

        REQUIRED JSON OUTPUT (Strictly adhere to this format):
        {
            "score": [0.0-10.0, precise to one decimal, based on the strength and convergence of bullish signals],
            "confidence": [0.0-1.0 confidence level, your conviction in the setup],
            "recommendation": "[strong_buy_monday|buy_monday|hold|avoid]",
            "expected_5day_return": [Your predicted return % if the technical pattern plays out],
            "setup_quality": "[A+ Grade|B Grade|C Grade|Avoid]",
            "key_findings": {
                "confirming_signals": "List up to 3 independent signals from the provided data that confirm a bullish thesis.",
                "conflicting_signals": "List any signals from the provided data that conflict with the bullish thesis."
            },
            "rationale": "Provide a concise summary of the technical setup, mentioning the key support/resistance levels and overall conviction."
        }
        """

    elif agent_name == "liquidity":
        specific_prompt = """
        PERSONA: You are the Head of Trade Execution for a multi-billion dollar quantitative fund. Your primary concern is executing large orders efficiently with minimal market impact and slippage. A great stock idea is worthless if you can't get in and out at a good price.

        MISSION: Analyze the provided liquidity data to assess the real-world feasibility and execution risk of establishing and exiting a significant position within a 5-day window.

        ANALYTICAL FRAMEWORK (Follow these steps):
        1.  **Volume Analysis:** Scrutinize the `average_daily_volume` and `volume_trend`. Is the volume consistently high, or does it only appear on volatile days?
        2.  **Market Impact & Slippage:** Based on the data, estimate the potential slippage for a moderately sized order. Can this stock absorb institutional-level flow without significant price dislocation?
        3.  **Liquidity Risk:** Is there a risk of liquidity suddenly drying up? What does the `liquidity_score` imply?

        - Use ONLY the FACTS/NEWS provided above. Do NOT invent numbers or events.
        - STRICT JSON output only. No text outside JSON.
        - If evidence/clarity is insufficient → set "expected_5day_return": null and lower "confidence".

        REQUIRED JSON OUTPUT (Strictly adhere to this format):
        {
            "score": [0.0-10.0, precise to one decimal, where 10 = extremely deep and stable liquidity with low execution risk],
            "confidence": [0.0-1.0 confidence level, your conviction in the liquidity assessment],
            "recommendation": "[strong_buy_monday|buy_monday|hold|avoid]",
            "liquidity_tier": "[Tier 1: Excellent|Tier 2: Good|Tier 3: Adequate|Tier 4: Poor/Avoid]",
            "estimated_slippage_bps": [Your estimate of slippage in basis points (e.g., 5 bps)],
            "max_recommended_position_usd": [The maximum single order size in USD you would recommend before expecting significant market impact],
            "key_findings": {
                "liquidity_strength": "The primary evidence of good liquidity (e.g., 'Consistently high volume above $500M daily value traded').",
                "execution_warning": "The most significant liquidity risk (e.g., 'Volume dries up significantly during midday')."
            },
            "rationale": "Provide a concise summary on the ease and safety of executing trades for this stock."
        }
        """

    elif agent_name == "risk":
        specific_prompt = """
        PERSONA: You are the Chief Risk Officer (CRO) of a hedge fund. Your primary mandate is capital preservation. You are paid to be paranoid and to identify risks that the bullish analysts might have missed.

        MISSION: Conduct a comprehensive 5-day risk assessment based on the provided data. Your goal is to uncover any and all factors that could lead to a significant loss. Your analysis MUST be grounded in the data provided.

        ANALYTICAL FRAMEWORK (Follow these steps):
        1.  **Quantitative Downside:** Analyze the `five_day_var` and `sortino_ratio`. What do these specific numbers from the data tell you about the stock's downside potential?
        2.  **Volatility:** Is the `volatility_spike` flag `true` or `false`? How does this impact your risk assessment?
        3.  **Risk vs. Reward:** Compare the `predicted_5day_return` against the `five_day_var`. Does the potential reward justify the quantifiable risk shown in the data?

        - Use ONLY the FACTS/NEWS provided above. Do NOT invent numbers or events.
        - STRICT JSON output only. No text outside JSON.
        - If evidence/clarity is insufficient → set "expected_5day_return": null and lower "confidence".
        - SAFETY scoring: 10 = very low risk (safer), 0 = extremely high risk (dangerous).

        REQUIRED JSON OUTPUT (Score is inverted: 10 = very low risk):
        {
            "score": [0.0-10.0, precise to one decimal, where 10 means VERY LOW RISK],
            "confidence": [0.0-1.0 confidence level, your conviction in the risk assessment],
            "recommendation": "[Approve|Approve with smaller size|Reject]",
            "risk_profile": "[Low|Moderate|High|Extreme|Unacceptable]",
            "realistic_disaster_scenario_loss": [A realistic potential percentage loss if the worst-case scenario occurs this week],
            "key_findings": {
                "most_dangerous_risk": "The single greatest risk factor, citing a specific data point (e.g., 'The 5-day VaR of -11.0% is unacceptably high').",
                "mitigating_factor": "The strongest factor that provides a safety buffer, citing a specific data point (e.g., 'The volatility_spike flag is false')."
            },
            "rationale": "Provide a CRO-level summary, referencing the key data points that led to your score. State clearly whether you approve or reject this trade from a risk perspective."
        }
        """

    elif agent_name == "timing":
        specific_prompt = """
        PERSONA: You are the Head of Algorithmic Strategy, specializing in short-term alpha capture. Your focus is on identifying repeatable intraday and weekly patterns to optimize trade entry and exit for a statistical edge.

        MISSION: Analyze the provided intraday and daily data to define the optimal timing strategy for executing a 5-day swing trade on this specific stock.

        ANALYTICAL FRAMEWORK (Follow these steps):
        1.  **Weekly Seasonality:** Does this stock's historical data suggest a pattern of being stronger early in the week versus later, or vice-versa?
        2.  **Intraday Patterns:** Analyze the `intraday_patterns` data. Does the stock typically experience a morning fade or a late-day rally? What does the `best_hour_to_buy` data suggest?
        3.  **Optimal Window:** Based on the patterns, define the ideal time window to execute entry and exit for the highest probability of success.

        - Use ONLY the FACTS/NEWS provided above. Do NOT invent numbers or events.
        - STRICT JSON output only. No text outside JSON.
        - If evidence/clarity is insufficient → set "expected_5day_return": null and lower "confidence".

        REQUIRED JSON OUTPUT (Strictly adhere to this format):
        {
            "score": [0.0-10.0, precise to one decimal, representing the strength and clarity of the timing pattern],
            "confidence": [0.0-1.0 confidence level, your conviction in this timing strategy],
            "recommendation": "[strong_buy_monday|buy_monday|hold|avoid]",
            "timing_edge": "[Significant|Moderate|Minor|None]",
            "optimal_entry_window": "Describe the ideal entry window and condition (e.g., 'Monday 9:45-10:30 AM ET, buying a dip towards the opening price').",
            "optimal_exit_window": "Describe the ideal exit window and condition (e.g., 'Friday 3:30-4:00 PM ET, selling into closing strength').",
            "key_findings": {
                "weekly_pattern_observed": "The dominant historical pattern for the week.",
                "intraday_pattern_observed": "The most common intraday behavior based on provided data."
            },
            "rationale": "Provide a concise justification for the recommended entry and exit windows based on the observed patterns."
        }
        """

    return base_context + specific_prompt

def get_current_time_et():
    """Get current time in Eastern timezone"""
    eastern = pytz.timezone('US/Eastern')
    return datetime.now(eastern)

# === UTILITY FUNCTIONS ===
def get_current_trading_week_dates():
   """Get Monday and Friday of current trading week with proper timezone handling"""
   eastern = pytz.timezone('US/Eastern')
   now_et = datetime.now(eastern)

   days_since_monday = now_et.weekday()
   monday = now_et - timedelta(days=days_since_monday)
   monday = monday.replace(hour=9, minute=30, second=0, microsecond=0)

   friday = monday + timedelta(days=4)
   friday = friday.replace(hour=16, minute=0, second=0, microsecond=0)

   return monday, friday

def is_trading_day(date_obj):
   """Check if date is a trading day"""
   if hasattr(date_obj, 'weekday'):
       return date_obj.weekday() < 5
   return True

def safe_datetime_comparison(dt1, dt2):
   """Safely compare datetime objects with timezone handling"""
   try:
       if dt1.tzinfo is None:
           dt1 = dt1.replace(tzinfo=timezone.utc)
       if dt2.tzinfo is None:
           dt2 = dt2.replace(tzinfo=timezone.utc)
       return dt1 >= dt2
   except Exception:
       return str(dt1) >= str(dt2)

# === DATA MODELS ===
class DataSourceInfo(BaseModel):
   """Track data sources for transparency"""
   source_name: str
   source_url: Optional[str] = None
   api_endpoint: Optional[str] = None
   data_timestamp: str = Field(default_factory=lambda: datetime.now(timezone.utc).isoformat())
   success: bool = True
   error_message: Optional[str] = None

class ShortTermPriceData(BaseModel):
   symbol: str
   current_price: Optional[float] = None

   monday_price: Optional[float] = None
   friday_price: Optional[float] = None
   week_high: Optional[float] = None
   week_low: Optional[float] = None
   week_to_date_return: Optional[float] = None

   predicted_friday_price: Optional[float] = None
   predicted_5day_return: Optional[float] = None
   prediction_confidence: Optional[float] = None

   daily_momentum: Optional[Dict[str, Any]] = None
   intraday_patterns: Optional[Dict[str, Any]] = None
   short_term_data: List[Dict[str, Any]] = Field(default_factory=list)

   data_sources: List[DataSourceInfo] = Field(default_factory=list)
   error: Optional[str] = None

class NewsCatalystData(BaseModel):
   symbol: str
   catalyst_strength: Optional[str] = None

   breaking_news: List[Dict[str, Any]] = Field(default_factory=list)
   earnings_calendar: Optional[Dict[str, Any]] = None
   analyst_updates: List[Dict[str, Any]] = Field(default_factory=list)

   weekly_sentiment_trend: Optional[Dict[str, Any]] = None
   social_media_buzz: Optional[Dict[str, Any]] = None
   upcoming_events: List[str] = Field(default_factory=list)

   data_sources: List[DataSourceInfo] = Field(default_factory=list)
   error: Optional[str] = None

class TechnicalSwingData(BaseModel):
   symbol: str

   swing_signals: Dict[str, Any] = Field(default_factory=dict)
   momentum_strength: Optional[str] = None

   suggested_entry_level: Optional[float] = None
   suggested_exit_level: Optional[float] = None
   stop_loss_level: Optional[float] = None

   chart_patterns: List[str] = Field(default_factory=list)
   volume_confirmation: Optional[bool] = None

   data_sources: List[DataSourceInfo] = Field(default_factory=list)
   error: Optional[str] = None

class VolumeLiquidityData(BaseModel):
   symbol: str

   average_daily_volume: Optional[float] = None
   volume_trend: Optional[str] = None
   liquidity_score: Optional[float] = None

   bid_ask_analysis: Optional[Dict[str, Any]] = None
   volume_spikes: List[Dict[str, Any]] = Field(default_factory=list)
   institutional_flow: Optional[Dict[str, Any]] = None
   execution_difficulty: Optional[str] = None

   data_sources: List[DataSourceInfo] = Field(default_factory=list)
   error: Optional[str] = None

class ShortTermRiskData(BaseModel):
   symbol: str

   five_day_var: Optional[float] = None
   max_intraweek_loss: Optional[float] = None

   optimal_volatility_range: Optional[Dict[str, float]] = None
   current_volatility_vs_optimal: Optional[str] = None

   tech_sector_beta: Optional[float] = None
   correlation_with_qqq: Optional[float] = None
   risk_reward_ratio: Optional[float] = None

   data_sources: List[DataSourceInfo] = Field(default_factory=list)
   error: Optional[str] = None

class EntryExitTimingData(BaseModel):
   symbol: str

   best_entry_time: Optional[str] = None
   best_exit_time: Optional[str] = None
   intraday_patterns: Optional[Dict[str, Any]] = None

   recommended_position_size: Optional[float] = None
   optimal_holding_days: Optional[int] = None
   success_probability: Optional[float] = None

   data_sources: List[DataSourceInfo] = Field(default_factory=list)
   error: Optional[str] = None

class ShortTermStockAnalysis(BaseModel):
   symbol: str
   company_name: Optional[str] = None
   sector: str = "Technology"
   analysis_timestamp: str = Field(default_factory=lambda: datetime.now(timezone.utc).isoformat())

   trading_week: Optional[Dict[str, Any]] = None

   price_data: Optional[Dict[str, Any]] = None
   news_catalyst_data: Optional[Dict[str, Any]] = None
   technical_swing_data: Optional[Dict[str, Any]] = None
   volume_liquidity_data: Optional[Dict[str, Any]] = None
   short_term_risk_data: Optional[Dict[str, Any]] = None
   timing_data: Optional[Dict[str, Any]] = None

   momentum_agent_analysis: Optional[Dict[str, Any]] = None
   catalyst_agent_analysis: Optional[Dict[str, Any]] = None
   swing_agent_analysis: Optional[Dict[str, Any]] = None
   liquidity_agent_analysis: Optional[Dict[str, Any]] = None
   risk_agent_analysis: Optional[Dict[str, Any]] = None
   timing_agent_analysis: Optional[Dict[str, Any]] = None

   agent_scores: Dict[str, float] = Field(default_factory=dict)
   weekly_opportunity_score: float = 0.0
   final_recommendation: Optional[str] = None

   all_data_sources: List[DataSourceInfo] = Field(default_factory=list)

class ShortTermPortfolioRecommendation(BaseModel):
   selected_stocks: List[Dict[str, Any]]

   trading_plan: Dict[str, Any] = Field(default_factory=dict)
   weekly_portfolio_analytics: Dict[str, Any] = Field(default_factory=dict)
   expected_weekly_returns: Dict[str, Any] = Field(default_factory=dict)
   weekly_risk_metrics: Dict[str, Any] = Field(default_factory=dict)
   market_timing_insights: Dict[str, Any] = Field(default_factory=dict)
   performance_tracking: Dict[str, Any] = Field(default_factory=dict)

   generation_timestamp: str = Field(default_factory=lambda: datetime.now(timezone.utc).isoformat())
   processing_stats: Dict[str, Any] = Field(default_factory=dict)
   all_data_sources: List[DataSourceInfo] = Field(default_factory=list)

   disclaimer: str = (
       "⚠️ SHORT-TERM TRADING AI ANALYSIS - HIGH RISK STRATEGY. "
       "Designed for 5-day holding periods (Monday-Friday). "
       "NOT SUITABLE FOR ALL INVESTORS. Consult professionals before trading."
   )

class ShortTermAnalysisState(TypedDict):
   target_symbols: List[str]
   current_batch_index: int
   all_stocks_data: Dict[str, ShortTermStockAnalysis]
   weekly_portfolio: Optional[ShortTermPortfolioRecommendation]
   error_messages: List[str]
   batch_completed: bool
   total_batches: int
   analysis_start_time: float
   trading_week_context: Dict[str, Any]

# === DATA COLLECTION FUNCTIONS ===

async def fetch_short_term_price_data(symbol: str, period: str = "3mo") -> ShortTermPriceData:
   """Fetch comprehensive price data with predictions and source tracking"""
   yahoo_url = f"https://finance.yahoo.com/quote/{symbol}"
   yfinance_api = f"yfinance.Ticker('{symbol}').history(period='{period}')"
   print(f"📈 [SHORT-TERM PRICE] Fetching data for {symbol}...")
   print(f"🔗 Yahoo Finance URL: {yahoo_url}")
   print(f"🔗 API Call: {yfinance_api}")
   await asyncio.sleep(API_CALL_DELAY_YFINANCE)

   data_sources = []
   yf_source = DataSourceInfo(
       source_name="Yahoo Finance",
       source_url=f"https://finance.yahoo.com/quote/{symbol}",
       api_endpoint=f"yfinance.Ticker('{symbol}').history(period='{period}')"
   )

   try:
       stock = yf.Ticker(symbol)
       hist_3m = stock.history(period=period)

       if hist_3m.empty:
           yf_source.success = False
           yf_source.error_message = "No historical data available"
           return ShortTermPriceData(
               symbol=symbol,
               error="No historical data",
               data_sources=[yf_source]
           )

       yf_source.success = True
       data_sources.append(yf_source)

       # Current price
       current_price = float(hist_3m['Close'].iloc[-1])

       # Trading week dates
       monday, friday = get_current_trading_week_dates()

       # Handle timezone conversion safely
       if hist_3m.index.tz is None:
           hist_3m.index = hist_3m.index.tz_localize('US/Eastern')
       else:
           hist_3m.index = hist_3m.index.tz_convert('US/Eastern')

       # Get current week data
       monday_naive = monday.replace(tzinfo=None)
       current_week_data = hist_3m[hist_3m.index.tz_localize(None) >= monday_naive - timedelta(days=7)]

       monday_price = None
       friday_price = None
       week_high = None
       week_low = None

       if not current_week_data.empty:
           monday_price = float(current_week_data['Open'].iloc[0])
           friday_price = current_price
           week_high = float(current_week_data['High'].max())
           week_low = float(current_week_data['Low'].min())

       # Week-to-date return
       week_to_date_return = 0.0
       if monday_price and current_price:
           week_to_date_return = ((current_price - monday_price) / monday_price) * 100

       # === ENHANCED PRICE PREDICTION ===
       # === AI-POWERED PRICE PREDICTION ===
       predicted_friday_price = None
       predicted_5day_return = None
       prediction_confidence = None

       if len(hist_3m) >= 20:
           try:
               ai_prediction = await get_ai_price_prediction(symbol, hist_3m, current_price)
               predicted_friday_price = ai_prediction['predicted_friday_price']
               predicted_5day_return = ai_prediction['predicted_5day_return']
               prediction_confidence = ai_prediction['prediction_confidence']

               print(f"🤖 AI Price Prediction for {symbol}: ${current_price:.2f} → ${predicted_friday_price:.2f} ({predicted_5day_return:+.1f}%)")

           except Exception as e:
               print(f"❌ AI price prediction failed for {symbol}: {e}")
               raise Exception(f"AI price prediction required but failed: {e}")
       else:
           raise Exception(f"Insufficient data for AI prediction: {symbol}")

       # Daily momentum analysis
       daily_returns = hist_3m['Close'].pct_change().dropna()
       daily_momentum = {
           "avg_daily_return": round(daily_returns.mean() * 100, 3),
           "daily_volatility": round(daily_returns.std() * 100, 3),
           "momentum_trend": "positive" if daily_returns.tail(5).mean() > 0 else "negative",
           "recent_5d_avg": round(daily_returns.tail(5).mean() * 100, 3),
           "momentum_strength": "strong" if abs(daily_returns.tail(5).mean()) > daily_returns.std() else "weak",
           "trend_consistency": round(min(abs(daily_returns.tail(5).mean()) / (daily_returns.std() + 0.001), 2.0), 2)
       }

       # Fixed intraday patterns (แก้ไข error อย่างสมบูรณ์)
       intraday_patterns = {}
       try:
           hist_5d = stock.history(period="5d", interval="1h")
           if not hist_5d.empty and len(hist_5d) > 10:
               hist_5d = hist_5d.copy()  # Create a copy to avoid SettingWithCopyWarning

               # แก้ไข error: object of type 'numpy.float64' has no len()
               # ต้องใช้ .index.hour แทน hist_5d['hour']
               hours = hist_5d.index.hour

               # สร้าง DataFrame ใหม่ที่มี hour column
               hourly_data = pd.DataFrame({
                   'hour': hours,
                   'close': hist_5d['Close'].values,
                   'volume': hist_5d['Volume'].values
               })

               # คำนวณ hourly returns และ volume
               hourly_data['return'] = hourly_data['close'].pct_change()

               # Group by hour และคำนวณ mean
               hourly_stats = hourly_data.groupby('hour').agg({
                   'return': 'mean',
                   'volume': 'mean'
               })

               # ตรวจสอบว่ามีข้อมูลมากพอ
               if len(hourly_stats) > 0 and not hourly_stats['return'].isna().all():
                   # หา best hours (แปลงเป็น int เพื่อป้องกัน numpy type issues)
                   best_buy_hour = int(hourly_stats['return'].idxmin()) if not hourly_stats['return'].isna().all() else 10
                   best_sell_hour = int(hourly_stats['return'].idxmax()) if not hourly_stats['return'].isna().all() else 15

                   # คำนวณ momentum สำหรับช่วงเวลาต่างๆ
                   morning_hours = hourly_stats[(hourly_stats.index >= 9) & (hourly_stats.index <= 12)]
                   afternoon_hours = hourly_stats[(hourly_stats.index >= 13) & (hourly_stats.index <= 16)]

                   morning_momentum = float(morning_hours['return'].mean()) * 100 if len(morning_hours) > 0 else 0.0
                   afternoon_momentum = float(afternoon_hours['return'].mean()) * 100 if len(afternoon_hours) > 0 else 0.0

                   # หา volume peak hour
                   volume_peak_hour = int(hourly_stats['volume'].idxmax()) if not hourly_stats['volume'].isna().all() else 10

                   intraday_patterns = {
                       "best_hour_to_buy": best_buy_hour,
                       "best_hour_to_sell": best_sell_hour,
                       "morning_momentum": round(morning_momentum, 3),
                       "afternoon_momentum": round(afternoon_momentum, 3),
                       "volume_peak_hour": volume_peak_hour,
                       "optimal_entry_window": "09:30-10:30" if morning_momentum < 0 else "10:30-11:30"
                   }
               else:
                   # ถ้าไม่มีข้อมูลเพียงพอ ให้ใช้ default values
                   intraday_patterns = {
                       "best_hour_to_buy": 10,
                       "best_hour_to_sell": 15,
                       "morning_momentum": 0.0,
                       "afternoon_momentum": 0.0,
                       "volume_peak_hour": 10,
                       "optimal_entry_window": "10:00-11:00"
                   }

       except Exception as e:
           print(f"⚠️ Intraday pattern error for {symbol}: {e}")
           # Default intraday patterns
           intraday_patterns = {
               "best_hour_to_buy": 10,
               "best_hour_to_sell": 15,
               "morning_momentum": 0.0,
               "afternoon_momentum": 0.0,
               "volume_peak_hour": 10,
               "optimal_entry_window": "10:00-11:00"
           }

       # Technical data preparation
       short_term_data = []
       for idx, row in hist_3m.tail(60).iterrows():
           record = {
               "date": idx.strftime('%Y-%m-%d') if hasattr(idx, 'strftime') else str(idx),
               "open": float(row['Open']),
               "high": float(row['High']),
               "low": float(row['Low']),
               "close": float(row['Close']),
               "volume": float(row['Volume'])
           }
           short_term_data.append(record)

       print(f"✅ Enhanced price data for {symbol}:")
       print(f"   📊 Current: ${current_price:.2f} | Predicted Friday: ${predicted_friday_price:.2f}")
       print(f"   📈 Expected 5-day return: {predicted_5day_return:+.1f}% (confidence: {prediction_confidence:.1%})")
       print(f"   📅 Week return: {week_to_date_return:+.1f}% | Momentum: {daily_momentum['momentum_trend']}")

       return ShortTermPriceData(
           symbol=symbol,
           current_price=current_price,
           monday_price=monday_price,
           friday_price=friday_price,
           week_high=week_high,
           week_low=week_low,
           week_to_date_return=week_to_date_return,
           predicted_friday_price=predicted_friday_price,
           predicted_5day_return=predicted_5day_return,
           prediction_confidence=prediction_confidence,
           daily_momentum=daily_momentum,
           intraday_patterns=intraday_patterns,
           short_term_data=short_term_data,
           data_sources=data_sources
       )

   except Exception as e:
       error_msg = f"Error fetching price data: {e}"
       print(f"❌ {error_msg} for {symbol}")
       yf_source.success = False
       yf_source.error_message = str(e)
       return ShortTermPriceData(
           symbol=symbol,
           error=error_msg,
           data_sources=[yf_source]
       )

async def get_ai_price_prediction(symbol: str, historical_data: pd.DataFrame,
                                current_price: float) -> Dict[str, Any]:
    """ใช้ AI วิเคราะห์และคาดการณ์ราคา"""

    if not GROQ_API_KEY:
        raise Exception("AI price prediction requires GROQ API key")

    # เตรียมข้อมูลสำหรับ AI
    recent_data = historical_data.tail(30).to_dict('records')

    # คำนวณ basic indicators สำหรับ AI
    technical_indicators = {
        'sma_10': float(historical_data['Close'].tail(10).mean()),
        'sma_20': float(historical_data['Close'].tail(20).mean()),
        'rsi_14': calculate_rsi(historical_data['Close'], 14),
        'volatility': float(historical_data['Close'].pct_change().std() * 100),
        'volume_trend': 'increasing' if historical_data['Volume'].tail(5).mean() > historical_data['Volume'].tail(10).mean() else 'decreasing',
        'price_trend': 'bullish' if current_price > historical_data['Close'].tail(10).mean() else 'bearish'
    }

    prediction_prompt = f"""
AI PRICE PREDICTION SPECIALIST

SYMBOL: {symbol}
CURRENT_PRICE: ${current_price:.2f}
ANALYSIS_DATE: {datetime.now().strftime('%Y-%m-%d')}

RECENT_PRICE_DATA (Last 30 days):
{json.dumps(recent_data[-10:], default=str, indent=2)}

TECHNICAL_INDICATORS:
- SMA 10: ${technical_indicators['sma_10']:.2f}
- SMA 20: ${technical_indicators['sma_20']:.2f}
- RSI 14: {technical_indicators['rsi_14']:.1f}
- Volatility: {technical_indicators['volatility']:.1f}%
- Volume Trend: {technical_indicators['volume_trend']}
- Price Trend: {technical_indicators['price_trend']}

MISSION: Predict precise 5-day price movement for Monday-to-Friday swing trading

ANALYSIS REQUIREMENTS:
1. Analyze price momentum and trend patterns from historical data
2. Evaluate volume-price relationships and momentum
3. Assess technical indicator signals (SMA, RSI, volatility)
4. Consider market microstructure and trading patterns
5. Calculate probability-weighted price targets with confidence

REQUIRED JSON OUTPUT:
{{
    "predicted_friday_price": [precise numeric value, e.g., 210.55],
    "predicted_5day_return": [precise numeric value for percentage, e.g., 3.75 or -1.5],
    "prediction_confidence": [0.0-1.0 numeric confidence level],
    "price_targets": {{
        "conservative": [numeric value],
        "most_likely": [numeric value],
        "optimistic": [numeric value]
    }},
    "key_factors": [
        "primary technical factor driving prediction",
        "momentum/trend supporting factor",
        "risk factor to monitor"
    ],
    "probability_analysis": {{
        "upside_probability": [0.0-1.0 numeric probability],
        "neutral_probability": [0.0-1.0 numeric probability],
        "downside_probability": [0.0-1.0 numeric probability]
    }},
    "rationale": "detailed explanation of prediction methodology and reasoning"
}}

CRITICAL INSTRUCTION: The value for 'predicted_5day_return' MUST be a raw number (float). DO NOT include the '%' symbol. For example, for a 5.25% return, the value should be 5.25.

ANALYZE THE DATA AND PREDICT NOW:
"""

    try:
        headers = {
            "Authorization": f"Bearer {GROQ_API_KEY}",
            "Content-Type": "application/json"
        }

        data = {
            "model": "llama-3.3-70b-versatile",
            "messages": [
                {"role": "system", "content": "You are an AI Price Prediction Specialist with advanced quantitative analysis capabilities and deep understanding of technical analysis and market patterns."},
                {"role": "user", "content": prediction_prompt}
            ],
            "temperature": 0.2,
            "max_tokens": 1500,
            "response_format": {"type": "json_object"}
        }

        async with httpx.AsyncClient(timeout=45.0) as client:
            response = await client.post(
                "https://api.groq.com/openai/v1/chat/completions",
                headers=headers,
                json=data
            )

            if response.status_code == 200:
                result = response.json()
                content = result["choices"][0]["message"]["content"]
                ai_prediction = json.loads(content)

                # Validate prediction
                validated_prediction = validate_price_prediction(ai_prediction, current_price)
                return validated_prediction

            else:
                raise Exception(f"API Error {response.status_code}: {response.text}")

    except Exception as e:
        print(f"❌ AI price prediction API failed: {e}")
        raise Exception(f"AI price prediction failed: {e}")

def calculate_rsi(prices, period=14):
    """Calculate RSI indicator"""
    try:
        delta = prices.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        return float(rsi.iloc[-1])
    except:
        return 50.0  # neutral RSI

def validate_price_prediction(prediction: Dict, current_price: float) -> Dict[str, Any]:
    """Validate AI price prediction"""

    try:
        predicted_price = float(prediction['predicted_friday_price'])
        predicted_return = float(prediction['predicted_5day_return'])
        confidence = float(prediction['prediction_confidence'])

        # Sanity checks
        if predicted_price <= 0:
            raise ValueError("Invalid predicted price")

        if abs(predicted_return) > 50:  # มากกว่า 50% ใน 5 วันไม่น่าเป็นไปได้
            print(f"⚠️ Unrealistic return prediction: {predicted_return}%, capping at ±25%")
            predicted_return = max(-25, min(25, predicted_return))
            predicted_price = current_price * (1 + predicted_return/100)

        if not (0 <= confidence <= 1):
            confidence = max(0, min(1, confidence))

        return {
            'predicted_friday_price': round(predicted_price, 2),
            'predicted_5day_return': round(predicted_return, 2),
            'prediction_confidence': round(confidence, 3),
            'price_targets': prediction.get('price_targets', {}),
            'key_factors': prediction.get('key_factors', []),
            'probability_analysis': prediction.get('probability_analysis', {}),
            'rationale': prediction.get('rationale', ''),
            'ai_generated': True,
            'prediction_method': 'ai_analysis'
        }

    except Exception as e:
        raise Exception(f"Invalid AI prediction format: {e}")

def _validate_ai_response(self, response: Dict) -> bool:
    """Validate AI response format"""
    required_fields = ['score', 'confidence', 'recommendation']

    for field in required_fields:
        if field not in response:
            return False

    # Validate score
    try:
        score = float(response['score'])
        if not (0 <= score <= 10):
            return False
    except:
        return False

    # Validate confidence
    try:
        confidence = float(response['confidence'])
        if not (0 <= confidence <= 1):
            return False
    except:
        return False

    # Validate recommendation
    valid_recommendations = [
        'strong_buy_monday', 'buy_monday', 'moderate_buy_monday',
        'hold', 'avoid', 'strong_avoid'
    ]
    if response['recommendation'] not in valid_recommendations:
        return False

    return True

def _extract_json_from_mixed_response(self, content: str) -> Dict[str, Any]:
    """Extract JSON from mixed text response"""
    import re

    # หา JSON block
    json_pattern = r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}'
    matches = re.findall(json_pattern, content)

    for match in matches:
        try:
            parsed = json.loads(match)
            if self._validate_ai_response(parsed):
                return parsed
        except:
            continue

    # ถ้าไม่เจอ JSON ให้ parse จาก text
    return self._parse_response_from_text(content)

def _parse_response_from_text(self, content: str) -> Dict[str, Any]:
    """Parse response from plain text"""
    import re

    # Extract score
    score_match = re.search(r'(?:score|rating)[:=]\s*([0-9.]+)', content, re.IGNORECASE)
    score = float(score_match.group(1)) if score_match else 6.0

    # Extract confidence
    conf_match = re.search(r'confidence[:=]\s*([0-9.]+)', content, re.IGNORECASE)
    confidence = float(conf_match.group(1)) if conf_match else 0.7

    # Extract recommendation
    recommendation = "moderate_buy_monday"  # Default
    content_lower = content.lower()

    if 'strong buy' in content_lower or 'strong_buy' in content_lower:
        recommendation = "strong_buy_monday"
    elif 'buy' in content_lower:
        recommendation = "buy_monday"
    elif 'avoid' in content_lower or 'sell' in content_lower:
        recommendation = "avoid"
    elif 'hold' in content_lower:
        recommendation = "hold"

    # Extract insights from content
    insights = []
    sentences = [s.strip() for s in content.split('.') if len(s.strip()) > 20]
    for sentence in sentences[:3]:
        if any(keyword in sentence.lower() for keyword in
              ['analysis', 'trend', 'momentum', 'signal', 'pattern', 'technical', 'bullish', 'bearish']):
            insights.append(sentence)

    if not insights:
        insights = ['AI text analysis completed', 'Analysis based on available data']

    return {
        'score': max(0.0, min(10.0, score)),
        'confidence': max(0.0, min(1.0, confidence)),
        'recommendation': recommendation,
        'key_insights': insights,
        'rationale': f'Text-parsed AI analysis with score {score:.1f}/10',
        'text_parsed': True,
        'ai_powered': True
    }

async def fetch_news_catalyst_data(symbol: str, deps) -> NewsCatalystData:
   """Fetch comprehensive news data with source tracking"""
   print(f"📰 [NEWS CATALYST] Fetching catalyst data for {symbol}...")

   # Alpha Vantage News
   if deps.alpha_vantage_key:
       av_url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={symbol}&limit=10&sort=LATEST&apikey={deps.alpha_vantage_key}"
       print(f"🔗 Alpha Vantage News API: {av_url}")

   # News API fallback
   if NEWS_API_KEY:
       news_url = f"https://newsapi.org/v2/everything?q={symbol}&sortBy=publishedAt&apiKey={NEWS_API_KEY}&pageSize=10"
       print(f"🔗 NewsAPI URL: {news_url}")

   # Yahoo Finance News
   yf_news_url = f"https://finance.yahoo.com/quote/{symbol}/news"
   print(f"🔗 Yahoo Finance News: {yf_news_url}")

   result = NewsCatalystData(symbol=symbol)
   data_sources = []

   try:
       # Alpha Vantage News
       breaking_news = []
       if deps.alpha_vantage_key:
           await asyncio.sleep(API_CALL_DELAY_ALPHA_VANTAGE)
           av_url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={symbol}&limit=20&sort=LATEST&apikey={deps.alpha_vantage_key}"

           av_source = DataSourceInfo(
               source_name="Alpha Vantage News API",
               source_url="https://www.alphavantage.co/documentation/#news-sentiment",
               api_endpoint=av_url
           )

           try:
               av_response = await deps.client.get(av_url)
               av_data = av_response.json()

               if "feed" in av_data:
                   av_source.success = True
                   for article in av_data["feed"][:10]:
                       title = article.get("title", "").lower()
                       summary = article.get("summary", "").lower()

                       # Catalyst keywords
                       catalyst_keywords = [
                           "earnings", "revenue", "profit", "beat", "miss", "guidance",
                           "partnership", "acquisition", "merger", "deal", "contract",
                           "breakthrough", "launch", "release", "approval", "upgrade", "downgrade",
                           "analyst", "target", "buy", "sell", "bullish", "bearish"
                       ]

                       catalyst_strength = 0
                       for keyword in catalyst_keywords:
                           if keyword in title or keyword in summary:
                               catalyst_strength += 1

                    #    if catalyst_strength >= 2:
                       breaking_news.append({
                               "title": article.get("title", ""),
                               "summary": article.get("summary", "")[:50] + "...",
                               "time_published": article.get("time_published", ""),
                               "sentiment_score": float(article.get("overall_sentiment_score", 0.0)),
                               "relevance_score": float(article.get("relevance_score", 0.0)),
                               "catalyst_strength": catalyst_strength,
                               "source": article.get("source", ""),
                               "url": article.get("source_domain", ""),
                               "data_source": "Alpha Vantage API"
                       })

               print(f"✅ Alpha Vantage: Found {len(breaking_news)} catalyst news for {symbol}")

           except Exception as e:
               av_source.success = False
               av_source.error_message = str(e)
               print(f"⚠️ Alpha Vantage News error: {e}")

           data_sources.append(av_source)

       # News API fallback
       if not breaking_news and NEWS_API_KEY:
           await asyncio.sleep(API_CALL_DELAY_NEWS)
           news_url = f"https://newsapi.org/v2/everything?q={symbol}&sortBy=publishedAt&apiKey={NEWS_API_KEY}&pageSize=10"

           news_source = DataSourceInfo(
               source_name="NewsAPI.org",
               source_url="https://newsapi.org/",
               api_endpoint=news_url
           )

           try:
               news_response = await deps.client.get(news_url)
               news_data = news_response.json()

               if "articles" in news_data:
                   news_source.success = True
                   for article in news_data["articles"][:10]:
                       breaking_news.append({
                           "title": article.get("title", ""),
                           "summary": article.get("description", "")[:150] + "..." if article.get("description") else "",
                           "time_published": article.get("publishedAt", ""),
                           "sentiment_score": 0.0,
                           "relevance_score": 1.0,
                           "catalyst_strength": 1,
                           "source": article.get("source", {}).get("name", ""),
                           "url": article.get("url", ""),
                           "data_source": "NewsAPI.org"
                       })

               print(f"✅ NewsAPI: Found {len(breaking_news)} articles for {symbol}")

           except Exception as e:
               news_source.success = False
               news_source.error_message = str(e)
               print(f"⚠️ NewsAPI error: {e}")

           data_sources.append(news_source)

       # Yahoo Finance News fallback
       if not breaking_news:
           yf_news_source = DataSourceInfo(
               source_name="Yahoo Finance News",
               source_url=f"https://finance.yahoo.com/quote/{symbol}/news",
               api_endpoint=f"yfinance.Ticker('{symbol}').news"
           )

           try:
               stock = yf.Ticker(symbol)
               news_data = stock.news

               if news_data:
                   yf_news_source.success = True
                   for article in news_data[:10]:
                       breaking_news.append({
                           "title": article.get("title", ""),
                           "summary": article.get("summary", "")[:150] + "..." if article.get("summary") else "",
                           "time_published": str(article.get("providerPublishTime", "")),
                           "sentiment_score": 0.0,
                           "relevance_score": 1.0,
                           "catalyst_strength": 1,
                           "source": article.get("publisher", ""),
                           "url": article.get("link", ""),
                           "data_source": "Yahoo Finance"
                       })

               print(f"✅ Yahoo Finance: Found {len(breaking_news)} news items for {symbol}")

           except Exception as e:
               yf_news_source.success = False
               yf_news_source.error_message = str(e)
               print(f"⚠️ Yahoo Finance News error: {e}")

           data_sources.append(yf_news_source)

       # Analyze catalyst strength
       if breaking_news:
           avg_catalyst_strength = sum(news["catalyst_strength"] for news in breaking_news) / len(breaking_news)
           avg_sentiment = sum(news["sentiment_score"] for news in breaking_news) / len(breaking_news)

           if avg_catalyst_strength >= 4 and avg_sentiment > 0.2:
               catalyst_strength = "strong"
           elif avg_catalyst_strength >= 3 or avg_sentiment > 0.1:
               catalyst_strength = "moderate"
           else:
               catalyst_strength = "weak"
       else:
           catalyst_strength = "none"

       result.breaking_news = breaking_news
       result.catalyst_strength = catalyst_strength
       result.data_sources = data_sources

       # Weekly sentiment trend
       if breaking_news:
           recent_sentiment = [news["sentiment_score"] for news in breaking_news[:5]]
           older_sentiment = [news["sentiment_score"] for news in breaking_news[5:10]]

           recent_avg = sum(recent_sentiment) / len(recent_sentiment) if recent_sentiment else 0
           older_avg = sum(older_sentiment) / len(older_sentiment) if older_sentiment else 0

           trend = "improving" if recent_avg > older_avg + 0.1 else "declining" if recent_avg < older_avg - 0.1 else "stable"

           result.weekly_sentiment_trend = {
               "current_sentiment": round(recent_avg, 3),
               "previous_sentiment": round(older_avg, 3),
               "trend": trend,
               "sentiment_momentum": round(recent_avg - older_avg, 3)
           }

       print(f"✅ News catalyst analysis for {symbol}: {catalyst_strength} strength, {len(breaking_news)} articles")

   except Exception as e:
       error_msg = f"Error fetching news data: {e}"
       print(f"❌ {error_msg} for {symbol}")
       result.error = error_msg

   return result

async def calculate_technical_swing_signals(symbol: str, price_data: ShortTermPriceData) -> TechnicalSwingData:
   """Calculate comprehensive technical signals with source tracking"""
   print(f"📊 [TECHNICAL SWING] Calculating swing signals for {symbol}...")
   print(f"🔗 Technical Analysis Library: https://github.com/twopirllc/pandas-ta")
   print(f"🔗 Indicators: RSI, MACD, Bollinger Bands, Moving Averages")
   print(f"🔗 Data Source: Yahoo Finance historical data")

   data_sources = [
       DataSourceInfo(
           source_name="pandas_ta Technical Analysis Library",
           source_url="https://github.com/twopirllc/pandas-ta",
           api_endpoint="pandas_ta indicators (RSI, MACD, Bollinger Bands, etc.)"
       )
   ]

   if price_data.error or not price_data.short_term_data:
       data_sources[0].success = False
       data_sources[0].error_message = "Missing price data"
       return TechnicalSwingData(
           symbol=symbol,
           error="Missing price data",
           data_sources=data_sources
       )

   try:
       # Create DataFrame
       df = pd.DataFrame(price_data.short_term_data)
       df['date'] = pd.to_datetime(df['date'])
       df = df.set_index('date')

       # Technical indicators
       df['sma_5'] = df['close'].rolling(5).mean()
       df['sma_10'] = df['close'].rolling(10).mean()
       df['sma_20'] = df['close'].rolling(20).mean()
       df['ema_8'] = df['close'].ewm(span=8).mean()
       df['ema_13'] = df['close'].ewm(span=13).mean()

       # RSI
       df['rsi'] = ta.rsi(df['close'], length=14)

       # MACD
       try:
           macd = ta.macd(df['close'], fast=8, slow=17, signal=9)
           if macd is not None and not macd.empty:
               df['macd'] = macd.iloc[:, 0]
               df['macd_signal'] = macd.iloc[:, 1]
               df['macd_hist'] = macd.iloc[:, 2]
       except Exception:
           df['macd'] = df['close'].ewm(span=8).mean() - df['close'].ewm(span=17).mean()
           df['macd_signal'] = df['macd'].ewm(span=9).mean()
           df['macd_hist'] = df['macd'] - df['macd_signal']

       # Bollinger Bands
       try:
           bbands = ta.bbands(df['close'], length=20, std=2)
           if bbands is not None and not bbands.empty:
               df['bb_upper'] = bbands.iloc[:, 0]
               df['bb_middle'] = bbands.iloc[:, 1]
               df['bb_lower'] = bbands.iloc[:, 2]
       except Exception:
           df['bb_middle'] = df['close'].rolling(20).mean()
           std = df['close'].rolling(20).std()
           df['bb_upper'] = df['bb_middle'] + (std * 2)
           df['bb_lower'] = df['bb_middle'] - (std * 2)

       # Volume indicators
       df['volume_sma'] = df['volume'].rolling(20).mean()
       df['volume_ratio'] = df['volume'] / df['volume_sma']

       # Latest data
       latest = df.iloc[-1]
       current_price = price_data.current_price or latest['close']

       # === SWING TRADING SIGNALS ANALYSIS ===
       swing_signals = {}
       bullish_signals = 0
       bearish_signals = 0

       # 1. Moving Average Signals
       if latest['close'] > latest['sma_5'] > latest['sma_10']:
           swing_signals['ma_trend'] = "bullish"
           bullish_signals += 1
       elif latest['close'] < latest['sma_5'] < latest['sma_10']:
           swing_signals['ma_trend'] = "bearish"
           bearish_signals += 1
       else:
           swing_signals['ma_trend'] = "neutral"

       # 2. RSI Signals
       rsi_current = latest['rsi'] if not pd.isna(latest['rsi']) else 50
       if rsi_current < 35:
           swing_signals['rsi_signal'] = "oversold_buy"
           bullish_signals += 1
       elif rsi_current > 65:
           swing_signals['rsi_signal'] = "overbought_sell"
           bearish_signals += 1
       else:
           swing_signals['rsi_signal'] = "neutral"

       # 3. MACD Signals
       if not pd.isna(latest['macd']) and not pd.isna(latest['macd_signal']):
           if latest['macd'] > latest['macd_signal'] and latest['macd_hist'] > 0:
               swing_signals['macd_signal'] = "bullish"
               bullish_signals += 1
           elif latest['macd'] < latest['macd_signal'] and latest['macd_hist'] < 0:
               swing_signals['macd_signal'] = "bearish"
               bearish_signals += 1
           else:
               swing_signals['macd_signal'] = "neutral"

       # 4. Bollinger Bands Signals
       if not pd.isna(latest['bb_upper']) and not pd.isna(latest['bb_lower']):
           bb_position = (current_price - latest['bb_lower']) / (latest['bb_upper'] - latest['bb_lower'])
           if bb_position < 0.2:
               swing_signals['bb_signal'] = "near_lower_band_buy"
               bullish_signals += 1
           elif bb_position > 0.8:
               swing_signals['bb_signal'] = "near_upper_band_sell"
               bearish_signals += 1
           else:
               swing_signals['bb_signal'] = "middle_range"

       # 5. Volume Confirmation
       volume_confirmation = latest['volume_ratio'] > 1.2 if not pd.isna(latest['volume_ratio']) else False
       swing_signals['volume_confirmation'] = volume_confirmation

       # === ENTRY/EXIT LEVELS ===
       recent_highs = df['high'].tail(20)
       recent_lows = df['low'].tail(20)

       resistance_level = recent_highs.max()
       support_level = recent_lows.min()

       # Entry level based on signals
       if bullish_signals > bearish_signals:
           suggested_entry_level = current_price * 1.005
           suggested_exit_level = resistance_level * 0.98
           stop_loss_level = support_level * 1.02
       else:
           suggested_entry_level = support_level * 1.01
           suggested_exit_level = current_price * 1.03
           stop_loss_level = support_level * 0.98

       # === MOMENTUM STRENGTH ===
       total_signals = bullish_signals + bearish_signals
       if total_signals == 0:
           momentum_strength = "neutral"
       else:
           bullish_ratio = bullish_signals / total_signals
           if bullish_ratio >= 0.8:
               momentum_strength = "strong_bullish"
           elif bullish_ratio >= 0.6:
               momentum_strength = "bullish"
           elif bullish_ratio <= 0.2:
               momentum_strength = "strong_bearish"
           elif bullish_ratio <= 0.4:
               momentum_strength = "bearish"
           else:
               momentum_strength = "neutral"

       # === CHART PATTERNS ===
       chart_patterns = []

       # Breakout patterns
       if current_price > resistance_level * 0.999:
           chart_patterns.append("resistance_breakout")
       if current_price < support_level * 1.001:
           chart_patterns.append("support_breakdown")

       # Bollinger squeeze
       if not pd.isna(latest['bb_upper']) and not pd.isna(latest['bb_lower']):
           bb_width = (latest['bb_upper'] - latest['bb_lower']) / latest['bb_middle']
           bb_avg_width = ((df['bb_upper'] - df['bb_lower']) / df['bb_middle']).tail(20).mean()
           if bb_width < bb_avg_width * 0.8:
               chart_patterns.append("bollinger_squeeze")

       data_sources[0].success = True

       print(f"✅ Technical analysis for {symbol}: {momentum_strength}, Entry: ${suggested_entry_level:.2f}, Exit: ${suggested_exit_level:.2f}")

       return TechnicalSwingData(
           symbol=symbol,
           swing_signals=swing_signals,
           momentum_strength=momentum_strength,
           suggested_entry_level=round(suggested_entry_level, 2),
           suggested_exit_level=round(suggested_exit_level, 2),
           stop_loss_level=round(stop_loss_level, 2),
           chart_patterns=chart_patterns,
           volume_confirmation=volume_confirmation,
           data_sources=data_sources
       )

   except Exception as e:
       error_msg = f"Error calculating technical signals: {e}"
       print(f"❌ {error_msg} for {symbol}")
       data_sources[0].success = False
       data_sources[0].error_message = str(e)
       return TechnicalSwingData(
           symbol=symbol,
           error=error_msg,
           data_sources=data_sources
       )

async def analyze_volume_liquidity(symbol: str, period: str = "2mo") -> VolumeLiquidityData:
   """Analyze volume and liquidity with source tracking"""
   yahoo_volume_url = f"https://finance.yahoo.com/quote/{symbol}"
   yfinance_api = f"yfinance.Ticker('{symbol}').history(period='{period}')"
   print(f"📊 [VOLUME/LIQUIDITY] Analyzing liquidity for {symbol}...")
   print(f"🔗 Yahoo Finance Volume: {yahoo_volume_url}")
   print(f"🔗 API Call: {yfinance_api}")
   print(f"🔗 Analysis Period: {period} (2 months)")

   data_sources = [
       DataSourceInfo(
           source_name="Yahoo Finance Volume Data",
           source_url=f"https://finance.yahoo.com/quote/{symbol}",
           api_endpoint=f"yfinance.Ticker('{symbol}').history(period='{period}')"
       )
   ]

   try:
       await asyncio.sleep(API_CALL_DELAY_YFINANCE)
       stock = yf.Ticker(symbol)
       hist = stock.history(period=period)

       if hist.empty:
           data_sources[0].success = False
           data_sources[0].error_message = "No volume data"
           return VolumeLiquidityData(
               symbol=symbol,
               error="No volume data",
               data_sources=data_sources
           )

       data_sources[0].success = True

       # Basic metrics
       avg_daily_volume = hist['Volume'].mean()
       recent_10d_volume = hist['Volume'].tail(10).mean()
       volume_trend = "increasing" if recent_10d_volume > avg_daily_volume * 1.1 else "decreasing" if recent_10d_volume < avg_daily_volume * 0.9 else "stable"

       # Liquidity score
       if avg_daily_volume > 50_000_000:
           liquidity_score = 10.0
           execution_difficulty = "easy"
       elif avg_daily_volume > 20_000_000:
           liquidity_score = 8.5
           execution_difficulty = "easy"
       elif avg_daily_volume > 10_000_000:
           liquidity_score = 7.0
           execution_difficulty = "easy"
       elif avg_daily_volume > 5_000_000:
           liquidity_score = 6.0
           execution_difficulty = "moderate"
       elif avg_daily_volume > 1_000_000:
           liquidity_score = 4.0
           execution_difficulty = "moderate"
       else:
           liquidity_score = 2.0
           execution_difficulty = "difficult"

       # Volume spikes
       volume_spikes = []
       volume_threshold = avg_daily_volume * 2

       for i, (date, volume) in enumerate(hist['Volume'].tail(20).items()):
           if volume > volume_threshold:
               price_change = 0
               if i > 0:
                   prev_close = hist['Close'].iloc[i-1]
                   curr_close = hist['Close'].iloc[i]
                   price_change = ((curr_close - prev_close) / prev_close) * 100

               volume_spikes.append({
                   "date": date.strftime('%Y-%m-%d'),
                   "volume": int(volume),
                   "volume_ratio": round(volume / avg_daily_volume, 1),
                   "price_change": round(price_change, 2)
               })

       volume_spikes.sort(key=lambda x: x['volume_ratio'], reverse=True)
       volume_spikes = volume_spikes[:5]

       print(f"✅ Volume analysis for {symbol}: Avg Vol={avg_daily_volume/1_000_000:.1f}M, Score={liquidity_score:.1f}/10")

       return VolumeLiquidityData(
           symbol=symbol,
           average_daily_volume=avg_daily_volume,
           volume_trend=volume_trend,
           liquidity_score=liquidity_score,
           volume_spikes=volume_spikes,
           execution_difficulty=execution_difficulty,
           data_sources=data_sources
       )

   except Exception as e:
       error_msg = f"Error analyzing volume: {e}"
       print(f"❌ {error_msg} for {symbol}")
       data_sources[0].success = False
       data_sources[0].error_message = str(e)
       return VolumeLiquidityData(
           symbol=symbol,
           error=error_msg,
           data_sources=data_sources
       )

async def assess_short_term_risk(symbol: str, price_data: ShortTermPriceData, period: str = "1mo") -> ShortTermRiskData:
   """Assess short-term risk with source tracking"""
   yahoo_risk_url = f"https://finance.yahoo.com/quote/{symbol}"
   qqq_comparison_url = f"https://finance.yahoo.com/quote/QQQ"
   yfinance_risk_api = f"yfinance.Ticker('{symbol}').history(period='{period}')"

   print(f"⚠️ [SHORT-TERM RISK] Assessing 5-day risk for {symbol}...")
   print(f"🔗 Yahoo Finance Risk Data: {yahoo_risk_url}")
   print(f"🔗 QQQ Correlation: {qqq_comparison_url}")
   print(f"🔗 API Call: {yfinance_risk_api}")
   print(f"🔗 Risk Metrics: VaR, Beta, Volatility, Correlation")

   data_sources = [
       DataSourceInfo(
           source_name="Yahoo Finance Risk Analysis",
           source_url=f"https://finance.yahoo.com/quote/{symbol}",
           api_endpoint=f"yfinance.Ticker('{symbol}').history() + QQQ correlation"
       )
   ]

   try:
       await asyncio.sleep(API_CALL_DELAY_YFINANCE)
       stock = yf.Ticker(symbol)
       hist = stock.history(period=period)

       if hist.empty:
           data_sources[0].success = False
           data_sources[0].error_message = "No risk data"
           return ShortTermRiskData(
               symbol=symbol,
               error="No risk data",
               data_sources=data_sources
           )

       data_sources[0].success = True

       # Daily returns
       returns = hist['Close'].pct_change().dropna()

       # 5-Day VaR (แก้ไข format error)
       five_day_var = None
       if len(returns) >= 20:
           daily_var_95 = np.percentile(returns, 5)
           five_day_var = daily_var_95 * np.sqrt(5) * 100
           five_day_var = float(five_day_var)  # แปลงเป็น float ป้องกัน format error

       # Maximum intraweek loss
       weekly_losses = []
       if len(hist) >= 25:
           for i in range(0, len(hist)-5, 5):
               week_data = hist.iloc[i:i+5]
               if len(week_data) == 5:
                   week_return = ((week_data['Close'].iloc[-1] - week_data['Open'].iloc[0]) / week_data['Open'].iloc[0]) * 100
                   weekly_losses.append(float(week_return))  # แปลงเป็น float

       max_intraweek_loss = float(min(weekly_losses)) if weekly_losses else None

       # Volatility analysis
       volatility = float(returns.std() * np.sqrt(252) * 100)

       if 25 <= volatility <= 45:
           current_vol_vs_optimal = "optimal"
       elif volatility < 25:
           current_vol_vs_optimal = "too_low"
       elif volatility > 60:
           current_vol_vs_optimal = "too_high"
       else:
           current_vol_vs_optimal = "acceptable"

       optimal_volatility_range = {"min": 25.0, "max": 45.0, "current": round(volatility, 1)}

       # Beta with QQQ
       tech_sector_beta = None
       correlation_with_qqq = None

       try:
           qqq = yf.Ticker("QQQ")
           qqq_hist = qqq.history(period=period)

           if not qqq_hist.empty and len(qqq_hist) == len(hist):
               stock_returns = hist['Close'].pct_change().dropna()
               qqq_returns = qqq_hist['Close'].pct_change().dropna()

               min_len = min(len(stock_returns), len(qqq_returns))
               if min_len > 20:
                   stock_returns_aligned = stock_returns.tail(min_len)
                   qqq_returns_aligned = qqq_returns.tail(min_len)

                   covariance = np.cov(stock_returns_aligned, qqq_returns_aligned)[0][1]
                   qqq_variance = np.var(qqq_returns_aligned)
                   if qqq_variance > 0:
                       tech_sector_beta = round(float(covariance / qqq_variance), 2)

                   correlation_with_qqq = round(float(np.corrcoef(stock_returns_aligned, qqq_returns_aligned)[0][1]), 3)
       except Exception as beta_error:
           print(f"⚠️ Beta calculation error: {beta_error}")

       # Risk-reward ratio
       risk_reward_ratio = None
       if price_data.current_price and len(returns) >= 5:
           expected_weekly_return = float(returns.mean() * 5 * 100)
           weekly_risk = float(returns.std() * np.sqrt(5) * 100)

           if weekly_risk > 0:
               risk_reward_ratio = round(float(abs(expected_weekly_return) / weekly_risk), 2)

       # แก้ไข format string เพื่อป้องกัน error
       var_display = f"{five_day_var:.1f}" if five_day_var is not None else "N/A"
       vol_display = f"{volatility:.1f}" if volatility is not None else "N/A"

       print(f"✅ Risk analysis for {symbol}: 5-day VaR={var_display}%, Vol={vol_display}%")

       return ShortTermRiskData(
           symbol=symbol,
           five_day_var=round(five_day_var, 1) if five_day_var is not None else None,
           max_intraweek_loss=round(max_intraweek_loss, 1) if max_intraweek_loss is not None else None,
           optimal_volatility_range=optimal_volatility_range,
           current_volatility_vs_optimal=current_vol_vs_optimal,
           tech_sector_beta=tech_sector_beta,
           correlation_with_qqq=correlation_with_qqq,
           risk_reward_ratio=risk_reward_ratio,
           data_sources=data_sources
       )

   except Exception as e:
       error_msg = f"Error assessing risk: {e}"
       print(f"❌ {error_msg} for {symbol}")
       data_sources[0].success = False
       data_sources[0].error_message = str(e)
       return ShortTermRiskData(
           symbol=symbol,
           error=error_msg,
           data_sources=data_sources
       )

async def optimize_entry_exit_timing(symbol: str, all_data: Dict[str, Any]) -> EntryExitTimingData:
   """Optimize timing with source tracking"""
   print(f"🎯 [TIMING] Optimizing entry/exit timing for {symbol}...")
   print(f"🔗 Data Sources: Combined analysis from all previous steps")
   print(f"🔗 Intraday Data: Yahoo Finance hourly data")
   print(f"🔗 Algorithm: AI-powered timing optimization")
   print(f"🔗 Strategy: Monday entry → Friday exit optimization")

   data_sources = [
       DataSourceInfo(
           source_name="AI Timing Optimization",
           source_url="Internal algorithm",
           api_endpoint="Combined data analysis for optimal timing"
       )
   ]

   try:
       price_data = all_data.get('price_data', {})
       technical_data = all_data.get('technical_swing_data', {})
       volume_data = all_data.get('volume_liquidity_data', {})
       risk_data = all_data.get('short_term_risk_data', {})

       # Analyze patterns
       intraday_patterns = price_data.get('intraday_patterns', {})

       # Best entry time
       best_entry_time = "monday_open"
       if intraday_patterns:
           best_buy_hour = intraday_patterns.get('best_hour_to_buy')
           morning_momentum = intraday_patterns.get('morning_momentum', 0)

           if best_buy_hour:
               if best_buy_hour <= 10:
                   best_entry_time = "monday_early"
               elif best_buy_hour <= 12:
                   best_entry_time = "monday_mid"
               else:
                   best_entry_time = "monday_afternoon"
           elif morning_momentum > 0.1:
               best_entry_time = "monday_early"
           elif morning_momentum < -0.1:
               best_entry_time = "tuesday_early"

       # Best exit time
       best_exit_time = "friday_close"
       if intraday_patterns:
           best_sell_hour = intraday_patterns.get('best_hour_to_sell')
           afternoon_momentum = intraday_patterns.get('afternoon_momentum', 0)

           if best_sell_hour:
               if best_sell_hour <= 12:
                   best_exit_time = "friday_morning"
               elif best_sell_hour <= 14:
                   best_exit_time = "friday_mid"
               else:
                   best_exit_time = "friday_close"


       # Position sizing ใหม่ - ใช้ AI score
       # Position sizing - ใช้ข้อมูลที่มี
       liquidity_score = volume_data.get('liquidity_score', 5.0)
       risk_reward_ratio = risk_data.get('risk_reward_ratio', 1.0)

       base_position = 20.0
       liquidity_adjustment = 1.0 if liquidity_score >= 8 else 0.8 if liquidity_score >= 6 else 0.6
       risk_adjustment = 1.2 if risk_reward_ratio and risk_reward_ratio >= 1.5 else 1.0 if risk_reward_ratio and risk_reward_ratio >= 1.0 else 0.7

       recommended_position_size = min(35.0, base_position * liquidity_adjustment * risk_adjustment)

       # Optimal holding period
       momentum_strength = technical_data.get('momentum_strength', 'neutral')
       volume_confirmation = technical_data.get('volume_confirmation', False)

       if momentum_strength in ['strong_bullish', 'strong_bearish'] and volume_confirmation:
           optimal_holding_days = 3
       elif momentum_strength in ['bullish', 'bearish']:
           optimal_holding_days = 4
       else:
           optimal_holding_days = 5

       # Success probability
       success_factors = 0
       total_factors = 0

       if momentum_strength in ['strong_bullish', 'bullish']:
           success_factors += 1
       total_factors += 1

       if volume_confirmation:
           success_factors += 1
       total_factors += 1

       if liquidity_score >= 7:
           success_factors += 1
       total_factors += 1

       if risk_reward_ratio and risk_reward_ratio >= 1.2:
           success_factors += 1
       total_factors += 1

       success_probability = success_factors / total_factors if total_factors > 0 else 0.5

       data_sources[0].success = True

       print(f"✅ Timing optimization for {symbol}: {best_entry_time} → {best_exit_time}, Success: {success_probability:.1%}")

       return EntryExitTimingData(
           symbol=symbol,
           best_entry_time=best_entry_time,
           best_exit_time=best_exit_time,
           intraday_patterns=intraday_patterns,
           recommended_position_size=round(recommended_position_size, 1),
           optimal_holding_days=optimal_holding_days,
           success_probability=round(success_probability, 3),
           data_sources=data_sources
       )

   except Exception as e:
       error_msg = f"Error optimizing timing: {e}"
       print(f"❌ {error_msg} for {symbol}")
       data_sources[0].success = False
       data_sources[0].error_message = str(e)
       return EntryExitTimingData(
           symbol=symbol,
           error=error_msg,
           data_sources=data_sources
       )

# === AI AGENT DEPENDENCIES ===
class ShortTermAgentDeps:
   def __init__(self, client: AsyncClient, symbol: Optional[str] = None):
       self.client = client
       self.current_symbol = symbol
       self.alpha_vantage_key = ALPHA_VANTAGE_API_KEY
       self.news_api_key = NEWS_API_KEY
       self.polygon_api_key = POLYGON_API_KEY
       self.analysis_timestamp = datetime.now(timezone.utc).isoformat()

# === AI AGENT RUNNER ===
async def run_short_term_ai_agent(agent_name: str, agent_instance: Agent, context_data: Any,
                               task_prompt: str, symbol: str, client: AsyncClient) -> Dict[str, Any]:
    """Run AI agent with REAL analysis ONLY - no fallbacks"""

    print(f"🤖 Running REAL {agent_name} AI agent for {symbol}...")

    if not GROQ_API_KEY:
        raise Exception("❌ GROQ API key required - system requires real AI analysis")

    try:
        # สร้าง enhanced prompt สำหรับ agent
        enhanced_prompt = create_enhanced_agent_prompt(agent_name, context_data, symbol)

        # เรียก AI จริงเท่านั้น
        deps = ShortTermAgentDeps(client=client, symbol=symbol)
        result = await agent_instance.run(enhanced_prompt, deps=deps)

        # Validate และ enhance response
        validated_result = validate_ai_agent_response(result, agent_name, symbol)

        print(f"✅ {agent_name} AI analysis: {validated_result['score']:.1f}/10 (confidence: {validated_result['confidence']:.1%})")
        return validated_result

    except Exception as e:
        print(f"❌ CRITICAL: {agent_name} AI agent failed for {symbol}: {e}")
        raise Exception(f"AI agent {agent_name} failed: {e}")

def validate_ai_agent_response(response: Dict, agent_name: str, symbol: str) -> Dict[str, Any]:
    """Validate and enhance AI agent response"""

    if not isinstance(response, dict):
        raise Exception(f"Invalid response type from {agent_name}")

    # Required fields validation
    required_fields = ['score', 'confidence', 'recommendation']
    for field in required_fields:
        if field not in response:
            raise Exception(f"Missing required field '{field}' from {agent_name}")

    # Score validation และปรับแต่ง
    try:
        score = float(response['score'])
        score = max(0.0, min(10.0, score))
        response['score'] = round(score, 1)
    except:
        raise Exception(f"Invalid score from {agent_name}: {response.get('score')}")

    # Confidence validation
    try:
        confidence = float(response['confidence'])
        confidence = max(0.0, min(1.0, confidence))
        response['confidence'] = round(confidence, 3)
    except:
        response['confidence'] = 0.7  # Default confidence

    # Expected return validation - CRITICAL FOR PRICE PREDICTION
    if 'expected_5day_return' in response:
        try:
            expected_return = float(response['expected_5day_return'])
            response['expected_5day_return'] = round(expected_return, 2)
        except:
            response['expected_5day_return'] = None

    # Enhanced metadata
    response.update({
        'agent_type': agent_name,
        'symbol': symbol,
        'analysis_timestamp': datetime.now().isoformat(),
        'ai_powered': True,
        'rule_based': False
    })

    return response


# def generate_enhanced_simulation_score(agent_name: str, symbol: str, context_data: Dict, error: bool = False) -> Dict[str, Any]:
#    """Generate enhanced simulation scores based on actual data"""

#    if error:
#        return {
#            "error": "Agent processing error",
#            "agent": agent_name,
#            "symbol": symbol,
#            "score": 5.0,
#            "confidence": 0.5,
#            "recommendation": "hold"
#        }

#    # Base score
#    base_score = 6.0

#    # Agent-specific scoring logic based on context data
#    try:
#        if agent_name == "momentum":
#            # Momentum scoring based on price data
#            predicted_return = context_data.get('predicted_5day_return', 0)
#            confidence = context_data.get('prediction_confidence', 0.5)
#            momentum_trend = context_data.get('daily_momentum', {}).get('momentum_trend', 'neutral')

#            score = base_score
#            if predicted_return > 2:
#                score += 2.0
#            elif predicted_return > 0:
#                score += 1.0
#            elif predicted_return < -2:
#                score -= 1.0

#            if momentum_trend == 'positive':
#                score += 0.5
#            elif momentum_trend == 'negative':
#                score -= 0.5

#            score += confidence * 1.5

#        elif agent_name == "catalyst":
#            # Catalyst scoring based on news data
#            catalyst_strength = context_data.get('catalyst_strength', 'none')
#            news_count = len(context_data.get('breaking_news', []))

#            score = base_score
#            if catalyst_strength == 'strong':
#                score += 2.5
#            elif catalyst_strength == 'moderate':
#                score += 1.5
#            elif catalyst_strength == 'weak':
#                score += 0.5

#            if news_count > 3:
#                score += 1.0
#            elif news_count > 1:
#                score += 0.5

#        elif agent_name == "technical":
#            # Technical scoring based on signals
#            momentum_strength = context_data.get('momentum_strength', 'neutral')
#            swing_signals = context_data.get('swing_signals', {})

#            score = base_score
#            if momentum_strength == 'strong_bullish':
#                score += 2.5
#            elif momentum_strength == 'bullish':
#                score += 1.5
#            elif momentum_strength == 'strong_bearish':
#                score -= 1.0
#            elif momentum_strength == 'bearish':
#                score -= 0.5

#            # Count bullish signals
#            bullish_signals = sum(1 for signal in swing_signals.values()
#                                if 'bullish' in str(signal) or 'buy' in str(signal))
#            score += bullish_signals * 0.3

#        elif agent_name == "liquidity":
#            # Liquidity scoring
#            liquidity_score = context_data.get('liquidity_score', 5.0)
#            avg_volume = context_data.get('average_daily_volume', 0)

#            score = base_score
#            score += (liquidity_score - 5.0) * 0.4

#            if avg_volume > 50_000_000:
#                score += 1.0
#            elif avg_volume > 20_000_000:
#                score += 0.5

#        elif agent_name == "risk":
#            # Risk scoring (inverted - lower risk = higher score)
#            five_day_var = abs(context_data.get('five_day_var', -5.0))
#            risk_reward = context_data.get('risk_reward_ratio', 1.0)

#            score = base_score
#            if five_day_var < 3:
#                score += 1.5
#            elif five_day_var < 5:
#                score += 1.0
#            elif five_day_var > 8:
#                score -= 1.0

#            if risk_reward > 1.5:
#                score += 1.0
#            elif risk_reward > 1.0:
#                score += 0.5

#        elif agent_name == "timing":
#            # Timing scoring
#            success_prob = context_data.get('success_probability', 0.5)

#            score = base_score
#            score += success_prob * 3.0

#        else:
#            score = base_score + random.uniform(-1, 1)

#    except Exception as e:
#        print(f"⚠️ Simulation scoring error for {agent_name}: {e}")
#        score = base_score + random.uniform(-0.5, 0.5)

#    # Ensure score is within bounds
#    score = max(1.0, min(10.0, score))

#    # Generate recommendation based on score
#    if score >= 8.0:
#        recommendation = "strong_buy_monday"
#    elif score >= 7.0:
#        recommendation = "buy_monday"
#    elif score >= 6.0:
#        recommendation = "moderate_buy_monday"
#    elif score >= 5.0:
#        recommendation = "hold"
#    else:
#        recommendation = "avoid_this_week"

#    return {
#        "agent": agent_name,
#        "symbol": symbol,
#        "score": round(score, 1),
#        "confidence": min(0.9, max(0.6, score / 10 + random.uniform(-0.1, 0.1))),
#        "recommendation": recommendation,
#        "key_insights": [
#            f"Enhanced {agent_name} analysis for {symbol}",
#            f"Data-driven scoring: {score:.1f}/10",
#            "Optimized for 5-day trading window"
#        ],
#        "rationale": f"Enhanced AI simulation suggests {symbol} shows {recommendation.replace('_', ' ')} signals for {agent_name} factors",
#        "weekly_risks": ["Market volatility", "Sector rotation risk", "Economic events"],
#        "optimal_entry_day": "monday",
#        "optimal_exit_day": "friday",
#        "simulation": True,
#        "enhanced": True
#    }

# === BATCH PROCESSING ===
async def process_short_term_batch(symbols_batch: List[str], client: AsyncClient) -> Dict[str, ShortTermStockAnalysis]:
   """Process tech stocks in batch for short-term trading"""
   print(f"\n🔄 === SHORT-TERM TECH BATCH: {', '.join(symbols_batch)} ===")

   batch_results = {}
   batch_start_time = time.time()

   # Trading week context
   monday, friday = get_current_trading_week_dates()
   now_et = get_current_time_et()

   trading_week_context = {
      "monday": monday.strftime('%Y-%m-%d'),
      "friday": friday.strftime('%Y-%m-%d'),
      "current_day": now_et.strftime('%A'),
      "days_in_week": str(max(0, (now_et - monday).days))
   }

   for symbol in symbols_batch:
      stock_start_time = time.time()

      try:
         print(f"\n🔍 === SHORT-TERM ANALYSIS: {symbol} ===")

         # Company info
         company_name = symbol
         try:
            ticker = yf.Ticker(symbol)
            info = ticker.info
            company_name = info.get('longName', info.get('shortName', symbol))
            sector = info.get('sector', 'Technology')

            print(f"🏢 [FUNDAMENTAL] Company info for {symbol}:")
            print(f"🔗 Yahoo Finance Profile: https://finance.yahoo.com/quote/{symbol}/profile")
            print(f"🔗 Yahoo Finance Financials: https://finance.yahoo.com/quote/{symbol}/financials")
            print(f"🔗 Company: {company_name}")
            print(f"🔗 Sector: {sector}")
            print(f"🔗 Market Cap: ${info.get('marketCap', 0):,.0f}" if info.get('marketCap') else "🔗 Market Cap: N/A")
            print(f"🔗 P/E Ratio: {info.get('trailingPE', 'N/A')}")

            if 'Technology' not in sector and 'Communication' not in sector:
               print(f"⚠️ {symbol} may not be pure tech stock (Sector: {sector})")

         except Exception as e:
            print(f"⚠️ Company info error for {symbol}: {e}")

         # === PHASE 1: DATA COLLECTION ===
         print(f"📊 Phase 1: Data collection for {symbol}")

         # 1. Price data
         print(f"   📈 Step 1: Fetching price data...")
         price_data = await fetch_short_term_price_data(symbol, period=SHORT_TERM_ANALYSIS_PERIOD)

         if price_data.error:
            print(f"   ⚠️ Price data warning: {price_data.error}")
            # ไม่ raise exception แต่สร้าง default analysis แทน
            error_analysis = ShortTermStockAnalysis(
                symbol=symbol,
                company_name=symbol,
                sector="Technology",
                weekly_opportunity_score=5.0,  # ให้คะแนนปานกลาง
                final_recommendation="skip_data_error"
            )
            batch_results[symbol] = error_analysis
            continue  # ข้ามไปหุ้นตัวถัดไป
         else:
            current = price_data.current_price or 0
            predicted = price_data.predicted_friday_price or 0
            return_pct = price_data.predicted_5day_return or 0
            confidence = price_data.prediction_confidence or 0

            print(f"   ✅ Price data: Current=${current:.2f}, Predicted=${predicted:.2f}")
            print(f"   📊 Expected 5-day return: {return_pct:+.1f}% (confidence: {confidence:.1%})")

         await asyncio.sleep(0.5)

         # 2. News catalyst
         print(f"   📰 Step 2: Fetching news data...")
         deps = ShortTermAgentDeps(client=client, symbol=symbol)
         news_data = await fetch_news_catalyst_data(symbol, deps)

         if news_data.error:
            print(f"   ⚠️ News data error: {news_data.error}")
         else:
            print(f"   ✅ News data: {len(news_data.breaking_news)} articles, strength: {news_data.catalyst_strength}")

         await asyncio.sleep(0.5)

         # 3. Technical analysis
         print(f"   📊 Step 3: Technical analysis...")
         technical_data = await calculate_technical_swing_signals(symbol, price_data)

         if technical_data.error:
            print(f"   ⚠️ Technical analysis error: {technical_data.error}")
         else:
            entry_level = technical_data.suggested_entry_level or 0
            print(f"   ✅ Technical: {technical_data.momentum_strength}, Entry=${entry_level:.2f}")

         await asyncio.sleep(0.5)

         # 4. Volume analysis
         print(f"   📊 Step 4: Volume analysis...")
         volume_data = await analyze_volume_liquidity(symbol, period=VOLUME_ANALYSIS_PERIOD)

         if volume_data.error:
            print(f"   ⚠️ Volume analysis error: {volume_data.error}")
         else:
            liq_score = volume_data.liquidity_score or 0
            avg_vol = volume_data.average_daily_volume or 0
            print(f"   ✅ Volume: Score={liq_score:.1f}/10, Avg={avg_vol/1_000_000:.1f}M")

         await asyncio.sleep(0.5)

         # 5. Risk assessment
         print(f"   ⚠️ Step 5: Risk assessment...")
         risk_data = await assess_short_term_risk(symbol, price_data, period=TECHNICAL_ANALYSIS_PERIOD)

         if risk_data.error:
            print(f"   ⚠️ Risk assessment error: {risk_data.error}")
         else:
            # แก้ไข format error ด้วยการตรวจสอบ None values
            var_value = risk_data.five_day_var
            ratio_value = risk_data.risk_reward_ratio

            var_display = f"{var_value:.1f}" if var_value is not None else "N/A"
            ratio_display = f"{ratio_value:.2f}" if ratio_value is not None else "N/A"

            print(f"   ✅ Risk: 5-day VaR={var_display}%, Ratio={ratio_display}")

         await asyncio.sleep(0.5)

         # 6. Timing optimization
         print(f"   🎯 Step 6: Timing optimization...")
         all_data_for_timing = {
            'price_data': price_data.model_dump(),
            'technical_swing_data': technical_data.model_dump(),
            'volume_liquidity_data': volume_data.model_dump(),
            'short_term_risk_data': risk_data.model_dump()
         }
         timing_data = await optimize_entry_exit_timing(symbol, all_data_for_timing)

         if timing_data.error:
            print(f"   ⚠️ Timing optimization error: {timing_data.error}")
         else:
            success_prob = timing_data.success_probability or 0
            print(f"   ✅ Timing: {timing_data.best_entry_time} → {timing_data.best_exit_time}, Success={success_prob:.1%}")

         await asyncio.sleep(0.5)

         # === PHASE 2: AI AGENT ANALYSIS ===
         print(f"🤖 Phase 2: AI agent analysis for {symbol}")

         # Create 6 specialized agents
        #  agents = {
        #     "momentum": Agent(
        #        ACTIVE_GROQ_MODEL,
        #        system_prompt="You are a MOMENTUM TRADING Expert for 5-day tech stock swings. Focus on short-term momentum and price acceleration patterns."
        #     ),
        #     "catalyst": Agent(
        #        ACTIVE_GROQ_MODEL,
        #        system_prompt="You are a NEWS CATALYST Expert. Identify news events that will move the stock THIS WEEK."
        #     ),
        #     "technical": Agent(
        #        ACTIVE_GROQ_MODEL,
        #        system_prompt="You are a SWING TRADING Technical Expert. Focus on 5-day technical setups and patterns."
        #     ),
        #     "liquidity": Agent(
        #        ACTIVE_GROQ_MODEL,
        #        system_prompt="You are a LIQUIDITY & EXECUTION Expert. Assess ease of entry/exit for swing trades."
        #     ),
        #     "risk": Agent(
        #        ACTIVE_GROQ_MODEL,
        #        system_prompt="You are a SHORT-TERM RISK Manager. Focus on 5-day holding period risks."
        #     ),
        #     "timing": Agent(
        #        ACTIVE_GROQ_MODEL,
        #        system_prompt="You are an ENTRY/EXIT TIMING Expert for weekly swings."
        #     )
        #  }
         agents = {
          "momentum": Agent(ACTIVE_GROQ_MODEL),
          "catalyst": Agent(ACTIVE_GROQ_MODEL),
          "technical": Agent(ACTIVE_GROQ_MODEL),
          "liquidity": Agent(ACTIVE_GROQ_MODEL),
          "risk": Agent(ACTIVE_GROQ_MODEL),
          "timing": Agent(ACTIVE_GROQ_MODEL)
         }
        #  agents = {
        #     "momentum": Agent(
        #         ACTIVE_GROQ_MODEL,
        #         system_prompt="""You are an ELITE MOMENTUM TRADING SPECIALIST with 15+ years of experience in short-term tech stock momentum analysis. You excel at identifying explosive 5-day price movements in technology stocks using advanced momentum indicators, volume patterns, and market microstructure analysis.

        # Your expertise includes:
        # - Detecting early momentum shifts before major price movements
        # - Analyzing intraday momentum patterns and volume confirmation
        # - Identifying momentum breakouts, reversals, and continuation patterns
        # - Understanding sector rotation effects on tech stock momentum
        # - Recognizing institutional buying/selling pressure signatures
        # - Predicting short-term price acceleration based on momentum convergence

        # You ALWAYS provide precise, actionable momentum analysis with specific entry/exit recommendations for 5-day swing trades. Your analysis is data-driven, technically sound, and optimized for Monday-to-Friday trading cycles in the technology sector."""
        #     ),

        #     "catalyst": Agent(
        #         ACTIVE_GROQ_MODEL,
        #         system_prompt="""You are a LEGENDARY NEWS CATALYST EXPERT and former Wall Street analyst with an exceptional track record of identifying market-moving events before they impact stock prices. You specialize in technology sector catalysts and have deep connections across Silicon Valley, understanding how news flows affect stock momentum within 5-day trading windows.

        # Your specialized knowledge includes:
        # - Earnings surprise prediction based on pre-announcement signals
        # - Product launch impact assessment and market positioning analysis
        # - Regulatory developments and their immediate market implications
        # - Partnership announcements and strategic alliance valuations
        # - Management changes and their short-term stock impact
        # - Social media sentiment shifts and viral news propagation
        # - Supply chain disruptions and semiconductor industry dynamics
        # - AI/technology breakthrough announcements and hype cycles
        # - Analyst upgrades/downgrades timing and institutional reactions
        # - Conference presentations and guidance revision patterns

        # You excel at distinguishing between noise and genuine catalysts that will drive significant price movements within a 5-day period. Your analysis incorporates sentiment momentum, news timing, and market positioning to predict immediate stock reactions."""
        #     ),

        #     "technical": Agent(
        #         ACTIVE_GROQ_MODEL,
        #         system_prompt="""You are a MASTER TECHNICAL ANALYST and former hedge fund quantitative researcher specializing in short-term swing trading setups for technology stocks. You have developed proprietary algorithms and indicators specifically optimized for 5-day momentum swings in high-beta tech names.

        # Your technical expertise encompasses:
        # - Multi-timeframe momentum convergence analysis (1H, 4H, Daily)
        # - Advanced RSI divergence patterns and overbought/oversold reversals
        # - MACD histogram momentum and signal line crossover timing
        # - Bollinger Band squeeze patterns and volatility breakout setups
        # - Volume-weighted moving average (VWAP) deviation analysis
        # - Support/resistance level identification using pivot points and Fibonacci
        # - Chart pattern recognition (flags, pennants, triangles, head & shoulders)
        # - Options flow analysis and unusual volume spike interpretation
        # - Relative strength vs. sector ETF (QQQ) for momentum confirmation
        # - Intraday momentum shifts and optimal entry/exit timing windows

        # You provide precise technical analysis with specific price targets, stop-loss levels, and probability assessments for 5-day swing trades. Your analysis integrates multiple technical indicators with volume confirmation and sector momentum for maximum accuracy."""
        #     ),

        #     "liquidity": Agent(
        #         ACTIVE_GROQ_MODEL,
        #         system_prompt="""You are an INSTITUTIONAL LIQUIDITY SPECIALIST and former market maker with deep expertise in equity market microstructure, order flow analysis, and execution quality for large tech stock positions. You understand how liquidity dynamics affect short-term price movements and optimal trade execution strategies.

        # Your liquidity analysis expertise includes:
        # - Average daily volume analysis and liquidity scoring across different market conditions
        # - Bid-ask spread analysis and market impact cost estimation
        # - Time-of-day liquidity patterns and optimal execution windows
        # - Pre-market and after-hours liquidity assessment for tech stocks
        # - Institutional order flow detection and dark pool activity analysis
        # - Volume spike interpretation and unusual activity pattern recognition
        # - Market maker positioning and specialist intervention levels
        # - Cross-exchange arbitrage opportunities and liquidity fragmentation
        # - Options market maker hedging flows and gamma squeeze potential
        # - ETF creation/redemption flows affecting underlying tech stocks

        # You assess execution difficulty, optimal position sizing, and entry/exit timing based on liquidity conditions. Your analysis helps determine whether large positions can be accumulated or liquidated within 5-day windows without adverse market impact."""
        #     ),

        #     "risk": Agent(
        #         ACTIVE_GROQ_MODEL,
        #         system_prompt="""You are an ELITE RISK MANAGEMENT EXPERT and former Chief Risk Officer with specialized expertise in short-term technology stock risk assessment. You excel at quantifying and managing downside risk for 5-day swing trading strategies while maximizing risk-adjusted returns.

        # Your risk management expertise includes:
        # - Value-at-Risk (VaR) modeling for 5-day holding periods with 95% confidence intervals
        # - Maximum drawdown analysis and position sizing optimization
        # - Correlation analysis with broader tech sector (QQQ) and market (SPY)
        # - Volatility regime analysis and GARCH modeling for tech stocks
        # - Beta stability assessment and systematic risk factor exposure
        # - Company-specific risk factors (earnings, product launches, regulatory)
        # - Sector rotation risk and technology subsector correlation dynamics
        # - Options implied volatility analysis and volatility risk premium
        # - Tail risk assessment and black swan event probability modeling
        # - Stress testing under various market scenarios (rate changes, geopolitical events)

        # You provide precise risk metrics including stop-loss levels, position sizing recommendations, and risk-reward ratio optimization for 5-day tech stock swings. Your analysis ensures capital preservation while maximizing profit potential within acceptable risk parameters."""
        #     ),

        #     "timing": Agent(
        #         ACTIVE_GROQ_MODEL,
        #         system_prompt="""You are a LEGENDARY MARKET TIMING SPECIALIST and former algorithmic trading strategist with an exceptional ability to identify optimal entry and exit windows for short-term technology stock positions. You understand market microstructure, institutional flow patterns, and behavioral finance principles that drive intraday and weekly trading cycles.

        # Your market timing expertise includes:
        # - Intraday momentum pattern analysis (morning gaps, afternoon reversals, close positioning)
        # - Weekly cyclical patterns in tech stocks (Monday effects, Friday positioning)
        # - Institutional rebalancing flows and quarter-end/month-end effects
        # - Economic calendar impact timing and Fed announcement positioning
        # - Earnings season timing and pre/post announcement volatility patterns
        # - Options expiration effects and gamma positioning impacts
        # - After-hours and pre-market price action interpretation
        # - European and Asian market influence on US tech stock opening gaps
        # - Volume pattern analysis and participation rate optimization
        # - Social media sentiment timing and viral news reaction windows

        # You provide precise timing recommendations for entry (Monday) and exit (Friday) with specific hour-by-hour guidance, success probability estimates, and contingency plans for different market scenarios. Your analysis maximizes the probability of profitable 5-day swing trades through optimal timing execution."""
        #     )
        #  }

         # Context data for each agent (แก้ไข contexts error)
         # ให้ AI Agent เป็นคนคาดการณ์ return เอง
         # ให้ AI Agent ทุกตัวเป็นคนวิเคราะห์และคาดการณ์เอง
         agent_contexts = {
            "momentum": {
                "current_price": price_data.current_price,
                "daily_momentum": price_data.daily_momentum,
                "week_to_date_return": price_data.week_to_date_return,
                "historical_price_data": price_data.short_term_data,  # ให้ AI ดูข้อมูลเอง
                "intraday_patterns": price_data.intraday_patterns,
                "ai_price_baseline": {  # ข้อมูลอ้างอิงเท่านั้น
                    "predicted_price": price_data.predicted_friday_price,
                    "predicted_return": price_data.predicted_5day_return,
                    "confidence": price_data.prediction_confidence
                },
                "mission": "Analyze momentum and provide YOUR OWN 5-day return prediction"
            },
            "catalyst": {
                "current_price": price_data.current_price,
                "breaking_news": news_data.breaking_news,
                "catalyst_strength": news_data.catalyst_strength,
                "weekly_sentiment_trend": news_data.weekly_sentiment_trend,
                # "historical_price_data": price_data.short_term_data,  # เพื่อดู price reaction
                "ai_price_baseline": {  # ข้อมูลอ้างอิงเท่านั้น
                    "predicted_price": price_data.predicted_friday_price,
                    "predicted_return": price_data.predicted_5day_return
                },
                "mission": "Analyze news catalysts and provide YOUR OWN catalyst-driven return prediction"
            },
            "technical": {
                "current_price": price_data.current_price,
                "swing_signals": technical_data.swing_signals,
                "momentum_strength": technical_data.momentum_strength,
                "suggested_entry_level": technical_data.suggested_entry_level,
                "suggested_exit_level": technical_data.suggested_exit_level,
                "historical_price_data": price_data.short_term_data,  # ให้ AI ดู chart เอง
                "chart_patterns": technical_data.chart_patterns,
                "volume_confirmation": technical_data.volume_confirmation,
                "mission": "Analyze technical patterns and provide YOUR OWN technical-based return prediction"
            },
            "liquidity": {
                "current_price": price_data.current_price,
                "average_daily_volume": volume_data.average_daily_volume,
                "liquidity_score": volume_data.liquidity_score,
                "execution_difficulty": volume_data.execution_difficulty,
                "volume_spikes": volume_data.volume_spikes,
                "volume_trend": volume_data.volume_trend,
                "historical_volume_data": [d for d in price_data.short_term_data if 'volume' in d],
                "mission": "Analyze liquidity conditions and assess execution feasibility"
            },
            "risk": {
                "current_price": price_data.current_price,
                "five_day_var": risk_data.five_day_var,
                "risk_reward_ratio": risk_data.risk_reward_ratio,
                "current_volatility_vs_optimal": risk_data.current_volatility_vs_optimal,
                "max_intraweek_loss": risk_data.max_intraweek_loss,
                "tech_sector_beta": risk_data.tech_sector_beta,
                "correlation_with_qqq": risk_data.correlation_with_qqq,
                "historical_price_data": price_data.short_term_data,  # ให้ AI คำนวณ risk เอง
                "mission": "Analyze risk factors and provide risk-adjusted assessment"
            },
            "timing": {
                "current_price": price_data.current_price,
                "best_entry_time": timing_data.best_entry_time,
                "best_exit_time": timing_data.best_exit_time,
                "success_probability": timing_data.success_probability,
                "recommended_position_size": timing_data.recommended_position_size,
                "intraday_patterns": price_data.intraday_patterns,
                "optimal_holding_days": timing_data.optimal_holding_days,
                "historical_price_data": price_data.short_term_data,  # ให้ AI ดู timing patterns เอง
                "mission": "Analyze optimal timing and provide success probability assessment"
            }
         }

         # Run all agents
         analyses = {}
         agent_scores = {}

         print(f"   🤖 Running 6 AI agents...")
         for agent_name, agent in agents.items():
            try:
               print(f"      🤖 {agent_name} agent...")

               task_prompt = f"""
Analyze {symbol} for 5-day swing trading opportunity.
Provide score 0-10 for THIS WEEK's {agent_name} opportunity.
Focus on Monday entry, Friday exit strategy.
"""

               analysis = await run_short_term_ai_agent(
                  agent_name, agent, agent_contexts[agent_name],
                  task_prompt, symbol, client
               )

               analyses[agent_name] = analysis

               # Extract score
               agent_score = analysis.get('score', 5.0)
               try:
                  agent_score = float(agent_score)
                  agent_score = max(0.0, min(10.0, agent_score))
               except:
                  print(f"         ⚠️ Invalid score from {agent_name}: {analysis.get('score')}")
                  agent_score = 5.0

               agent_scores[agent_name] = agent_score
               print(f"         {agent_name.upper()}: {agent_score:.1f}/10")

               await asyncio.sleep(1)

            except Exception as e:
               print(f"      ❌ {agent_name} agent error: {e}")
               agent_scores[agent_name] = 5.0  # Default score instead of 0
               analyses[agent_name] = {
                  "agent": agent_name,
                  "symbol": symbol,
                  "score": 5.0,
                  "error": str(e)
               }

         # === ENHANCED WEIGHTED SCORING SYSTEM ===
         weights = {
            "momentum": 0.20,    # 20% - สำคัญที่สุดสำหรับ 5-day trading
            "catalyst": 0.20,    # 20% - ข่าวสารที่จะขับเคลื่อนราคา
            "technical": 0.20,   # 20% - สัญญาณทางเทคนิค
            "risk": 0.20,        # 20% - การจัดการความเสี่ยง
            "liquidity": 0.10,   # 10% - สภาพคล่องสำหรับ entry/exit
            "timing": 0.10       # 10% - จังหวะเข้า-ออก
         }

         # คำนวณคะแนนถ่วงน้ำหนัก
         weighted_score = sum(agent_scores[agent] * weights[agent] for agent in agent_scores)

         # Bonus scoring สำหรับ combination ที่ดี
         bonus_score = 0.0

         # Momentum + Technical alignment bonus
         if agent_scores.get('momentum', 0) >= 7.0 and agent_scores.get('technical', 0) >= 7.0:
            bonus_score += 0.5

         # High liquidity bonus for swing trading
         if agent_scores.get('liquidity', 0) >= 8.0:
            bonus_score += 0.3

         # News catalyst bonus
         if agent_scores.get('catalyst', 0) >= 7.5:
            bonus_score += 0.4

         # Strong timing bonus
         if agent_scores.get('timing', 0) >= 7.0:
            bonus_score += 0.2

         # Final weighted score with bonus
         final_weighted_score = min(10.0, weighted_score + bonus_score)

         print(f"   📊 Agent Scores: {agent_scores}")
         print(f"   📊 Weighted Score: {final_weighted_score:.2f}/10 (base: {weighted_score:.2f} + bonus: {bonus_score:.2f})")

         # Enhanced recommendation logic
         if final_weighted_score >= 8.5:
            final_recommendation = "strong_buy_monday"
         elif final_weighted_score >= 7.5:
            final_recommendation = "buy_monday"
         elif final_weighted_score >= 6.5:
            final_recommendation = "moderate_buy_monday"
         elif final_weighted_score >= 5.5:
            final_recommendation = "hold"
         elif final_weighted_score >= 4.0:
            final_recommendation = "weak_hold"
         else:
            final_recommendation = "avoid_this_week"

         # Collect all data sources
         all_data_sources = []
         for data in [price_data, news_data, technical_data, volume_data, risk_data, timing_data]:
            if hasattr(data, 'data_sources'):
               all_data_sources.extend(data.data_sources)

         # Create stock analysis
         stock_analysis = ShortTermStockAnalysis(
            symbol=symbol,
            company_name=company_name,
            sector="Technology",
            trading_week=trading_week_context,
            price_data=price_data.model_dump(),
            news_catalyst_data=news_data.model_dump(),
            technical_swing_data=technical_data.model_dump(),
            volume_liquidity_data=volume_data.model_dump(),
            short_term_risk_data=risk_data.model_dump(),
            timing_data=timing_data.model_dump(),
            momentum_agent_analysis=analyses.get('momentum', {}),
            catalyst_agent_analysis=analyses.get('catalyst', {}),
            swing_agent_analysis=analyses.get('technical', {}),
            liquidity_agent_analysis=analyses.get('liquidity', {}),
            risk_agent_analysis=analyses.get('risk', {}),
            timing_agent_analysis=analyses.get('timing', {}),
            agent_scores=agent_scores,
            weekly_opportunity_score=final_weighted_score,
            final_recommendation=final_recommendation,
            all_data_sources=all_data_sources
         )

         batch_results[symbol] = stock_analysis

         stock_processing_time = time.time() - stock_start_time
         print(f"✅ {symbol} completed in {stock_processing_time:.1f}s")

         # แก้ไข format error ในส่วนสุดท้าย
         current_price_display = price_data.current_price or 0
         predicted_price_display = price_data.predicted_friday_price or 0
         return_pct_display = price_data.predicted_5day_return or 0
         confidence_display = price_data.prediction_confidence or 0

         print(f"   📊 Current: ${current_price_display:.2f} | Predicted Friday: ${predicted_price_display:.2f}")
         print(f"   📈 Expected 5-day return: {return_pct_display:+.1f}% (confidence: {confidence_display:.1%})")
         print(f"   📈 Weekly Score: {final_weighted_score:.2f}/10 ({final_recommendation})")

      except Exception as e:
         print(f"❌ CRITICAL ERROR processing {symbol}: {e}")
         import traceback
         traceback.print_exc()

         # สร้าง default analysis แทนการให้คะแนน 0
         error_analysis = ShortTermStockAnalysis(
            symbol=symbol,
            company_name=symbol,
            sector="Technology",
            weekly_opportunity_score=4.0,  # Default score แทน 0
            final_recommendation="avoid_this_week"
         )
         batch_results[symbol] = error_analysis

   batch_processing_time = time.time() - batch_start_time
   print(f"\n📦 Short-term batch completed in {batch_processing_time:.1f}s")
   return batch_results

async def calculate_enhanced_technical_signals(symbol: str, price_data: ShortTermPriceData) -> Dict[str, Any]:
   """Calculate enhanced technical signals for better 5-day predictions"""

   if price_data.error or not price_data.short_term_data:
       return {"error": "Missing price data"}

   try:
       # Create DataFrame
       df = pd.DataFrame(price_data.short_term_data)
       df['date'] = pd.to_datetime(df['date'])
       df = df.set_index('date')

       # Enhanced technical indicators
       df['sma_5'] = df['close'].rolling(5).mean()
       df['sma_10'] = df['close'].rolling(10).mean()
       df['sma_20'] = df['close'].rolling(20).mean()
       df['ema_8'] = df['close'].ewm(span=8).mean()
       df['ema_13'] = df['close'].ewm(span=13).mean()
       df['ema_21'] = df['close'].ewm(span=21).mean()

       # Advanced momentum indicators
       df['rsi'] = ta.rsi(df['close'], length=14)
       df['rsi_fast'] = ta.rsi(df['close'], length=7)  # Fast RSI for short-term

       # MACD with multiple timeframes
       macd_12_26 = ta.macd(df['close'], fast=12, slow=26, signal=9)
       macd_8_17 = ta.macd(df['close'], fast=8, slow=17, signal=9)  # Faster for 5-day

       if macd_12_26 is not None and not macd_12_26.empty:
           df['macd'] = macd_12_26.iloc[:, 0]
           df['macd_signal'] = macd_12_26.iloc[:, 1]
           df['macd_hist'] = macd_12_26.iloc[:, 2]

       if macd_8_17 is not None and not macd_8_17.empty:
           df['macd_fast'] = macd_8_17.iloc[:, 0]
           df['macd_fast_signal'] = macd_8_17.iloc[:, 1]
           df['macd_fast_hist'] = macd_8_17.iloc[:, 2]

       # Bollinger Bands (multiple periods)
       bbands_20 = ta.bbands(df['close'], length=20, std=2)
       bbands_10 = ta.bbands(df['close'], length=10, std=2)  # Shorter for swing trading

       if bbands_20 is not None and not bbands_20.empty:
           df['bb_upper'] = bbands_20.iloc[:, 0]
           df['bb_middle'] = bbands_20.iloc[:, 1]
           df['bb_lower'] = bbands_20.iloc[:, 2]
           df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / df['bb_middle']

       if bbands_10 is not None and not bbands_10.empty:
           df['bb_upper_fast'] = bbands_10.iloc[:, 0]
           df['bb_lower_fast'] = bbands_10.iloc[:, 2]

       # Volume-based indicators
       df['volume_sma'] = df['volume'].rolling(20).mean()
       df['volume_ratio'] = df['volume'] / df['volume_sma']
       df['price_volume'] = df['close'] * df['volume']  # Money flow
       df['pv_sma'] = df['price_volume'].rolling(10).mean()

       # Support and Resistance levels
       df['support'] = df['low'].rolling(20).min()
       df['resistance'] = df['high'].rolling(20).max()
       df['support_fast'] = df['low'].rolling(10).min()
       df['resistance_fast'] = df['high'].rolling(10).max()

       # Price position within range
       df['price_position'] = (df['close'] - df['support']) / (df['resistance'] - df['support'])

       latest = df.iloc[-1]
       current_price = price_data.current_price or latest['close']

       # Enhanced signal scoring
       signals_score = 0
       max_signals = 0
       signal_details = {}

       # 1. Moving Average Signals (weight: 20%)
       ma_score = 0
       if latest['close'] > latest['sma_5'] > latest['sma_10'] > latest['sma_20']:
           ma_score = 10  # Perfect bullish alignment
       elif latest['close'] > latest['sma_5'] > latest['sma_10']:
           ma_score = 7   # Good bullish
       elif latest['close'] > latest['sma_10']:
           ma_score = 5   # Neutral bullish
       elif latest['close'] < latest['sma_10']:
           ma_score = 3   # Bearish
       else:
           ma_score = 1   # Very bearish

       signals_score += ma_score * 0.2
       max_signals += 10 * 0.2
       signal_details['moving_average'] = ma_score

       # 2. RSI Signals (weight: 15%)
       rsi_score = 5  # Default neutral
       rsi_current = latest['rsi'] if not pd.isna(latest['rsi']) else 50
       rsi_fast = latest['rsi_fast'] if not pd.isna(latest['rsi_fast']) else 50

       if 40 <= rsi_current <= 60 and rsi_fast > rsi_current:  # RSI divergence bullish
           rsi_score = 9
       elif 30 <= rsi_current <= 40:  # Oversold opportunity
           rsi_score = 8
       elif 45 <= rsi_current <= 55:  # Neutral good
           rsi_score = 6
       elif rsi_current > 70:  # Overbought
           rsi_score = 3
       elif rsi_current < 30:  # Very oversold
           rsi_score = 7

       signals_score += rsi_score * 0.15
       max_signals += 10 * 0.15
       signal_details['rsi'] = rsi_score

       # 3. MACD Signals (weight: 20%)
       macd_score = 5  # Default
       if not pd.isna(latest['macd']) and not pd.isna(latest['macd_signal']):
           if latest['macd'] > latest['macd_signal'] and latest['macd_hist'] > 0:
               if latest['macd_fast'] > latest['macd_fast_signal']:  # Confirmation
                   macd_score = 9
               else:
                   macd_score = 7
           elif latest['macd'] < latest['macd_signal'] and latest['macd_hist'] < 0:
               macd_score = 2
           else:
               macd_score = 5

       signals_score += macd_score * 0.2
       max_signals += 10 * 0.2
       signal_details['macd'] = macd_score

       # 4. Bollinger Bands Position (weight: 15%)
       bb_score = 5
       if not pd.isna(latest['bb_upper']) and not pd.isna(latest['bb_lower']):
           bb_position = (current_price - latest['bb_lower']) / (latest['bb_upper'] - latest['bb_lower'])
           bb_width = latest['bb_width'] if not pd.isna(latest['bb_width']) else 0.1

           if 0.2 <= bb_position <= 0.4 and bb_width < 0.1:  # Lower band + squeeze
               bb_score = 9
           elif 0.1 <= bb_position <= 0.3:  # Near lower band
               bb_score = 8
           elif 0.4 <= bb_position <= 0.6:  # Middle range
               bb_score = 6
           elif bb_position > 0.8:  # Near upper band
               bb_score = 3
           elif bb_position < 0.1:  # Below lower band (oversold)
               bb_score = 7

       signals_score += bb_score * 0.15
       max_signals += 10 * 0.15
       signal_details['bollinger_bands'] = bb_score

       # 5. Volume Confirmation (weight: 15%)
       volume_score = 5
       volume_ratio = latest['volume_ratio'] if not pd.isna(latest['volume_ratio']) else 1.0

       if volume_ratio > 1.5:  # High volume confirmation
           volume_score = 9
       elif volume_ratio > 1.2:  # Good volume
           volume_score = 7
       elif volume_ratio > 0.8:  # Normal volume
           volume_score = 6
       elif volume_ratio < 0.5:  # Low volume (bad)
           volume_score = 3

       signals_score += volume_score * 0.15
       max_signals += 10 * 0.15
       signal_details['volume'] = volume_score

       # 6. Support/Resistance (weight: 15%)
       sr_score = 5
       price_position = latest['price_position'] if not pd.isna(latest['price_position']) else 0.5

       if 0.1 <= price_position <= 0.3:  # Near support (good buy opportunity)
           sr_score = 8
       elif 0.3 <= price_position <= 0.7:  # Middle range
           sr_score = 6
       elif price_position > 0.9:  # Near resistance
           sr_score = 3
       elif price_position < 0.1:  # Below support
           sr_score = 4

       signals_score += sr_score * 0.15
       max_signals += 10 * 0.15
       signal_details['support_resistance'] = sr_score

       # Calculate final technical score (0-10)
       final_technical_score = (signals_score / max_signals) * 10

       return {
           "enhanced_technical_score": round(final_technical_score, 1),
           "signal_breakdown": signal_details,
           "confidence": min(0.9, max(0.6, final_technical_score / 10)),
           "recommendation": "strong_buy" if final_technical_score >= 8 else
                           "buy" if final_technical_score >= 7 else
                           "hold" if final_technical_score >= 5 else "avoid",
           "key_levels": {
               "support": float(latest['support']) if not pd.isna(latest['support']) else current_price * 0.95,
               "resistance": float(latest['resistance']) if not pd.isna(latest['resistance']) else current_price * 1.05,
               "entry_zone": f"{current_price * 0.995:.2f} - {current_price * 1.005:.2f}",
               "target_zone": f"{current_price * 1.02:.2f} - {current_price * 1.05:.2f}"
           }
       }

   except Exception as e:
       return {"error": f"Enhanced technical analysis failed: {e}"}


# === WORKFLOW NODES ===

async def initialize_short_term_analysis(state: ShortTermAnalysisState) -> Dict[str, Any]:
   """Initialize comprehensive analysis for ALL tech stocks"""
   print("🚀 === COMPREHENSIVE TECH STOCK TRADING AI ===")
   print(f"🎯 Strategy: Buy Monday, Sell Friday (5-day swings)")
   print(f"🏭 Focus: Technology sector stocks")
   print(f"🧠 AI Model: {ACTIVE_GROQ_MODEL}")
   print(f"🔑 API Status: {'Connected' if GROQ_API_KEY else 'Simulation Mode'}")

   # Use ALL tech stocks
   all_tech_symbols = TECH_STOCKS_UNIVERSE.copy()

   print(f"🎯 COMPREHENSIVE ANALYSIS:")
   print(f"📊 Total tech stocks to analyze: {len(all_tech_symbols)}")
   print(f"🏆 Will select top {MAX_STOCKS_TO_SELECT} for elite portfolio")
   print(f"💯 Minimum weekly score: {MIN_WEEKLY_SCORE}/10")
   print(f"📈 Symbols include: {', '.join(all_tech_symbols[:10])}... (+{len(all_tech_symbols)-10} more)")

   # Trading week context
   monday, friday = get_current_trading_week_dates()
   now_et = get_current_time_et()
   trading_week_context = {
       "monday": monday.strftime('%Y-%m-%d'),
       "friday": friday.strftime('%Y-%m-%d'),
       "current_day": now_et.strftime('%A'),
       "days_remaining": max(0, 5 - (now_et - monday).days),
       "is_trading_week": is_trading_day(now_et)
   }

   print(f"📅 Trading Week: {trading_week_context['monday']} to {trading_week_context['friday']}")
   print(f"📅 Today: {trading_week_context['current_day']}")

   # Calculate batches for comprehensive analysis
   total_batches = math.ceil(len(all_tech_symbols) / BATCH_SIZE)
   print(f"📦 Total batches: {total_batches} (analyzing ALL stocks)")
   estimated_time = total_batches * 60  # 60 seconds per batch
   print(f"⏱️ Estimated analysis time: {estimated_time/60:.1f} minutes")

   # Initialize data for all symbols
   initial_data = {}
   for symbol in all_tech_symbols:
       initial_data[symbol] = ShortTermStockAnalysis(
           symbol=symbol,
           company_name=symbol,
           sector="Technology"
       )

   return {
       "target_symbols": all_tech_symbols,
       "current_batch_index": 0,
       "all_stocks_data": initial_data,
       "weekly_portfolio": None,
       "error_messages": [],
       "batch_completed": False,
       "total_batches": total_batches,
       "analysis_start_time": time.time(),
       "trading_week_context": trading_week_context
   }

async def short_term_batch_processing_node(state: ShortTermAnalysisState) -> Dict[str, Any]:
   """Process stocks in batches with comprehensive progress tracking"""
   current_batch = state["current_batch_index"]
   total_batches = state["total_batches"]

   print(f"\n📦 === BATCH PROCESSING (Batch {current_batch + 1}/{total_batches}) ===")

   # Calculate detailed progress
   progress_pct = ((current_batch + 1) / total_batches) * 100
   completed_stocks = current_batch * BATCH_SIZE
   total_stocks = len(state["target_symbols"])

   print(f"📊 Progress: {progress_pct:.1f}% complete")
   print(f"📈 Analyzed: {completed_stocks}/{total_stocks} stocks")
   print(f"🔥 Remaining: {total_stocks - completed_stocks} stocks")

   symbols = state["target_symbols"]
   start_idx = current_batch * BATCH_SIZE
   end_idx = min(start_idx + BATCH_SIZE, len(symbols))
   batch_symbols = symbols[start_idx:end_idx]

   print(f"🎯 Current batch: {', '.join(batch_symbols)}")

   # Enhanced time estimation
   if current_batch > 0:
       elapsed = time.time() - state["analysis_start_time"]
       avg_time_per_batch = elapsed / (current_batch + 1)
       avg_time_per_stock = elapsed / (completed_stocks + len(batch_symbols))
       remaining_batches = total_batches - (current_batch + 1)
       remaining_stocks = total_stocks - (completed_stocks + len(batch_symbols))
       estimated_remaining = remaining_batches * avg_time_per_batch

       print(f"⏱️ Average time per stock: {avg_time_per_stock:.1f} seconds")
       print(f"⏱️ Estimated time remaining: {estimated_remaining/60:.1f} minutes")
       print(f"🚀 Processing speed: {60/avg_time_per_stock:.1f} stocks/minute")

   # Process batch...
   try:
      async with AsyncClient(timeout=300.0) as client:
         batch_results = await process_short_term_batch(batch_symbols, client)
   except Exception as e:
      print(f"❌ MAJOR BATCH ERROR: {e}")
      batch_results = {}
      for symbol in batch_symbols:
         batch_results[symbol] = ShortTermStockAnalysis(
            symbol=symbol,
            company_name=symbol,
            sector="Technology",
            weekly_opportunity_score=4.0,
            final_recommendation="avoid_this_week"
         )

   # Update state
   all_stocks_data = state["all_stocks_data"].copy()
   all_stocks_data.update(batch_results)

   next_batch_index = current_batch + 1
   batch_completed = next_batch_index >= total_batches

   # Enhanced batch summary
   print(f"\n📊 Batch {current_batch + 1} Detailed Results:")
   batch_scores = []
   for symbol, analysis in batch_results.items():
      score = getattr(analysis, 'weekly_opportunity_score', 0) or 0
      recommendation = getattr(analysis, 'final_recommendation', 'unknown') or 'unknown'
      batch_scores.append(score)

      try:
         if hasattr(analysis, 'price_data') and analysis.price_data:
            price_dict = analysis.price_data
            current_price = price_dict.get('current_price', 0) or 0
            predicted_price = price_dict.get('predicted_friday_price', 0) or 0
            predicted_return = price_dict.get('predicted_5day_return', 0) or 0

            print(f"  📈 {symbol}: ${current_price:.2f} → ${predicted_price:.2f} ({predicted_return:+.1f}%)")
            print(f"      🧠 Score: {score:.1f}/10 | 📋 Rec: {recommendation}")

            if hasattr(analysis, 'agent_scores') and analysis.agent_scores:
               agent_scores_str = ", ".join([f"{k}:{v:.1f}" for k, v in analysis.agent_scores.items()])
               print(f"      🤖 Agents: {agent_scores_str}")
         else:
            print(f"  ❌ {symbol}: NO DATA | Score: {score:.1f}/10")
      except Exception as e:
         print(f"  ⚠️ {symbol}: Display error - {e}")

   # Batch statistics
   if batch_scores:
       avg_batch_score = sum(batch_scores) / len(batch_scores)
       max_batch_score = max(batch_scores)
       qualified_in_batch = sum(1 for s in batch_scores if s >= MIN_WEEKLY_SCORE)

       print(f"\n📊 Batch {current_batch + 1} Statistics:")
       print(f"   📈 Average Score: {avg_batch_score:.2f}/10")
       print(f"   🏆 Highest Score: {max_batch_score:.2f}/10")
       print(f"   ✅ Qualified (≥{MIN_WEEKLY_SCORE}): {qualified_in_batch}/{len(batch_scores)}")

   # === COMPREHENSIVE FINAL RESULTS ===
   if batch_completed:
      elapsed = time.time() - state["analysis_start_time"]
      print(f"\n🎉 === COMPREHENSIVE ANALYSIS COMPLETED! ===")
      print(f"⏱️ Total Analysis Time: {elapsed:.1f} seconds ({elapsed/60:.1f} minutes)")
      print(f"📊 Total Stocks Analyzed: {len(all_stocks_data)}")
      print(f"🚀 Average Speed: {len(all_stocks_data)/(elapsed/60):.1f} stocks/minute")

      # Sort ALL stocks by score
      try:
         sorted_stocks = dict(sorted(all_stocks_data.items(),
                                   key=lambda x: getattr(x[1], 'weekly_opportunity_score', 0) or 0,
                                   reverse=True))
      except Exception as e:
         print(f"⚠️ Sorting error: {e}")
         sorted_stocks = all_stocks_data

      # === COMPLETE RANKINGS (ทั้งหมด) ===
      print(f"\n📊 === COMPLETE TECH STOCK RANKINGS (ALL {len(sorted_stocks)} STOCKS) ===")

      # แบ่งเป็นกลุ่ม
      excellent_stocks = []  # 8.5+
      great_stocks = []      # 7.5-8.4
      good_stocks = []       # 6.5-7.4
      average_stocks = []    # 5.5-6.4
      poor_stocks = []       # <5.5

      rank = 1
      for symbol, analysis in sorted_stocks.items():
         try:
            score = getattr(analysis, 'weekly_opportunity_score', 0) or 0
            current_price = 0
            predicted_return = 0

            if hasattr(analysis, 'price_data') and analysis.price_data:
               price_dict = analysis.price_data
               current_price = price_dict.get('current_price', 0) or 0
               predicted_return = price_dict.get('predicted_5day_return', 0) or 0

            rec = getattr(analysis, 'final_recommendation', 'unknown') or 'unknown'

            stock_info = {
               'rank': rank,
               'symbol': symbol,
               'score': score,
               'price': current_price,
               'return': predicted_return,
               'recommendation': rec
            }

            if score >= 8.5:
               excellent_stocks.append(stock_info)
            elif score >= 7.5:
               great_stocks.append(stock_info)
            elif score >= 6.5:
               good_stocks.append(stock_info)
            elif score >= 5.5:
               average_stocks.append(stock_info)
            else:
               poor_stocks.append(stock_info)

            rank += 1
         except Exception as e:
            continue

      # แสดงผลแยกกลุ่ม
      print(f"\n🏆 === EXCELLENT STOCKS (Score ≥ 8.5) === Total: {len(excellent_stocks)}")
      for stock in excellent_stocks:
         print(f"  #{stock['rank']:2d}. {stock['symbol']}: {stock['score']:.1f}/10 | ${stock['price']:.2f} ({stock['return']:+.1f}%) | {stock['recommendation']}")

      print(f"\n🌟 === GREAT STOCKS (Score 7.5-8.4) === Total: {len(great_stocks)}")
      for stock in great_stocks:
         print(f"  #{stock['rank']:2d}. {stock['symbol']}: {stock['score']:.1f}/10 | ${stock['price']:.2f} ({stock['return']:+.1f}%) | {stock['recommendation']}")

      print(f"\n👍 === GOOD STOCKS (Score 6.5-7.4) === Total: {len(good_stocks)}")
      for stock in good_stocks:
         print(f"  #{stock['rank']:2d}. {stock['symbol']}: {stock['score']:.1f}/10 | ${stock['price']:.2f} ({stock['return']:+.1f}%) | {stock['recommendation']}")

      print(f"\n😐 === AVERAGE STOCKS (Score 5.5-6.4) === Total: {len(average_stocks)}")
      for stock in average_stocks[:10]:  # แสดงแค่ 10 ตัวแรก
         print(f"  #{stock['rank']:2d}. {stock['symbol']}: {stock['score']:.1f}/10 | ${stock['price']:.2f} ({stock['return']:+.1f}%) | {stock['recommendation']}")
      if len(average_stocks) > 10:
         print(f"  ... และอีก {len(average_stocks)-10} ตัว")

      print(f"\n👎 === POOR STOCKS (Score < 5.5) === Total: {len(poor_stocks)}")
      for stock in poor_stocks[:5]:  # แสดงแค่ 5 ตัวแรก
         print(f"  #{stock['rank']:2d}. {stock['symbol']}: {stock['score']:.1f}/10 | ${stock['price']:.2f} ({stock['return']:+.1f}%) | {stock['recommendation']}")
      if len(poor_stocks) > 5:
         print(f"  ... และอีก {len(poor_stocks)-5} ตัว")

      # === COMPREHENSIVE STATISTICS ===
      all_scores = [getattr(stock, 'weekly_opportunity_score', 0) or 0 for stock in sorted_stocks.values()]
      qualified_count = len(excellent_stocks) + len(great_stocks)
      positive_return_count = sum(1 for stock in excellent_stocks + great_stocks + good_stocks if stock['return'] > 0)

      print(f"\n📈 === COMPREHENSIVE ANALYSIS STATISTICS ===")
      print(f"📊 Total Stocks Analyzed: {len(all_scores)}")
      print(f"🏆 Excellent (≥8.5): {len(excellent_stocks)} ({len(excellent_stocks)/len(all_scores)*100:.1f}%)")
      print(f"🌟 Great (7.5-8.4): {len(great_stocks)} ({len(great_stocks)/len(all_scores)*100:.1f}%)")
      print(f"👍 Good (6.5-7.4): {len(good_stocks)} ({len(good_stocks)/len(all_scores)*100:.1f}%)")
      print(f"😐 Average (5.5-6.4): {len(average_stocks)} ({len(average_stocks)/len(all_scores)*100:.1f}%)")
      print(f"👎 Poor (<5.5): {len(poor_stocks)} ({len(poor_stocks)/len(all_scores)*100:.1f}%)")
      print(f"✅ Qualified for Portfolio (≥{MIN_WEEKLY_SCORE}): {qualified_count}/{len(all_scores)} ({qualified_count/len(all_scores)*100:.1f}%)")
      print(f"💰 Positive Expected Returns: {positive_return_count}/{len(all_scores)} ({positive_return_count/len(all_scores)*100:.1f}%)")

      if all_scores:
         print(f"📊 Score Distribution:")
         print(f"   Average Score: {sum(all_scores)/len(all_scores):.2f}/10")
         print(f"   Highest Score: {max(all_scores):.2f}/10")
         print(f"   Lowest Score: {min(all_scores):.2f}/10")

      all_stocks_data = sorted_stocks

   return {
      "current_batch_index": next_batch_index,
      "all_stocks_data": all_stocks_data,
      "batch_completed": batch_completed
   }


def emergency_portfolio_selection(all_stocks_data: Dict[str, Any]) -> List[Any]:
    """ระบบสำรองเลือกหุ้นเมื่อไม่มีหุ้นผ่านเงื่อนไขปกติ"""

    print(f"\n🚨 === EMERGENCY FALLBACK ACTIVATED ===")
    print(f"💡 Selecting top {MAX_STOCKS_TO_SELECT} stocks by AI score only")
    print(f"🎯 Target: {MAX_STOCKS_TO_SELECT} stocks (not just 1)")

    # กรองเอาเฉพาะหุ้นที่มีคะแนน AI > 4.0
    valid_stocks = []
    for stock in all_stocks_data.values():
        score = getattr(stock, 'weekly_opportunity_score', 0) or 0
        if score > 4.0:
            valid_stocks.append(stock)

    if not valid_stocks:
        print(f"🚨 CRITICAL: No stocks have AI score > 5.0!")
        # ลดเงื่อนไขอีก
        valid_stocks = [stock for stock in all_stocks_data.values()
                       if (getattr(stock, 'weekly_opportunity_score', 0) or 0) > 0]

    if not valid_stocks:
        print(f"🚨 SYSTEM FAILURE: No stocks have any valid AI scores!")
        return []

    # เรียงตามคะแนน AI
    valid_stocks.sort(key=lambda x: getattr(x, 'weekly_opportunity_score', 0) or 0, reverse=True)

    # เลือกหุ้นดีที่สุด
    # เลือกหุ้นดีที่สุด - ต้องการ 5 ตัว
    target_count = min(MAX_STOCKS_TO_SELECT, len(valid_stocks))
    emergency_selection = valid_stocks[:target_count]
    print(f"🔍 DEBUG Emergency selection:")
    for i, stock in enumerate(emergency_selection):
        score = getattr(stock, 'weekly_opportunity_score', 0)
        print(f"   Emergency {i+1}. {stock.symbol}: {score:.1f}/10")

    print(f"\n🏥 === EMERGENCY PORTFOLIO (TOP {len(emergency_selection)} STOCKS) ===")
    print(f"🎯 Selected {len(emergency_selection)} out of {len(valid_stocks)} valid stocks")
    print(f"📊 Target was {MAX_STOCKS_TO_SELECT} stocks")

    if len(emergency_selection) < MAX_STOCKS_TO_SELECT:
        print(f"⚠️ WARNING: Only {len(emergency_selection)} stocks met minimum criteria")
        print(f"💡 Consider lowering MIN_WEEKLY_SCORE from {MIN_WEEKLY_SCORE} to get more stocks")

    for i, stock in enumerate(emergency_selection):
        score = getattr(stock, 'weekly_opportunity_score', 0) or 0

        predicted_return = 0
        current_price = 0
        if hasattr(stock, 'price_data') and stock.price_data:
            predicted_return = stock.price_data.get('predicted_5day_return', 0) or 0
            current_price = stock.price_data.get('current_price', 0) or 0

        print(f"   {i+1}. {stock.symbol}: AI Score {score:.1f}/10")
        print(f"      💰 Price: ${current_price:.2f}, Return: {predicted_return:+.1f}%")
        print(f"      🏥 Status: Emergency selection")

    print(f"\n⚠️ === EMERGENCY MODE WARNINGS ===")
    print(f"🚨 These stocks bypassed normal quality filters")
    print(f"💡 Use extra caution and smaller position sizes")

    return emergency_selection


async def short_term_portfolio_selector_node(state: ShortTermAnalysisState) -> Dict[str, Any]:
  """Select stocks for weekly trading portfolio - ULTIMATE RESEARCH VERSION"""
  print(f"\n🎯 === ULTIMATE RESEARCH-GRADE PORTFOLIO SELECTION ===")

  all_stocks = state["all_stocks_data"]
  analysis_start_time = state["analysis_start_time"]
  trading_week_context = state["trading_week_context"]

  # === ENHANCED FILTERING SYSTEM ===
  def enhanced_stock_filtering(all_stocks_data: Dict[str, Any]) -> tuple:
     """กรองหุ้นด้วยเกณฑ์ที่ถูกต้องและเข้าถึงได้จริง"""

     qualified_stocks = []
     rejection_reasons = {}
     detailed_analysis = {}

     print(f"\n🔍 === ENHANCED FILTERING (REALISTIC & ACCESSIBLE) ===")
     print(f"📊 Filtering {len(all_stocks_data)} stocks with REALISTIC criteria:")
     print(f"   💯 Min AI Score: {MIN_WEEKLY_SCORE}/10 (LOWERED from 7.0)")
     print(f"   📈 Min Expected Return: {MIN_POSITIVE_RETURN:+.1f}% (LOWERED from 0.0%)")
     print(f"   🎯 Min Confidence: {MIN_CONFIDENCE_LEVEL:.0%} (LOWERED from 20%)")
     print(f"   💧 Min Liquidity: {MIN_LIQUIDITY_THRESHOLD}/10 (LOWERED from 5.0)")
     print(f"   ⚖️ Min Risk-Reward: {MIN_RISK_REWARD_RATIO:.2f} (LOWERED from 0.30)")
     print(f"   ⚡ Momentum: Accept ALL except strong_bearish (RELAXED)")
     print(f"   📰 News Catalyst: OPTIONAL (not required)")
     print(f"   🎯 Agent Tolerance: Accept up to 5 low scores (RELAXED)")
     print(f"")
     print(f"🎯 Goal: Select quality stocks that can actually pass these criteria")

     for symbol, stock in all_stocks_data.items():
        rejection_reason = []
        analysis_details = {}

        # เงื่อนไข 1: AI Score (ปรับเป็น 6.0)
        score = getattr(stock, 'weekly_opportunity_score', 0) or 0
        analysis_details['ai_score'] = score
        if score < MIN_WEEKLY_SCORE:
            rejection_reason.append(f"Low AI score: {score:.1f} < {MIN_WEEKLY_SCORE}")

        # เงื่อนไข 2: Predicted Return (ปรับเป็น -1.0%)
        predicted_return = 0
        prediction_confidence = 0
        if hasattr(stock, 'price_data') and stock.price_data:
            predicted_return = stock.price_data.get('predicted_5day_return', 0) or 0
            prediction_confidence = stock.price_data.get('prediction_confidence', 0) or 0

        analysis_details['predicted_return'] = predicted_return
        analysis_details['prediction_confidence'] = prediction_confidence

        if predicted_return < MIN_POSITIVE_RETURN:
            rejection_reason.append(f"Very low return: {predicted_return:+.1f}% < {MIN_POSITIVE_RETURN:+.1f}%")

        # เงื่อนไข 3: Prediction Confidence (ปรับเป็น 15%)
        if prediction_confidence < MIN_CONFIDENCE_LEVEL:
            rejection_reason.append(f"Very low confidence: {prediction_confidence:.1%} < {MIN_CONFIDENCE_LEVEL:.0%}")

        # เงื่อนไข 4: Success Probability (ไม่บังคับ - ตั้งเป็น 0.0)
        success_probability = 0
        if hasattr(stock, 'timing_data') and stock.timing_data:
            success_probability = stock.timing_data.get('success_probability', 0) or 0

        analysis_details['success_probability'] = success_probability
        # ไม่ตรวจสอบเงื่อนไขนี้เพราะตั้งเป็น 0.0

        # เงื่อนไข 5: Technical Momentum (ปรับให้ผ่อนปรนมาก)
        momentum_strength = ""
        volume_confirmation = False
        if hasattr(stock, 'technical_swing_data') and stock.technical_swing_data:
            momentum_strength = stock.technical_swing_data.get('momentum_strength', '')
            volume_confirmation = stock.technical_swing_data.get('volume_confirmation', False)

        analysis_details['momentum_strength'] = momentum_strength
        analysis_details['volume_confirmation'] = volume_confirmation

        # เข้มงวดแค่ strong_bearish เท่านั้น
        if momentum_strength == 'strong_bearish':
            rejection_reason.append(f"Strong bearish momentum: {momentum_strength}")

        # เงื่อนไข 6: Liquidity Score (ปรับเป็น 2.0)
        liquidity_score = 0
        if hasattr(stock, 'volume_liquidity_data') and stock.volume_liquidity_data:
            liquidity_score = stock.volume_liquidity_data.get('liquidity_score', 0) or 0

        analysis_details['liquidity_score'] = liquidity_score
        if liquidity_score < MIN_LIQUIDITY_THRESHOLD:
            rejection_reason.append(f"Very low liquidity: {liquidity_score:.1f} < {MIN_LIQUIDITY_THRESHOLD}")

        # เงื่อนไข 7: Risk-Reward Ratio (ปรับเป็น 0.05)
        risk_reward_ratio = 0
        five_day_var = 0
        if hasattr(stock, 'short_term_risk_data') and stock.short_term_risk_data:
            risk_reward_ratio = stock.short_term_risk_data.get('risk_reward_ratio', 0) or 0
            five_day_var = stock.short_term_risk_data.get('five_day_var', 0) or 0

        analysis_details['risk_reward_ratio'] = risk_reward_ratio
        analysis_details['five_day_var'] = five_day_var

        if risk_reward_ratio < MIN_RISK_REWARD_RATIO:
            rejection_reason.append(f"Very low risk-reward: {risk_reward_ratio:.2f} < {MIN_RISK_REWARD_RATIO:.2f}")

        # เงื่อนไข 8: Agent Score Consistency (ปรับให้ผ่อนปรนมาก)
        agent_scores = getattr(stock, 'agent_scores', {})
        very_low_agent_scores = [name for name, score in agent_scores.items() if score < 3.0]  # ลดจาก 4.0 เป็น 3.0
        analysis_details['agent_scores'] = agent_scores
        analysis_details['very_low_agent_scores'] = very_low_agent_scores

        if len(very_low_agent_scores) > 5:  # อนุญาตให้มี 5 agents ที่คะแนนต่ำได้
            rejection_reason.append(f"Too many very low agent scores: {len(very_low_agent_scores)}/6")

        # เงื่อนไข 9: Price Trend (ปรับให้ผ่อนปรนมาก)
        daily_momentum = {}
        if hasattr(stock, 'price_data') and stock.price_data:
            daily_momentum = stock.price_data.get('daily_momentum', {})

        momentum_trend = daily_momentum.get('momentum_trend', 'neutral')
        recent_5d_avg = daily_momentum.get('recent_5d_avg', 0)
        analysis_details['momentum_trend'] = momentum_trend
        analysis_details['recent_5d_avg'] = recent_5d_avg

        if momentum_trend == 'negative' and recent_5d_avg < -5.0:  # ลดจาก -3.0% เป็น -5.0%
            rejection_reason.append(f"Extremely negative trend: {recent_5d_avg:.1f}%")

        # บันทึกผลการวิเคราะห์
        detailed_analysis[symbol] = analysis_details

        # ผลการตัดสิน (ผ่อนปรนมาก - ต้องมีปัญหาใหญ่ 2 ข้อขึ้นไปถึงจะถูกปฏิเสธ)
        major_issues = len([r for r in rejection_reason if 'Very low' in r or 'Extremely' in r or 'Strong bearish' in r])
        if major_issues >= 2:
            rejection_reasons[symbol] = rejection_reason
            print(f"   ❌ {symbol}: {', '.join(rejection_reason[:2])}")
            if len(rejection_reason) > 2:
                print(f"       + {len(rejection_reason)-2} more issues")
        else:
            qualified_stocks.append(stock)
            if len(rejection_reason) == 0:
                print(f"   ✅ {symbol}: Score {score:.1f}, Return {predicted_return:+.1f}%, Perfect!")
            elif len(rejection_reason) == 1:
                print(f"   ✅ {symbol}: Score {score:.1f}, Return {predicted_return:+.1f}% (1 minor issue)")
            else:
                print(f"   ✅ {symbol}: Score {score:.1f}, Return {predicted_return:+.1f}% (minor issues: {len(rejection_reason)})")

     # แสดงสถิติการกรอง
     print(f"\n📊 === REALISTIC FILTERING RESULTS ===")
     print(f"✅ Qualified: {len(qualified_stocks)}/{len(all_stocks_data)} ({len(qualified_stocks)/len(all_stocks_data)*100:.1f}%)")
     print(f"❌ Rejected: {len(rejection_reasons)}/{len(all_stocks_data)} ({len(rejection_reasons)/len(all_stocks_data)*100:.1f}%)")
     print(f"🔍 DEBUG: Qualified stocks list:")
     for i, stock in enumerate(qualified_stocks[:10]):
         score = getattr(stock, 'weekly_opportunity_score', 0)
         print(f"   {i+1}. {stock.symbol}: {score:.1f}/10")
     if len(qualified_stocks) > 10:
         print(f"   ... and {len(qualified_stocks)-10} more")

     if len(qualified_stocks) == 0:
         print(f"🚨 CRITICAL: Still no stocks qualified!")
         print(f"💡 Consider using emergency fallback: select top 5 by score regardless of other criteria")
     elif len(qualified_stocks) < MAX_STOCKS_TO_SELECT:
         print(f"⚠️ Only {len(qualified_stocks)} stocks qualified (need {MAX_STOCKS_TO_SELECT})")
         print(f"💡 Will select all {len(qualified_stocks)} qualified stocks")
     else:
         print(f"🎉 SUCCESS: {len(qualified_stocks)} stocks qualified (enough for portfolio)")

     return qualified_stocks, rejection_reasons, detailed_analysis

  # === MARKET CONDITION DETECTION ===
  def detect_market_condition(qualified_stocks: list) -> Dict[str, Any]:
     """ตรวจจับสภาวะตลาดขั้นสูง"""

     if not qualified_stocks:
        return {
           "condition": "BEARISH/UNSUITABLE",
           "recommendation": "DO_NOT_TRADE_THIS_WEEK",
           "reason": "No stocks meet research-grade criteria",
           "action": "Wait for better market conditions",
           "confidence": "HIGH",
           "risk_level": "AVOID",
           "market_quality": "POOR"
        }

     # คำนวณสถิติขั้นสูง
     returns = []
     scores = []
     confidences = []
     success_probs = []
     risk_rewards = []

     for stock in qualified_stocks:
        if hasattr(stock, 'price_data') and stock.price_data:
           returns.append(stock.price_data.get('predicted_5day_return', 0) or 0)
           confidences.append(stock.price_data.get('prediction_confidence', 0) or 0)

        scores.append(getattr(stock, 'weekly_opportunity_score', 0) or 0)

        if hasattr(stock, 'timing_data') and stock.timing_data:
           success_probs.append(stock.timing_data.get('success_probability', 0) or 0)

        if hasattr(stock, 'short_term_risk_data') and stock.short_term_risk_data:
           risk_rewards.append(stock.short_term_risk_data.get('risk_reward_ratio', 0) or 0)

     # คำนวณค่าเฉลี่ยและสถิติ
     avg_predicted_return = np.mean(returns) if returns else 0
     avg_score = np.mean(scores) if scores else 0
     avg_confidence = np.mean(confidences) if confidences else 0
     avg_success_prob = np.mean(success_probs) if success_probs else 0
     avg_risk_reward = np.mean(risk_rewards) if risk_rewards else 0

     # นับหุ้นที่มีคุณภาพสูง
     high_return_stocks = sum(1 for r in returns if r >= 3.0)
     excellent_stocks = sum(1 for s in scores if s >= 8.5)
     high_confidence_stocks = sum(1 for c in confidences if c >= 0.8)

     # ประเมินสภาวะตลาดแบบละเอียด
     if (len(qualified_stocks) >= 8 and avg_predicted_return >= 4.0 and
         avg_score >= 8.5 and avg_confidence >= 0.8):
        condition = "EXTREMELY_BULLISH"
        recommendation = "MAXIMUM_BUY_SIGNAL"
        confidence = "VERY_HIGH"
        risk_level = "LOW"
        market_quality = "EXCEPTIONAL"
     elif (len(qualified_stocks) >= 5 and avg_predicted_return >= 3.0 and
           avg_score >= 8.0 and avg_confidence >= 0.75):
        condition = "VERY_BULLISH"
        recommendation = "STRONG_BUY_SIGNAL"
        confidence = "HIGH"
        risk_level = "LOW"
        market_quality = "EXCELLENT"
     elif (len(qualified_stocks) >= 3 and avg_predicted_return >= 2.5 and
           avg_score >= 7.5 and avg_confidence >= 0.7):
        condition = "BULLISH"
        recommendation = "BUY_SIGNAL"
        confidence = "HIGH"
        risk_level = "MEDIUM"
        market_quality = "VERY_GOOD"
     elif (len(qualified_stocks) >= 1 and avg_score >= 7.0):
        condition = "MODERATELY_BULLISH"
        recommendation = "CAUTIOUS_BUY"
        confidence = "MEDIUM"
        risk_level = "MEDIUM"
        market_quality = "GOOD"
     else:
        condition = "NEUTRAL_TO_BEARISH"
        recommendation = "DO_NOT_TRADE"
        confidence = "HIGH"
        risk_level = "HIGH"
        market_quality = "POOR"

     return {
        "condition": condition,
        "recommendation": recommendation,
        "confidence": confidence,
        "risk_level": risk_level,
        "market_quality": market_quality,
        "qualified_count": len(qualified_stocks),
        "avg_predicted_return": round(avg_predicted_return, 2),
        "avg_score": round(avg_score, 2),
        "avg_confidence": round(avg_confidence, 3),
        "avg_success_prob": round(avg_success_prob, 3),
        "avg_risk_reward": round(avg_risk_reward, 2),
        "high_return_count": high_return_stocks,
        "excellent_count": excellent_stocks,
        "high_confidence_count": high_confidence_stocks,
        "portfolio_strength": "STRONG" if excellent_stocks >= 3 else "MODERATE" if excellent_stocks >= 1 else "WEAK"
     }

  # === COMPOSITE SCORING SYSTEM ===
  def calculate_ai_composite_score(stock) -> float:
     """คำนวณคะแนนรวมจาก AI agents เท่านั้น"""

     agent_scores = getattr(stock, 'agent_scores', {})
     if not agent_scores:
         raise Exception(f"No AI agent scores for {stock.symbol}")

     # น้ำหนักตาม AI expertise
     ai_weights = {
         "momentum": 0.20,    # AI momentum analysis
         "technical": 0.20,   # AI technical analysis
         "catalyst": 0.20,    # AI news analysis
         "risk": 0.20,       # AI risk analysis
         "liquidity": 0.10,   # AI liquidity analysis
         "timing": 0.10       # AI timing analysis

     }

     # คำนวณคะแนนถ่วงน้ำหนักจาก AI เท่านั้น
     weighted_score = sum(agent_scores.get(agent, 0) * weight
                         for agent, weight in ai_weights.items())

     # AI consensus bonus - เมื่อ AI หลายตัวเห็นด้วย
     ai_consensus_bonus = 0
     high_scoring_agents = sum(1 for score in agent_scores.values() if score >= 8.0)
     if high_scoring_agents >= 4:  # AI consensus แข็งแกร่ง
         ai_consensus_bonus = 0.5
     elif high_scoring_agents >= 3:
         ai_consensus_bonus = 0.3

     # AI consistency bonus - AI ให้คะแนนสม่ำเสมอ
     score_std = np.std(list(agent_scores.values())) if len(agent_scores) > 1 else 0
     consistency_bonus = max(0, (3 - score_std) * 0.1)  # ยิ่งสม่ำเสมอยิ่งดี

     # AI expected return integration - ใช้ AI prediction
     ai_return_bonus = 0
     if hasattr(stock, 'price_data') and stock.price_data:
         predicted_return = stock.price_data.get('predicted_5day_return', 0) or 0
         ai_confidence = stock.price_data.get('prediction_confidence', 0) or 0

         # Bonus สำหรับ AI prediction ที่ดี
         if predicted_return > 3 and ai_confidence > 0.7:
             ai_return_bonus = 0.8
         elif predicted_return > 1 and ai_confidence > 0.6:
             ai_return_bonus = 0.4

     final_score = weighted_score + ai_consensus_bonus + consistency_bonus + ai_return_bonus

     print(f"🤖 AI Composite for {stock.symbol}: Base={weighted_score:.1f} + Consensus={ai_consensus_bonus:.1f} + Consistency={consistency_bonus:.1f} + Return={ai_return_bonus:.1f} = {final_score:.1f}")

     return min(10.0, max(0.0, final_score))

  # === รันการกรองและวิเคราะห์ ===
  qualified_stocks, rejection_reasons, detailed_analysis = enhanced_stock_filtering(all_stocks)
  if len(qualified_stocks) == 0:
     print(f"🚨 === ACTIVATING EMERGENCY FALLBACK ===")
     qualified_stocks = emergency_portfolio_selection(all_stocks)

     if len(qualified_stocks) == 0:
        print(f"🚨 SYSTEM FAILURE: Even emergency fallback failed!")
        return {"weekly_portfolio": ShortTermPortfolioRecommendation(
            selected_stocks=[],
            trading_plan={"week_status": "SYSTEM_FAILURE - No stocks available"}
        )}

     # ปรับ market condition สำหรับ emergency mode
     market_condition = {
         "condition": "EMERGENCY_SELECTION",
         "recommendation": "EXTRA_CAUTION_REQUIRED",
         "confidence": "LOW",
         "risk_level": "HIGH",
         "market_quality": "POOR",
         "emergency_mode": True
     }
  else:
     market_condition = detect_market_condition(qualified_stocks)


  print(f"\n📊 === ADVANCED MARKET CONDITION ANALYSIS ===")
  print(f"🎯 Market Condition: {market_condition['condition']}")
  print(f"💡 Recommendation: {market_condition['recommendation']}")
  print(f"🎲 Confidence Level: {market_condition['confidence']}")
  print(f"⚠️ Risk Level: {market_condition['risk_level']}")
  print(f"🏆 Market Quality: {market_condition['market_quality']}")
  print(f"✅ Qualified Stocks: {market_condition.get('qualified_count', 0)}")
  print(f"📈 Avg Expected Return: {market_condition.get('avg_predicted_return', 0):+.1f}%")
  print(f"💯 Avg AI Score: {market_condition.get('avg_score', 0):.1f}/10")
  print(f"🎯 Avg Confidence: {market_condition.get('avg_confidence', 0):.1%}")
  print(f"🎲 Avg Success Prob: {market_condition.get('avg_success_prob', 0):.1%}")
  print(f"⚖️ Avg Risk-Reward: {market_condition.get('avg_risk_reward', 0):.2f}")
  print(f"🌟 Excellent Stocks: {market_condition.get('excellent_count', 0)}")
  print(f"🚀 High Return Stocks: {market_condition.get('high_return_count', 0)}")

  # === ตัดสินใจการลงทุน ===
  if market_condition['recommendation'] in ['DO_NOT_TRADE_THIS_WEEK']:
     print(f"\n🚨 === INVESTMENT DECISION: DO NOT TRADE ===")
     print(f"❌ This week is not suitable for trading")
     print(f"💡 Reason: {market_condition.get('reason', 'Market conditions not favorable')}")
     print(f"🎯 Market Quality: {market_condition.get('market_quality', 'POOR')}")

     # แสดงสาเหตุที่ถูกปฏิเสธแบบละเอียด
     print(f"\n📋 === DETAILED REJECTION ANALYSIS ===")
     rejection_stats = {}
     for symbol, reasons in rejection_reasons.items():
        for reason in reasons:
           category = reason.split(':')[0]
           if category not in rejection_stats:
              rejection_stats[category] = 0
           rejection_stats[category] += 1

     print(f"📊 Most Common Rejection Reasons:")
     for reason, count in sorted(rejection_stats.items(), key=lambda x: x[1], reverse=True):
        print(f"   • {reason}: {count} stocks")

     print(f"\n📋 === TOP 15 REJECTED STOCKS ===")
     for i, (symbol, reasons) in enumerate(list(rejection_reasons.items())[:15]):
        analysis = detailed_analysis.get(symbol, {})
        score = analysis.get('ai_score', 0)
        pred_return = analysis.get('predicted_return', 0)
        print(f"   {i+1:2d}. {symbol}: Score {score:.1f}, Return {pred_return:+.1f}%")
        print(f"       Issues: {', '.join(reasons[:3])}")
        if len(reasons) > 3:
           print(f"       + {len(reasons)-3} more issues")

     return {"weekly_portfolio": ShortTermPortfolioRecommendation(
        selected_stocks=[],
        trading_plan={
           "week_status": f"DO NOT TRADE - {market_condition['condition']}",
           "reason": market_condition.get('reason', 'Market conditions unfavorable'),
           "market_condition": market_condition,
           "qualified_stocks": len(qualified_stocks),
           "total_analyzed": len(all_stocks),
           "rejection_stats": rejection_stats,
           "recommendation": "Wait for better market opportunities"
        },
        processing_stats={
           "analysis_result": "NO_TRADE_RECOMMENDED",
           "market_quality": market_condition.get('market_quality', 'POOR'),
           "qualified_rate": round(len(qualified_stocks) / len(all_stocks) * 100, 1),
           "research_grade": True
        }
     )}

  # === เลือกหุ้นชั้นเลิศ ===
  if not qualified_stocks:
     print(f"❌ No qualified stocks after enhanced filtering")
     return {"weekly_portfolio": ShortTermPortfolioRecommendation(
        selected_stocks=[],
        trading_plan={"week_status": "No qualified stocks meet criteria"}
     )}

  # เรียงลำดับตาม AI composite score
  qualified_stocks.sort(key=calculate_ai_composite_score, reverse=True)

#   # เลือกหุ้นดีที่สุด
#   num_to_select = min(MAX_STOCKS_TO_SELECT, len(qualified_stocks))
#   selected_stocks = qualified_stocks[:num_to_select]
# เลือกหุ้นดีที่สุด 5 ตัว (หรือทั้งหมดถ้ามีน้อยกว่า 5)
  if len(qualified_stocks) < MAX_STOCKS_TO_SELECT:
      print(f"🚨 Need more stocks! Only {len(qualified_stocks)}, need {MAX_STOCKS_TO_SELECT}")
      print(f"🚨 Adding emergency selections...")
      emergency_stocks = emergency_portfolio_selection(all_stocks)
      added_count = 0
      for stock in emergency_stocks:
          if stock.symbol not in [s.symbol for s in qualified_stocks]:
              qualified_stocks.append(stock)
              added_count += 1
              print(f"   ➕ Added {stock.symbol}")
              if len(qualified_stocks) >= MAX_STOCKS_TO_SELECT:
                break
      print(f"✅ Added {added_count} emergency stocks, total now: {len(qualified_stocks)}")

  # เรียงตามคะแนน
  qualified_stocks.sort(key=lambda x: getattr(x, 'weekly_opportunity_score', 0) or 0, reverse=True)

  # เลือกหุ้น MAX_STOCKS_TO_SELECT ตัว
  num_to_select = min(MAX_STOCKS_TO_SELECT, len(qualified_stocks))
  selected_stocks = qualified_stocks[:num_to_select]

  print(f"\n🔍 === PORTFOLIO SELECTION DEBUG ===")
  print(f"📊 Qualified stocks: {len(qualified_stocks)}")
  print(f"🎯 MAX_STOCKS_TO_SELECT: {MAX_STOCKS_TO_SELECT}")
  print(f"🔢 num_to_select: {num_to_select}")
  print(f"✅ Actually selected: {len(selected_stocks)} stocks")

  print(f"\n🏆 === FINAL SELECTION ({len(selected_stocks)} STOCKS) ===")
  for i, stock in enumerate(selected_stocks):
     score = getattr(stock, 'weekly_opportunity_score', 0)
     print(f"   {i+1}. {stock.symbol}: {score:.1f}/10")

  # === สร้าง PORTFOLIO ANALYSIS ===
  portfolio_stocks = []
  total_composite_score = sum(calculate_ai_composite_score(stock) for stock in selected_stocks)

  for i, stock in enumerate(selected_stocks):
      print(f"🔄 Processing stock {i+1}/{len(selected_stocks)}: {stock.symbol}")

      # คำนวณน้ำหนักตาม composite score
      composite_score = calculate_ai_composite_score(stock)
      if total_composite_score > 0:
           score_weight = (composite_score / total_composite_score) * 100
      else:
           score_weight = 100 / len(selected_stocks)

      # ปรับน้ำหนักให้สมดุลสำหรับ portfolio 5 ตัว
      base_weight = 100 / len(selected_stocks)  # เฉลี่ย 20% ต่อตัวถ้ามี 5 ตัว
      min_weight = base_weight * 0.6   # ขั้นต่ำ 12% ต่อตัว
      max_weight = base_weight * 1.6   # สูงสุด 32% ต่อตัว
      adjusted_weight = max(min_weight, min(score_weight, max_weight))

      print(f"       💼 Position {i+1}: {adjusted_weight:.1f}% allocation")
      money_allocation = (adjusted_weight / 100) * MONEY
      print(f"       💰 Investment: ${money_allocation:,.0f} ({adjusted_weight:.1f}% of ${MONEY:,})")

      # ข้อมูลราคาและการคาดการณ์
      price_data = stock.price_data or {}
      timing_data = stock.timing_data or {}
      technical_data = stock.technical_swing_data or {}
      risk_data = stock.short_term_risk_data or {}
      news_data = stock.news_catalyst_data or {}

      current_price = price_data.get('current_price', 0) or 0
      predicted_friday_price = price_data.get('predicted_friday_price', 0) or 0
      predicted_5day_return = price_data.get('predicted_5day_return', 0) or 0
      prediction_confidence = price_data.get('prediction_confidence', 0) or 0

      entry_level = technical_data.get('suggested_entry_level') or current_price
      exit_level = technical_data.get('suggested_exit_level') or predicted_friday_price
      stop_loss = technical_data.get('stop_loss_level') or current_price * 0.95

      success_probability = timing_data.get('success_probability', 0.7) or 0.7
      optimal_entry_day = timing_data.get('best_entry_time', 'monday_open') or 'monday_open'
      optimal_exit_day = timing_data.get('best_exit_time', 'friday_close') or 'friday_close'

      # ข้อมูลความเสี่ยงขั้นสูง
      five_day_var = risk_data.get('five_day_var')
      risk_reward_ratio = risk_data.get('risk_reward_ratio', 1.0) or 1.0

      if five_day_var is not None:
           max_expected_loss = round(abs(five_day_var) * (adjusted_weight / 100), 1)
      else:
           max_expected_loss = round(3.0 * (adjusted_weight / 100), 1)

      # ข้อมูล catalyst
      catalyst_strength = news_data.get('catalyst_strength', 'none')
      breaking_news_count = len(news_data.get('breaking_news', []))

      # สร้างข้อมูลหุ้นแบบครบถ้วน
      portfolio_stock = {
           "rank": i + 1,
           "symbol": stock.symbol,
           "company_name": getattr(stock, 'company_name', stock.symbol),
           "sector": "Technology",

           # คะแนนและการประเมิน
           "composite_score": round(composite_score, 2),
           "weekly_opportunity_score": getattr(stock, 'weekly_opportunity_score', 0),
           "final_recommendation": getattr(stock, 'final_recommendation', 'strong_buy'),
           "confidence_score": min(getattr(stock, 'weekly_opportunity_score', 0) / 10, 1.0),
           "qualification_margin": getattr(stock, 'weekly_opportunity_score', 0) - MIN_WEEKLY_SCORE,
           "research_grade": "EXCELLENT" if composite_score >= 12 else "VERY_GOOD" if composite_score >= 10 else "GOOD",

           # การวิเคราะห์ราคา
           "current_price": current_price,
           "predicted_friday_price": predicted_friday_price,
           "predicted_5day_return": predicted_5day_return,
           "prediction_confidence": prediction_confidence,

           # การจัดการพอร์ต
           "suggested_percentage_allocation": round(adjusted_weight, 1),
           "recommended_position_size": round(adjusted_weight, 1),
           "position_justification": f"Composite score {composite_score:.2f} (AI: {getattr(stock, 'weekly_opportunity_score', 0):.1f}/10)",

           # แผนการเทรดขั้นสูง
           "trading_plan": {
                "optimal_entry_day": optimal_entry_day,
                "optimal_exit_day": optimal_exit_day,
                "entry_price_target": round(entry_level, 2),
                "exit_price_target": round(exit_level, 2),
                "stop_loss_level": round(stop_loss, 2),
                "expected_5day_return": round(predicted_5day_return, 1),
                "success_probability": round(success_probability * 100, 1),
                "hold_duration": "5 trading days (Monday → Friday)",
                "strategy_type": "Research-Grade Momentum Swing",
                "risk_reward_ratio": round(risk_reward_ratio, 2)
           },

           # การวิเคราะห์ละเอียดจาก agents
           "agent_scores": {
                "momentum": getattr(stock, 'agent_scores', {}).get('momentum', 0) or 0,
                "catalyst": getattr(stock, 'agent_scores', {}).get('catalyst', 0) or 0,
                "technical": getattr(stock, 'agent_scores', {}).get('technical', 0) or 0,
                "liquidity": getattr(stock, 'agent_scores', {}).get('liquidity', 0) or 0,
                "risk": getattr(stock, 'agent_scores', {}).get('risk', 0) or 0,
                "timing": getattr(stock, 'agent_scores', {}).get('timing', 0) or 0
           },

           # การวิเคราะห์ความเสี่ยงขั้นสูง
           "risk_metrics": {
                "five_day_var": five_day_var if five_day_var is not None else -3.0,
                "risk_reward_ratio": risk_reward_ratio,
                "max_expected_loss": max_expected_loss,
                "position_risk_level": "low" if risk_reward_ratio > 2.0 else "moderate" if risk_reward_ratio > 1.5 else "high",
                "volatility_assessment": risk_data.get('current_volatility_vs_optimal', 'optimal'),
                "correlation_with_qqq": risk_data.get('correlation_with_qqq', 0.7)
           },

           # ข้อมูลตัวเร่งและโมเมนตัม
           "catalyst_analysis": {
                "catalyst_strength": catalyst_strength,
                "breaking_news_count": breaking_news_count,
                "momentum_strength": technical_data.get('momentum_strength', 'bullish'),
                "volume_confirmation": technical_data.get('volume_confirmation', True),
                "sentiment_trend": news_data.get('weekly_sentiment_trend', {}).get('trend', 'stable')
           },

           # ความโปร่งใสของข้อมูล
           "data_sources_count": len(getattr(stock, 'all_data_sources', [])),
           "data_sources": [
                {
                     "name": source.source_name,
                     "url": source.source_url,
                     "success": source.success
                } for source in getattr(stock, 'all_data_sources', [])[:5]
           ],

           # เมตาดาต้าการวิเคราะห์
           "analysis_quality": "exceptional" if prediction_confidence > 0.85 else "high" if prediction_confidence > 0.75 else "good",
           "recommendation_strength": "very_strong" if getattr(stock, 'weekly_opportunity_score', 0) >= 9.0 else "strong" if getattr(stock, 'weekly_opportunity_score', 0) >= 8.5 else "moderate",
           "selection_tier": "tier_1" if i < 2 else "tier_2" if i < 4 else "tier_3"
      }

      portfolio_stocks.append(portfolio_stock)
      print(f"       ✅ Added {stock.symbol} to portfolio_stocks (total: {len(portfolio_stocks)})")

      # แสดงการวิเคราะห์รายตัวแบบละเอียด
      print(f"\n       🏆 #{i+1}. {stock.symbol} - COMPREHENSIVE ANALYSIS")
      print(f"            🧮 Composite Score: {composite_score:.2f} (Research Grade)")
      print(f"            💯 AI Score: {getattr(stock, 'weekly_opportunity_score', 0):.2f}/10 (Margin: +{getattr(stock, 'weekly_opportunity_score', 0) - MIN_WEEKLY_SCORE:.1f})")
      print(f"            💰 Price Analysis: ${current_price:.2f} → ${predicted_friday_price:.2f} ({predicted_5day_return:+.1f}%)")
      print(f"            🎯 Confidence: {prediction_confidence:.1%} | Success Prob: {success_probability:.1%}")
      print(f"            📈 Allocation: {adjusted_weight:.1f}% | Max Loss: {max_expected_loss:.1f}%")
      print(f"            📅 Timing: {optimal_entry_day} → {optimal_exit_day}")
      print(f"            ⚖️ Risk-Reward: {risk_reward_ratio:.2f}:1 | VaR: {five_day_var:.1f}%" if five_day_var else f"            ⚖️ Risk-Reward: {risk_reward_ratio:.2f}:1")
      print(f"            🏅 Grade: {portfolio_stock['research_grade']} | Quality: {portfolio_stock['analysis_quality']}")

      # แสดงคะแนน agents แบบละเอียด
      agent_scores = getattr(stock, 'agent_scores', {})
      if agent_scores:
           print(f"            🧠 Agent Analysis:")
           print(f"                 Momentum: {agent_scores.get('momentum', 0):.1f} | Catalyst: {agent_scores.get('catalyst', 0):.1f} | Technical: {agent_scores.get('technical', 0):.1f}")
           print(f"                 Liquidity: {agent_scores.get('liquidity', 0):.1f} | Risk: {agent_scores.get('risk', 0):.1f} | Timing: {agent_scores.get('timing', 0):.1f}")
           print(f"                 Consistency: {np.std(list(agent_scores.values())):.1f} (lower is better)")

      # แสดงข้อมูล catalyst และโมเมนตัม
      print(f"            📰 Catalyst: {catalyst_strength} ({breaking_news_count} news)")
      print(f"            ⚡ Momentum: {technical_data.get('momentum_strength', 'unknown')} | Volume: {'✓' if technical_data.get('volume_confirmation', False) else '✗'}")

  # จบ Loop ตรงนี้!
  print(f"🎯 Final portfolio_stocks count: {len(portfolio_stocks)}")

 # === การคำนวณผลตอบแทนพอร์ตขั้นสูง ===
  # ใช้ AI agent predictions แทน
  expected_portfolio_return = 0
  for stock in portfolio_stocks:
      # ลองใช้ return จาก momentum agent ก่อน
      ai_return = stock.get('momentum_agent_analysis', {}).get('expected_5day_return', 0)
      if not ai_return:
          # ถ้าไม่มีใช้จาก price prediction
          ai_return = stock['predicted_5day_return'] or 0

      weight = stock['suggested_percentage_allocation'] / 100
      expected_portfolio_return += ai_return * weight

  print(f"🤖 Portfolio Expected Return (AI-driven): {expected_portfolio_return:+.1f}%")

  confidence_weighted_return = sum(
      (stock['predicted_5day_return'] or 0) *
      (stock['prediction_confidence'] or 0.5) *
      (stock['suggested_percentage_allocation'] / 100)
      for stock in portfolio_stocks
  )

  # === แผนการเทรดขั้นสูง ===
  trading_plan = {
      "strategy": "Research-Grade 5-Day Tech Swing Trading",
      "trading_week": f"{trading_week_context['monday']} to {trading_week_context['friday']}",
      "current_day": trading_week_context['current_day'],
      "days_remaining": trading_week_context['days_remaining'],
      "portfolio_theme": "Elite Technology Research Portfolio",
      "selection_methodology": "Enhanced AI Analysis + Composite Scoring + Strict Filtering",
      "market_condition": market_condition,

      # การดำเนินการขั้นสูง
      "monday_actions": [
           f"BUY {stock['symbol']} ({stock['suggested_percentage_allocation']:.1f}%) - Target: ${stock['trading_plan']['entry_price_target']:.2f}"
           for stock in portfolio_stocks
      ],
      "friday_actions": [
           f"SELL {stock['symbol']} (Target: {stock['predicted_5day_return']:+.1f}%) - Exit: ${stock['trading_plan']['exit_price_target']:.2f}"
           for stock in portfolio_stocks
      ],
      "midweek_monitoring": [
           f"Monitor {stock['symbol']} | Stop: ${stock['trading_plan']['stop_loss_level']:.2f} | RR: {stock['trading_plan']['risk_reward_ratio']:.2f}"
           for stock in portfolio_stocks
      ],

      # การจัดการพอร์ตขั้นสูง
      "total_allocation": sum(stock['suggested_percentage_allocation'] for stock in portfolio_stocks),
      "cash_reserve": max(0, 100 - sum(stock['suggested_percentage_allocation'] for stock in portfolio_stocks)),
      "position_count": len(portfolio_stocks),
      "average_position_size": round(sum(stock['suggested_percentage_allocation'] for stock in portfolio_stocks) / len(portfolio_stocks), 1),
      "tier_1_allocation": sum(stock['suggested_percentage_allocation'] for stock in portfolio_stocks if stock['selection_tier'] == 'tier_1'),
      "tier_2_allocation": sum(stock['suggested_percentage_allocation'] for stock in portfolio_stocks if stock['selection_tier'] == 'tier_2'),

      # เป้าหมายประสิทธิภาพขั้นสูง
      "expected_weekly_return": round(expected_portfolio_return, 2),
      "confidence_weighted_return": round(confidence_weighted_return, 2),
      "success_probability_portfolio": round(np.mean([stock['trading_plan']['success_probability'] for stock in portfolio_stocks]), 1),
      "average_risk_reward": round(np.mean([stock['trading_plan']['risk_reward_ratio'] for stock in portfolio_stocks]), 2),
      "portfolio_quality_score": round(np.mean([stock['composite_score'] for stock in portfolio_stocks]), 2)
  }

  # คำนวณ risk-adjusted return
  total_risk = sum(stock['risk_metrics']['max_expected_loss'] for stock in portfolio_stocks)
  if total_risk > 0:
       trading_plan["risk_adjusted_return"] = round(trading_plan["expected_weekly_return"] / total_risk, 2)
  else:
       trading_plan["risk_adjusted_return"] = trading_plan["expected_weekly_return"]

  # === การวิเคราะห์พอร์ตขั้นสูง ===
  weekly_portfolio_analytics = {
      "total_stocks": len(portfolio_stocks),
      "total_analyzed": len(all_stocks),
      "total_qualified": len(qualified_stocks),
      "selection_rate": round(len(portfolio_stocks) / len(all_stocks) * 100, 2),
      "qualification_rate": round(len(qualified_stocks) / len(all_stocks) * 100, 2),

      "average_weekly_score": round(np.mean([getattr(stock, 'weekly_opportunity_score', 0) for stock in selected_stocks]), 2),
      "average_composite_score": round(np.mean([stock['composite_score'] for stock in portfolio_stocks]), 2),
      "score_range": f"{getattr(selected_stocks[-1], 'weekly_opportunity_score', 0):.1f} to {getattr(selected_stocks[0], 'weekly_opportunity_score', 0):.1f}",
      "composite_range": f"{min(stock['composite_score'] for stock in portfolio_stocks):.1f} to {max(stock['composite_score'] for stock in portfolio_stocks):.1f}",
      "score_std_dev": round(np.std([getattr(stock, 'weekly_opportunity_score', 0) for stock in selected_stocks]), 2),

      "all_tech_sector": True,
      "selection_methodology": "Research-Grade Enhanced AI Analysis",
      "focus": "Technology sector 5-day swing trades with strict quality control",
      "diversification": "Quality-concentrated strategy",
      "research_grade": True,

      "agent_score_averages": {
           "momentum": round(np.mean([stock['agent_scores']['momentum'] for stock in portfolio_stocks]), 2),
           "catalyst": round(np.mean([stock['agent_scores']['catalyst'] for stock in portfolio_stocks]), 2),
           "technical": round(np.mean([stock['agent_scores']['technical'] for stock in portfolio_stocks]), 2),
           "liquidity": round(np.mean([stock['agent_scores']['liquidity'] for stock in portfolio_stocks]), 2),
           "risk": round(np.mean([stock['agent_scores']['risk'] for stock in portfolio_stocks]), 2),
           "timing": round(np.mean([stock['agent_scores']['timing'] for stock in portfolio_stocks]), 2)
      },

      "quality_distribution": {
           "excellent": sum(1 for stock in portfolio_stocks if stock['research_grade'] == 'EXCELLENT'),
           "very_good": sum(1 for stock in portfolio_stocks if stock['research_grade'] == 'VERY_GOOD'),
           "good": sum(1 for stock in portfolio_stocks if stock['research_grade'] == 'GOOD')
      },

      "tier_distribution": {
           "tier_1": sum(1 for stock in portfolio_stocks if stock['selection_tier'] == 'tier_1'),
           "tier_2": sum(1 for stock in portfolio_stocks if stock['selection_tier'] == 'tier_2'),
           "tier_3": sum(1 for stock in portfolio_stocks if stock['selection_tier'] == 'tier_3')
      }
  }

  # === ผลตอบแทนที่คาดหวังขั้นสูง ===
  individual_returns = [stock['trading_plan']['expected_5day_return'] for stock in portfolio_stocks]
  valid_returns = [r for r in individual_returns if r is not None]
  positive_returns = [r for r in valid_returns if r > 0]
  high_returns = [r for r in valid_returns if r >= 3.0]

  expected_weekly_returns = {
      "portfolio_expected_return": round(trading_plan['expected_weekly_return'], 2),
      "confidence_weighted_return": round(trading_plan['confidence_weighted_return'], 2),
      "individual_returns": individual_returns,
      "return_range": f"{min(valid_returns):.1f}% to {max(valid_returns):.1f}%" if valid_returns else "N/A",
      "average_individual_return": round(np.mean(valid_returns), 2) if valid_returns else 0,
      "median_return": round(np.median(valid_returns), 2) if valid_returns else 0,
      "positive_return_count": len(positive_returns),
      "high_return_count": len(high_returns),
      "positive_return_rate": round(len(positive_returns) / len(valid_returns) * 100, 1) if valid_returns else 0,
      "high_return_rate": round(len(high_returns) / len(valid_returns) * 100, 1) if valid_returns else 0,
      "success_probability": round(trading_plan['success_probability_portfolio'], 1),
      "methodology": "Enhanced AI predictions with composite scoring",
      "return_consistency": round(np.std(valid_returns), 2) if valid_returns else 0
  }

  # === เมตริกความเสี่ยงขั้นสูง ===
  valid_losses = [stock['risk_metrics']['max_expected_loss'] for stock in portfolio_stocks if stock['risk_metrics']['max_expected_loss'] is not None]
  valid_vars = [stock['risk_metrics']['five_day_var'] for stock in portfolio_stocks if stock['risk_metrics']['five_day_var'] is not None]
  risk_reward_ratios = [stock['risk_metrics']['risk_reward_ratio'] for stock in portfolio_stocks if stock['risk_metrics']['risk_reward_ratio'] is not None]

  weekly_risk_metrics = {
      "portfolio_5day_var": round(sum(valid_losses), 1) if valid_losses else 0.0,
      "max_individual_risk": max(abs(v) for v in valid_vars) if valid_vars else 3.0,
      "average_individual_risk": round(np.mean([abs(v) for v in valid_vars]), 2) if valid_vars else 3.0,
      "min_individual_risk": min(abs(v) for v in valid_vars) if valid_vars else 1.0,
      "average_risk_reward_ratio": round(np.mean(risk_reward_ratios), 2) if risk_reward_ratios else 1.5,
      "min_risk_reward_ratio": min(risk_reward_ratios) if risk_reward_ratios else 1.0,
      "risk_distribution": "Research-Grade diversified across tech leaders",
      "risk_level": "Controlled-Moderate (Research optimized)",
      "portfolio_correlation": "Technology sector concentrated with quality focus",
      "max_portfolio_loss": round(sum(valid_losses), 1) if valid_losses else 0.0,
      "risk_adjusted_score": trading_plan["risk_adjusted_return"],
      "downside_protection": "Enhanced with stop-loss and quality filtering"
  }

  # === ข้อมูลจังหวะตลาดขั้นสูง ===
  market_timing_insights = {
      "optimal_entry_consensus": "Research-optimized Monday morning" if all('monday' in stock['trading_plan']['optimal_entry_day'] for stock in portfolio_stocks) else "Mixed timing strategy",
      "optimal_exit_consensus": "Research-optimized Friday close" if all('friday' in stock['trading_plan']['optimal_exit_day'] for stock in portfolio_stocks) else "Mixed exit strategy",
      "average_success_probability": round(np.mean([stock['trading_plan']['success_probability'] for stock in portfolio_stocks]), 1),
      "success_probability_range": f"{min(stock['trading_plan']['success_probability'] for stock in portfolio_stocks):.1f}% to {max(stock['trading_plan']['success_probability'] for stock in portfolio_stocks):.1f}%",
      "timing_confidence": "Very High" if np.std([stock['trading_plan']['success_probability'] for stock in portfolio_stocks]) < 5 else "High" if np.std([stock['trading_plan']['success_probability'] for stock in portfolio_stocks]) < 10 else "Medium",
      "market_conditions": f"Research analysis during {trading_week_context['current_day']} - {trading_week_context['days_remaining']} days remaining",
      "timing_consistency": round(100 - np.std([stock['trading_plan']['success_probability'] for stock in portfolio_stocks]), 1)
  }

  # === การติดตามประสิทธิภาพขั้นสูง ===
  performance_tracking = {
      "tracking_period": f"{trading_week_context['monday']} to {trading_week_context['friday']}",
      "benchmark": "QQQ (Technology Sector ETF)",
      "research_benchmark": "Research-Grade Portfolio Performance",
      "key_metrics_to_track": [
           "Individual stock returns vs enhanced predictions",
           "Composite score accuracy vs actual performance",
           "Portfolio return vs confidence-weighted expectations",
           "Risk management effectiveness",
           "Agent scoring accuracy",
           "Market condition assessment accuracy",
           "Entry/exit timing optimization"
      ],
      "tracking_frequency": "Real-time during trading week",
      "performance_review_date": trading_week_context['friday'],
      "success_criteria": f"Portfolio return > {expected_weekly_returns['portfolio_expected_return']:.1f}% with max loss < {weekly_risk_metrics['portfolio_5day_var']:.1f}%",
      "research_objectives": [
           "Validate enhanced filtering system",
           "Test composite scoring accuracy",
           "Measure risk-adjusted returns",
           "Assess market timing precision"
      ]
  }

  # === สถิติการประมวลผลขั้นสูง ===
  total_elapsed = time.time() - analysis_start_time
  processing_stats = {
      "total_analysis_time": round(total_elapsed, 1),
      "total_analysis_minutes": round(total_elapsed / 60, 1),
      "tech_stocks_analyzed": len(all_stocks),
      "stocks_qualified": len(qualified_stocks),
      "stocks_selected": len(portfolio_stocks),
      "processing_efficiency": f"{len(all_stocks) / (total_elapsed / 60):.1f} stocks/minute",
      "qualification_rate": round(len(qualified_stocks) / len(all_stocks) * 100, 1),
      "selection_rate": round(len(portfolio_stocks) / len(all_stocks) * 100, 1),
      "quality_ratio": round(len(portfolio_stocks) / len(qualified_stocks) * 100, 1) if qualified_stocks else 0,
      "total_ai_evaluations": len(all_stocks) * 6,
      "ai_agents_deployed": 6,
      "focus_sector": "Technology",
      "analysis_completeness": "100% comprehensive with enhanced filtering",
      "research_grade": True,
      "filtering_stages": 10,
      "composite_scoring": True,
      "market_condition_analysis": True,
      "data_sources_used": len(set(source.source_name for stock in selected_stocks for source in getattr(stock, 'all_data_sources', []))),
      "average_data_sources_per_stock": round(np.mean([len(getattr(stock, 'all_data_sources', [])) for stock in selected_stocks]), 1) if selected_stocks else 0,
      "system_version": "Ultimate Research Grade v2.0"
  }

  # === รวบรวมแหล่งข้อมูลทั้งหมด ===
  all_portfolio_sources = []
  for stock in selected_stocks:
      all_portfolio_sources.extend(getattr(stock, 'all_data_sources', []))

  # === สร้างคำแนะนำขั้นสุดท้าย ===
  final_recommendation = ShortTermPortfolioRecommendation(
      selected_stocks=portfolio_stocks,
      trading_plan=trading_plan,
      weekly_portfolio_analytics=weekly_portfolio_analytics,
      expected_weekly_returns=expected_weekly_returns,
      weekly_risk_metrics=weekly_risk_metrics,
      market_timing_insights=market_timing_insights,
      performance_tracking=performance_tracking,
      processing_stats=processing_stats,
      all_data_sources=all_portfolio_sources
  )

  return {"weekly_portfolio": final_recommendation}




# === DECISION FUNCTIONS ===
def should_continue_short_term_batch(state: ShortTermAnalysisState) -> str:
   """Decide whether to continue batch processing or move to portfolio selection"""
   if state.get("batch_completed", False):
       return "portfolio_selector"
   else:
       return "batch_processing"

# === WORKFLOW DEFINITION ===
short_term_workflow = StateGraph(ShortTermAnalysisState)

# Add nodes
short_term_workflow.add_node("initialize", initialize_short_term_analysis)
short_term_workflow.add_node("batch_processing", short_term_batch_processing_node)
short_term_workflow.add_node("portfolio_selector", short_term_portfolio_selector_node)

# Set edges
short_term_workflow.set_entry_point("initialize")
short_term_workflow.add_edge("initialize", "batch_processing")
short_term_workflow.add_conditional_edges("batch_processing", should_continue_short_term_batch,
   {"batch_processing": "batch_processing", "portfolio_selector": "portfolio_selector"})
short_term_workflow.add_edge("portfolio_selector", END)

# Compile workflow
short_term_app = short_term_workflow.compile()

# === PERFORMANCE TRACKING SYSTEM ===

@dataclass
class TradingPerformanceMetrics:
   """Trading performance measurement"""
   portfolio_id: str
   start_date: str
   end_date: str
   expected_return: float
   actual_return: float
   individual_stock_performance: List[Dict[str, Any]]
   risk_metrics: Dict[str, float]
   execution_quality: Dict[str, float]
   ai_accuracy: Dict[str, float]

class PerformanceTracker:
   """Performance tracking system"""

   def __init__(self):
       self.trading_history = []
       self.ai_prediction_accuracy = []
       self.execution_analytics = []

   async def track_portfolio_performance(self, portfolio: ShortTermPortfolioRecommendation,
                                       start_date: str, end_date: str) -> TradingPerformanceMetrics:
       """Track portfolio performance"""
       print(f"📊 Tracking performance for portfolio: {start_date} to {end_date}")

       individual_performance = []
       total_expected_return = 0.0
       total_actual_return = 0.0

       for stock in portfolio.selected_stocks:
           symbol = stock['symbol']
           expected_return = stock['trading_plan']['expected_5day_return']
           weight = stock['suggested_percentage_allocation'] / 100

           try:
               ticker = yf.Ticker(symbol)
               hist = ticker.history(start=start_date, end=end_date)

               if not hist.empty and len(hist) >= 2:
                   start_price = hist['Open'].iloc[0]
                   end_price = hist['Close'].iloc[-1]
                   actual_return = ((end_price - start_price) / start_price) * 100

                   prediction_error = abs(actual_return - expected_return)
                   accuracy_score = max(0, 100 - prediction_error * 10)

                   stock_performance = {
                       'symbol': symbol,
                       'weight': weight,
                       'expected_return': expected_return,
                       'actual_return': actual_return,
                       'prediction_error': prediction_error,
                       'accuracy_score': accuracy_score,
                       'success': actual_return > 0,
                       'start_price': start_price,
                       'end_price': end_price
                   }

                   individual_performance.append(stock_performance)
                   total_expected_return += expected_return * weight
                   total_actual_return += actual_return * weight

           except Exception as e:
               print(f"⚠️ Error tracking {symbol}: {e}")

       # Calculate metrics
       if individual_performance:
           actual_returns = [p['actual_return'] for p in individual_performance]
           risk_metrics = {
               'portfolio_volatility': round(np.std(actual_returns), 2),
               'max_individual_loss': round(min(actual_returns), 2),
               'win_rate': round(sum(1 for p in individual_performance if p['success']) / len(individual_performance) * 100, 1),
               'avg_prediction_error': round(np.mean([p['prediction_error'] for p in individual_performance]), 2)
           }

           ai_accuracy = {
               'avg_accuracy_score': round(np.mean([p['accuracy_score'] for p in individual_performance]), 1),
               'prediction_correlation': round(np.corrcoef([p['expected_return'] for p in individual_performance],
                                                         [p['actual_return'] for p in individual_performance])[0,1], 3) if len(individual_performance) > 1 else 0.0,
               'successful_predictions': sum(1 for p in individual_performance if p['success']) / len(individual_performance) * 100
           }
       else:
           risk_metrics = {}
           ai_accuracy = {}

       execution_quality = {
           'data_availability': len(individual_performance) / len(portfolio.selected_stocks) * 100,
           'portfolio_completion': 100.0 if individual_performance else 0.0
       }

       performance = TradingPerformanceMetrics(
           portfolio_id=f"tech_swing_{start_date}",
           start_date=start_date,
           end_date=end_date,
           expected_return=round(total_expected_return, 2),
           actual_return=round(total_actual_return, 2),
           individual_stock_performance=individual_performance,
           risk_metrics=risk_metrics,
           execution_quality=execution_quality,
           ai_accuracy=ai_accuracy
       )

       self.trading_history.append(performance)
       return performance

   def generate_performance_report(self, performance: TradingPerformanceMetrics) -> str:
       """Generate performance report"""
       report = f"""
📊 === WEEKLY TRADING PERFORMANCE REPORT ===
Portfolio ID: {performance.portfolio_id}
Period: {performance.start_date} to {performance.end_date}

💰 RETURN ANALYSIS:
 Expected Return: {performance.expected_return:+.1f}%
 Actual Return: {performance.actual_return:+.1f}%
 Return Difference: {performance.actual_return - performance.expected_return:+.1f}%

⚠️ RISK METRICS:
 Portfolio Volatility: {performance.risk_metrics.get('portfolio_volatility', 'N/A')}%
 Win Rate: {performance.risk_metrics.get('win_rate', 'N/A')}%
 Max Individual Loss: {performance.risk_metrics.get('max_individual_loss', 'N/A')}%

🧠 AI ACCURACY:
 Average Accuracy Score: {performance.ai_accuracy.get('avg_accuracy_score', 'N/A')}/100
 Prediction Correlation: {performance.ai_accuracy.get('prediction_correlation', 'N/A')}
 Successful Predictions: {performance.ai_accuracy.get('successful_predictions', 'N/A')}%

📈 INDIVIDUAL STOCK PERFORMANCE:
"""
       for stock in performance.individual_stock_performance:
           report += f"   {stock['symbol']}: {stock['actual_return']:+.1f}% (expected: {stock['expected_return']:+.1f}%) | Accuracy: {stock['accuracy_score']:.0f}/100\n"

       return report

# === MAIN EXECUTION FUNCTION ===
async def main_short_term_tech_analysis():
   """Main function for short-term tech stock analysis"""

   # Current time with timezone
   bangkok_tz = pytz.timezone('Asia/Bangkok')
   current_time = datetime.now(bangkok_tz)
   current_time_str = current_time.strftime('%Y-%m-%d %H:%M:%S %Z')
   current_date_str = current_time.strftime('%A, %B %d, %Y')

   print("🚀 === SHORT-TERM TECH STOCK TRADING AI SYSTEM ===")
   print(f"📅 Analysis Date: {current_date_str}")
   print(f"🕐 Analysis Time: {current_time_str}")
   print(f"🌏 Timezone: Bangkok (UTC+7)")
   print("🎯 Specialized for 5-day swing trading in technology sector")
   print("📊 Strategy: Buy Monday → Sell Friday")
   print("🧠 Powered by 6 Specialized AI Agents:")
   print("   1. 📈 Momentum Agent (25% weight) - Short-term momentum analysis")
   print("   2. 📰 News Catalyst Agent (20% weight) - Breaking news and events")
   print("   3. 📊 Technical Swing Agent (20% weight) - 5-day swing setups")
   print("   4. 💧 Liquidity Agent (15% weight) - Volume and execution analysis")
   print("   5. ⚠️ Risk Management Agent (10% weight) - Short-term risk assessment")
   print("   6. 🎯 Timing Agent (10% weight) - Entry/exit optimization")

   print(f"\n🎯 Configuration:")
   print(f"📊 Tech stocks to analyze: {MAX_TECH_STOCKS_TO_ANALYZE}")
   print(f"🔥 Portfolio size: {MAX_STOCKS_TO_SELECT} stocks")
   print(f"💯 Minimum weekly score: {MIN_WEEKLY_SCORE}/10")
   print(f"⚡ Batch processing: {BATCH_SIZE} stocks per batch")

   # API status
   api_status = []
   if GROQ_API_KEY:
       api_status.append("✅ GROQ AI")
   else:
       api_status.append("❌ GROQ AI (simulation mode)")

   if ALPHA_VANTAGE_API_KEY:
       api_status.append("✅ Alpha Vantage")
   else:
       api_status.append("⚠️ Alpha Vantage (limited)")

   if NEWS_API_KEY:
       api_status.append("✅ News API")
   else:
       api_status.append("⚠️ News API (limited)")

   print(f"🔑 API Status: {' | '.join(api_status)}")

   # Trading week context
   monday, friday = get_current_trading_week_dates()
   now_et = get_current_time_et()
   current_day = now_et.strftime('%A')
   days_remaining = max(0, 5 - (now_et - monday).days)

   print(f"\n📅 Trading Week Context:")
   print(f"   This Week: {monday.strftime('%Y-%m-%d')} to {friday.strftime('%Y-%m-%d')}")
   print(f"   Today: {current_day}")
   print(f"   Days Remaining: {max(0, days_remaining)}")

   if current_day == 'Saturday' or current_day == 'Sunday':
       print("   🔄 Weekend: Preparing for next week's trading")
   elif current_day == 'Monday':
       print("   🔥 Monday: Ideal for position entry")
   elif current_day == 'Friday':
       print("   💰 Friday: Position exit day")
   else:
       print(f"   📊 {current_day}: Mid-week analysis")

   start_time = time.time()

   try:
       # === RUN MAIN WORKFLOW ===
       print(f"\n🔄 Starting comprehensive tech stock analysis...")
       final_state = await short_term_app.ainvoke({})

       elapsed = time.time() - start_time
       print(f"\n🏁 === ANALYSIS COMPLETE ===")
       print(f"⏱️ Total Time: {elapsed:.1f} seconds ({elapsed/60:.1f} minutes)")

       # Process results
       portfolio = final_state.get("weekly_portfolio")
       if portfolio and portfolio.selected_stocks:
           print(f"\n📊 === WEEKLY TECH PORTFOLIO RESULTS ===")

           # Overall statistics
           stats = portfolio.processing_stats
           print(f"✅ Tech Stocks Analyzed: {stats.get('tech_stocks_analyzed', 0)}")
           print(f"🎯 Qualified Candidates: {stats.get('stocks_qualified', 0)}")
           print(f"📈 Selected for Portfolio: {stats.get('stocks_selected', 0)}")
           print(f"🧠 Total AI Evaluations: {stats.get('total_ai_evaluations', 0)}")
           print(f"⚡ Processing Efficiency: {stats.get('processing_efficiency', 'N/A')}")

           # Performance projections
           expected_returns = portfolio.expected_weekly_returns
           print(f"\n🎯 === PERFORMANCE PROJECTIONS ===")
           print(f"📈 Expected Portfolio Return: {expected_returns.get('portfolio_expected_return', 0):+.1f}% (5 days)")
           print(f"📊 Individual Returns Range: {expected_returns.get('return_range', 'N/A')}")
           print(f"🎲 Success Probability: {expected_returns.get('success_probability', 0):.1f}%")

           # Risk analysis
           risk_metrics = portfolio.weekly_risk_metrics
           print(f"⚠️ Portfolio Risk (5-day VaR): {risk_metrics.get('portfolio_5day_var', 0):.1f}%")
           print(f"📊 Max Individual Risk: {risk_metrics.get('max_individual_risk', 0):.1f}%")

        #    # Selected stocks summary
        #    print(f"\n📋 === SELECTED TECH STOCKS SUMMARY ===")
        #    for stock in portfolio.selected_stocks:
        #        symbol = stock['symbol']
        #        current_price = stock['current_price']
        #        predicted_price = stock['predicted_friday_price']
        #        predicted_return = stock['predicted_5day_return']
        #        confidence = stock['prediction_confidence']
        #        score = stock['weekly_opportunity_score']
        #        weight = stock['suggested_percentage_allocation']
        #        success_prob = stock['trading_plan']['success_probability']

        #        print(f"🔹 {symbol}: ${current_price:.2f} → ${predicted_price:.2f} ({predicted_return:+.1f}%)")
        #        print(f"   📊 AI Score: {score:.1f}/10 | Weight: {weight:.1f}% | Success: {success_prob:.0f}%")
        #        print(f"   🎯 Confidence: {confidence:.1%} | Entry: {stock['trading_plan']['optimal_entry_day']}")
        # === ENHANCED PORTFOLIO DISPLAY ===
           def display_beautiful_portfolio(portfolio_stocks):
               """แสดงผลพอร์ตแบบสวยงามและครบถ้วน"""

               print(f"\n" + "="*80)
               print(f"🏆 ELITE TECH PORTFOLIO - TOP {len(portfolio_stocks)} STOCKS")
               print(f"📅 Trading Week: Monday Buy → Friday Sell Strategy")
               print(f"="*80)

               for i, stock in enumerate(portfolio_stocks):
                   # ข้อมูลพื้นฐาน
                   symbol = stock['symbol']
                   company_name = stock.get('company_name', symbol)
                   rank = i + 1

                   # คะแนนและการประเมิน
                   ai_score = stock.get('weekly_opportunity_score', 0)
                   composite_score = stock.get('composite_score', 0)
                   confidence = stock.get('prediction_confidence', 0) * 100

                   # ราคาและผลตอบแทน
                   current_price = stock.get('current_price', 0)
                   predicted_price = stock.get('predicted_friday_price', 0)
                   expected_return = stock.get('predicted_5day_return', 0)

                   # การจัดการพอร์ต
                   weight = stock.get('suggested_percentage_allocation', 0)

                   # ความเสี่ยงและโอกาส
                   success_prob = stock.get('trading_plan', {}).get('success_probability', 0)
                   max_loss = stock.get('risk_metrics', {}).get('max_expected_loss', 0)
                   risk_reward = stock.get('trading_plan', {}).get('risk_reward_ratio', 0)

                   # จุดเข้า-ออก
                   entry_time = stock.get('trading_plan', {}).get('optimal_entry_day', 'monday_open')
                   exit_time = stock.get('trading_plan', {}).get('optimal_exit_day', 'friday_close')
                   entry_price = stock.get('trading_plan', {}).get('entry_price_target', current_price)
                   exit_price = stock.get('trading_plan', {}).get('exit_price_target', predicted_price)
                   stop_loss = stock.get('trading_plan', {}).get('stop_loss_level', current_price * 0.95)

                   # กำหนดสีและไอคอนตามผลตอบแทน
                   if expected_return > 3:
                       return_icon = "🚀"
                       return_color = "EXCELLENT"
                   elif expected_return > 1:
                       return_icon = "📈"
                       return_color = "GOOD"
                   elif expected_return > -1:
                       return_icon = "📊"
                       return_color = "MODERATE"
                   else:
                       return_icon = "⚠️"
                       return_color = "CAUTION"

                   # กำหนดระดับความเสี่ยง
                   if max_loss < 2:
                       risk_icon = "🟢"
                       risk_level = "LOW"
                   elif max_loss < 4:
                       risk_icon = "🟡"
                       risk_level = "MODERATE"
                   else:
                       risk_icon = "🔴"
                       risk_level = "HIGH"

                   # แสดงผลแต่ละหุ้น
                   print(f"\n{'─'*80}")
                   print(f"📊 STOCK #{rank}: {symbol} - {company_name}")
                   print(f"{'─'*80}")

                   # บรรทัดที่ 1: คะแนนและการประเมิน
                   print(f"🧠 AI ANALYSIS:")
                   print(f"   💯 AI Score: {ai_score:.1f}/10 | 🎯 Composite: {composite_score:.1f} | 🎲 Confidence: {confidence:.1f}%")

                   # บรรทัดที่ 2: ราคาและผลตอบแทน
                   print(f"💰 PRICE & RETURNS:")
                   print(f"   💵 Current: ${current_price:.2f} → Predicted: ${predicted_price:.2f}")
                   print(f"   {return_icon} Expected 5-Day Return: {expected_return:+.1f}% ({return_color})")

                   # บรรทัดที่ 3: การจัดการพอร์ต
                   print(f"📈 PORTFOLIO ALLOCATION:")
                   money_allocation = (weight / 100) * MONEY
                   print(f"   💼 Weight: {weight:.1f}% | 💰 Investment:${money_allocation:,.0f}")
                   print(f"   🎯 Success Probability: {success_prob:.1f}%")

                  # บรรทัดที่ 4: ความเสี่ยงและการป้องกัน
                   print(f"⚠️ RISK MANAGEMENT:")
                   print(f"   {risk_icon} Max Loss: {max_loss:.1f}% ({risk_level}) | ⚖️ Risk-Reward: {risk_reward:.2f}:1")
                   print(f"   🛡️ Stop Loss: ${stop_loss:.2f}")

                   # บรรทัดที่ 5: แผนการเทรด
                   print(f"📅 TRADING PLAN:")
                   print(f"   🟢 ENTRY: {entry_time.replace('_', ' ').title()} @ ${entry_price:.2f}")
                   print(f"   🔴 EXIT: {exit_time.replace('_', ' ').title()} @ ${exit_price:.2f}")

                   # Agent scores breakdown
                   agent_scores = stock.get('agent_scores', {})
                   if agent_scores:
                       print(f"🤖 AI AGENTS BREAKDOWN:")
                       agents_line1 = f"   📈 Momentum: {agent_scores.get('momentum', 0):.1f} | 📰 Catalyst: {agent_scores.get('catalyst', 0):.1f} | 📊 Technical: {agent_scores.get('technical', 0):.1f}"
                       agents_line2 = f"   💧 Liquidity: {agent_scores.get('liquidity', 0):.1f} | ⚠️ Risk: {agent_scores.get('risk', 0):.1f} | 🎯 Timing: {agent_scores.get('timing', 0):.1f}"
                       print(agents_line1)
                       print(agents_line2)

               # สรุปพอร์ตรวม
               print(f"\n" + "="*80)
               print(f"📊 PORTFOLIO SUMMARY")
               print(f"="*80)

               total_allocation = sum(stock.get('suggested_percentage_allocation', 0) for stock in portfolio_stocks)
               avg_expected_return = sum(stock.get('predicted_5day_return', 0) * stock.get('suggested_percentage_allocation', 0) / 100 for stock in portfolio_stocks)
               avg_success_prob = sum(stock.get('trading_plan', {}).get('success_probability', 0) for stock in portfolio_stocks) / len(portfolio_stocks)
               total_max_loss = sum(stock.get('risk_metrics', {}).get('max_expected_loss', 0) for stock in portfolio_stocks)

               print(f"🎯 Portfolio Size: {len(portfolio_stocks)} elite tech stocks")
               print(f"💰 Total Allocation: {total_allocation:.1f}% | 🏦 Cash Reserve: {100-total_allocation:.1f}%")
               print(f"📈 Expected Portfolio Return: {avg_expected_return:+.1f}% (5 days)")
               print(f"🎲 Average Success Probability: {avg_success_prob:.1f}%")
               print(f"⚠️ Maximum Portfolio Risk: {total_max_loss:.1f}%")

               # แนวโน้มตลาด
               positive_stocks = sum(1 for stock in portfolio_stocks if stock.get('predicted_5day_return', 0) > 0)
               high_confidence = sum(1 for stock in portfolio_stocks if stock.get('prediction_confidence', 0) > 0.75)

               print(f"\n📊 MARKET SENTIMENT:")
               print(f"🟢 Positive Outlook: {positive_stocks}/{len(portfolio_stocks)} stocks ({positive_stocks/len(portfolio_stocks)*100:.0f}%)")
               print(f"🎯 High Confidence: {high_confidence}/{len(portfolio_stocks)} stocks ({high_confidence/len(portfolio_stocks)*100:.0f}%)")

               print(f"\n" + "="*80)
               print(f"✅ READY FOR MONDAY MARKET OPEN!")
               print(f"="*80)

           # เรียกใช้ function นี้แทนการแสดงผลเดิม
           display_beautiful_portfolio(portfolio.selected_stocks)

           # Data sources summary
           total_sources = len(portfolio.all_data_sources)
           successful_sources = sum(1 for source in portfolio.all_data_sources if source.success)
           print(f"\n🔗 === DATA SOURCES SUMMARY ===")
           print(f"📊 Total Data Sources Used: {total_sources}")
           print(f"✅ Successful Sources: {successful_sources} ({successful_sources/total_sources*100:.1f}%)")

           # Unique data sources
           unique_sources = {}
           for source in portfolio.all_data_sources:
               if source.source_name not in unique_sources:
                   unique_sources[source.source_name] = {
                       'url': source.source_url,
                       'success_count': 0,
                       'total_count': 0
                   }
               unique_sources[source.source_name]['total_count'] += 1
               if source.success:
                   unique_sources[source.source_name]['success_count'] += 1

           print(f"🌐 Data Source Breakdown:")
           for source_name, info in unique_sources.items():
               success_rate = info['success_count'] / info['total_count'] * 100
               status = "✅" if success_rate > 80 else "⚠️" if success_rate > 50 else "❌"
               print(f"   {status} {source_name}: {info['success_count']}/{info['total_count']} ({success_rate:.0f}%)")
               if info['url']:
                   print(f"      🔗 {info['url']}")

           # Trading plan
           trading_plan = portfolio.trading_plan
           print(f"\n📅 === WEEKLY TRADING PLAN ===")
           print(f"🗓️ Strategy: {trading_plan.get('strategy', 'N/A')}")
           print(f"📅 Trading Week: {trading_plan.get('trading_week', 'N/A')}")
           print(f"💰 Total Allocation: {trading_plan.get('total_allocation', 0):.1f}%")
           print(f"🏦 Cash Reserve: {trading_plan.get('cash_reserve', 0):.1f}%")

           monday_actions = trading_plan.get('monday_actions', [])
           friday_actions = trading_plan.get('friday_actions', [])

           if monday_actions:
               print(f"📈 Monday Actions: {', '.join(monday_actions)}")
           if friday_actions:
               print(f"📉 Friday Actions: {', '.join(friday_actions)}")

           # Next steps
           print(f"\n📋 === NEXT STEPS FOR IMPLEMENTATION ===")
           print(f"1. 📊 Review individual stock analysis and risk metrics")
           print(f"2. 💰 Adjust position sizes based on your risk tolerance")
           print(f"3. 📅 Set calendar reminders for Monday entry and Friday exit")
           print(f"4. 🔔 Monitor news and catalyst events during the week")
           print(f"5. 📈 Track actual vs predicted performance for system improvement")

           # Performance tracking setup
           print(f"\n📊 === PERFORMANCE TRACKING SETUP ===")
           print(f"📅 Start Date: {monday.strftime('%Y-%m-%d')} (Monday)")
           print(f"📅 End Date: {friday.strftime('%Y-%m-%d')} (Friday)")
           print(f"📊 Benchmark: QQQ (Technology Select Sector)")
           print(f"🎯 Success Metrics: Return vs prediction, win rate, risk management")

           # Example tracking code
           print(f"\n🔧 === EXAMPLE: PERFORMANCE TRACKING ===")
           print(f"# To track this week's performance after Friday close:")
           print(f"# performance_tracker = PerformanceTracker()")
           print(f"# performance = await performance_tracker.track_portfolio_performance(")
           print(f"#     portfolio, '{monday.strftime('%Y-%m-%d')}', '{friday.strftime('%Y-%m-%d')}')")
           print(f"# report = performance_tracker.generate_performance_report(performance)")
           print(f"# print(report)")

       else:
           print("❌ No portfolio generated")
           print("🔍 Possible issues:")
           print(f"   - No stocks met minimum score requirement ({MIN_WEEKLY_SCORE}/10)")
           print(f"   - Data collection failures")
           print(f"   - API connectivity issues")

           # Diagnostic information
           all_stocks_data = final_state.get("all_stocks_data", {})
           if all_stocks_data:
               scores = [stock.weekly_opportunity_score for stock in all_stocks_data.values() if stock.weekly_opportunity_score > 0]
               if scores:
                   print(f"📊 Score Distribution:")
                   print(f"   Highest Score: {max(scores):.1f}/10")
                   print(f"   Average Score: {np.mean(scores):.1f}/10")
                   print(f"   Stocks Above 5.0: {len([s for s in scores if s >= 5.0])}")
                   print(f"💡 Consider lowering minimum score threshold")

   except Exception as e:
       print(f"❌ System error: {e}")
       import traceback
       traceback.print_exc()

       # Error diagnosis
       print(f"\n🔍 === ERROR DIAGNOSIS ===")
       error_type = type(e).__name__
       error_msg = str(e)

       print(f"❌ Error Type: {error_type}")
       print(f"📝 Error Message: {error_msg}")

       # Suggest solutions
       if "KeyError" in error_type:
           print(f"💡 Solution: Missing data field - check data model structure")
       elif "HTTPError" in error_type or "ConnectionError" in error_type:
           print(f"💡 Solution: API connectivity issue - check network and API keys")
       elif "JSONDecodeError" in error_type:
           print(f"💡 Solution: AI response parsing issue - check GROQ API status")
       elif "TimeoutError" in error_type:
           print(f"💡 Solution: Request timeout - try reducing batch size")
       else:
           print(f"💡 Solution: Check logs above for specific error details")

       print(f"\n🔧 === RECOVERY SUGGESTIONS ===")
       print(f"1. 🔑 Verify all API keys are correctly set")
       print(f"2. 🌐 Check internet connectivity")
       print(f"3. 📊 Try with smaller batch size (current: {BATCH_SIZE})")
       print(f"4. 💯 Lower minimum score threshold (current: {MIN_WEEKLY_SCORE})")
       print(f"5. 🔄 Restart the system and try again")

   finally:
       # System execution summary
       final_elapsed = time.time() - start_time
       print(f"\n🎯 === SYSTEM EXECUTION SUMMARY ===")
       print(f"⏱️ Total Execution Time: {final_elapsed:.1f} seconds")
       print(f"🎯 Strategy Focus: 5-day tech stock swing trading")
       print(f"🧠 AI Agents Used: 6 specialized agents")
       print(f"📊 Analysis Scope: {MAX_TECH_STOCKS_TO_ANALYZE} tech stocks")
       print(f"🔄 Processing Mode: {'AI-powered' if GROQ_API_KEY else 'Simulation'}")

       # System health summary
       health_indicators = []
       if GROQ_API_KEY:
           health_indicators.append("🟢 AI Agents Active")
       else:
           health_indicators.append("🟡 AI Simulation Mode")

       if ALPHA_VANTAGE_API_KEY or NEWS_API_KEY:
           health_indicators.append("🟢 News Data Available")
       else:
           health_indicators.append("🟡 Limited News Data")

       health_indicators.append("🟢 Yahoo Finance Data")

       print(f"🏥 System Health: {' | '.join(health_indicators)}")

       print(f"\n⚠️ === IMPORTANT DISCLAIMERS ===")
       print(f"🚨 HIGH-RISK STRATEGY: Short-term trading involves significant risk")
       print(f"📚 EDUCATIONAL PURPOSE: This system is for learning and research")
       print(f"💼 NOT FINANCIAL ADVICE: Always consult professional advisors")
       print(f"💰 CAPITAL RISK: Only trade with money you can afford to lose")
       print(f"📊 BACKTESTING REQUIRED: Test thoroughly before live trading")

       print(f"\n✅ === SHORT-TERM TECH TRADING AI SYSTEM READY ===")

# === UTILITY FUNCTIONS ===

def show_configuration_summary():
   """Display system configuration summary"""
   print(f"\n⚙️ === SYSTEM CONFIGURATION SUMMARY ===")

   print(f"📊 ANALYSIS PARAMETERS:")
   print(f"   Max Stocks to Analyze: {MAX_TECH_STOCKS_TO_ANALYZE}")
   print(f"   Portfolio Size: {MAX_STOCKS_TO_SELECT}")
   print(f"   Minimum Score: {MIN_WEEKLY_SCORE}/10")
   print(f"   Batch Size: {BATCH_SIZE}")

   print(f"\n🎯 STRATEGY CONFIGURATION:")
   print(f"   Focus Sector: Technology")
   print(f"   Holding Period: 5 days (Monday → Friday)")
   print(f"   Trading Style: Swing Trading")
   print(f"   Model Temperature: {TEMPERATURE_SETTING}")

   print(f"\n📅 TIME PERIODS:")
   print(f"   Short-term Analysis: {SHORT_TERM_ANALYSIS_PERIOD}")
   print(f"   Technical Analysis: {TECHNICAL_ANALYSIS_PERIOD}")
   print(f"   Volume Analysis: {VOLUME_ANALYSIS_PERIOD}")
   print(f"   News Articles Target: {NEWS_ARTICLES_TARGET}")

   print(f"\n🤖 AI AGENT WEIGHTS:")
   print(f"   Momentum Agent: 25%")
   print(f"   Catalyst Agent: 20%")
   print(f"   Technical Agent: 20%")
   print(f"   Liquidity Agent: 15%")
   print(f"   Risk Agent: 10%")
   print(f"   Timing Agent: 10%")

   print(f"\n🏭 TECH STOCKS UNIVERSE:")
   print(f"   Total Symbols: {len(TECH_STOCKS_UNIVERSE)}")
   print(f"   Sample: {', '.join(TECH_STOCKS_UNIVERSE[:10])}...")

   print(f"\n🔑 API STATUS:")
   print(f"   GROQ API: {'✅ Connected' if GROQ_API_KEY else '❌ Missing'}")
   print(f"   Alpha Vantage: {'✅ Connected' if ALPHA_VANTAGE_API_KEY else '❌ Missing'}")
   print(f"   News API: {'✅ Connected' if NEWS_API_KEY else '❌ Missing'}")
   print(f"   Yahoo Finance: ✅ Available (free)")

def get_trading_calendar_info():
   """Get trading calendar information"""
   monday, friday = get_current_trading_week_dates()
   today = datetime.now()

   info = {
       "current_week": {
           "monday": monday.strftime('%Y-%m-%d'),
           "friday": friday.strftime('%Y-%m-%d'),
           "today": today.strftime('%Y-%m-%d %A'),
           "is_trading_day": is_trading_day(today),
           "days_remaining": max(0, 5 - (today - monday).days)
       },
       "next_week": {},
       "market_status": "open" if is_trading_day(today) and 9 <= today.hour <= 16 else "closed"
   }

   # Next week
   next_monday = monday + timedelta(days=7)
   next_friday = next_monday + timedelta(days=4)

   info["next_week"] = {
       "monday": next_monday.strftime('%Y-%m-%d'),
       "friday": next_friday.strftime('%Y-%m-%d')
   }

   return info

def print_trading_calendar():
   """Display trading calendar"""
   cal_info = get_trading_calendar_info()

   print("📅 === TRADING CALENDAR ===")
   print(f"This Week: {cal_info['current_week']['monday']} to {cal_info['current_week']['friday']}")
   print(f"Today: {cal_info['current_week']['today']}")
   print(f"Trading Day: {'Yes' if cal_info['current_week']['is_trading_day'] else 'No'}")
   print(f"Days Remaining: {cal_info['current_week']['days_remaining']}")
   print(f"Market Status: {cal_info['market_status'].title()}")
   print(f"Next Week: {cal_info['next_week']['monday']} to {cal_info['next_week']['friday']}")

async def quick_system_test(test_symbols: List[str] = None):
   """Quick system functionality test"""
   if not test_symbols:
       test_symbols = ["AAPL", "MSFT"]

   print(f"🧪 === QUICK SYSTEM TEST ===")
   print(f"Testing with symbols: {', '.join(test_symbols)}")

   start_time = time.time()

   try:
       async with AsyncClient(timeout=120.0) as client:
           for symbol in test_symbols:
               print(f"\n🔍 Testing {symbol}...")

               # Price data test
               price_data = await fetch_short_term_price_data(symbol)
               print(f"   📈 Price data: {'✅' if not price_data.error else '❌'}")
               if not price_data.error:
                   print(f"      Current: ${price_data.current_price:.2f}")
                   print(f"      Predicted: ${price_data.predicted_friday_price:.2f} ({price_data.predicted_5day_return:+.1f}%)")

               # News data test
               deps = ShortTermAgentDeps(client, symbol)
               news_data = await fetch_news_catalyst_data(symbol, deps)
               print(f"   📰 News data: {'✅' if not news_data.error else '❌'}")
               if not news_data.error:
                   print(f"      Catalyst strength: {news_data.catalyst_strength}")
                   print(f"      News articles: {len(news_data.breaking_news)}")

               # Technical analysis test
               technical_data = await calculate_technical_swing_signals(symbol, price_data)
               print(f"   📊 Technical analysis: {'✅' if not technical_data.error else '❌'}")
               if not technical_data.error:
                   print(f"      Momentum: {technical_data.momentum_strength}")
                   print(f"      Entry: ${technical_data.suggested_entry_level:.2f}")

               # Volume analysis test
               volume_data = await analyze_volume_liquidity(symbol)
               print(f"   📊 Volume analysis: {'✅' if not volume_data.error else '❌'}")
               if not volume_data.error:
                   print(f"      Liquidity score: {volume_data.liquidity_score:.1f}/10")
                   print(f"      Avg volume: {volume_data.average_daily_volume/1_000_000:.1f}M")

               if not price_data.error and not technical_data.error:
                   print(f"   ✅ {symbol} ready for AI analysis")
               else:
                   print(f"   ❌ {symbol} has data issues")

               await asyncio.sleep(1)

       elapsed = time.time() - start_time
       print(f"\n🏁 Quick test completed in {elapsed:.1f} seconds")
       print(f"✅ System appears to be functioning correctly")

   except Exception as e:
       print(f"❌ Quick test failed: {e}")
       print(f"🔧 Check API keys and network connectivity")

async def validate_ai_agents(test_symbol: str = "AAPL"):
   """Test AI agents functionality"""
   print(f"🤖 === AI AGENTS VALIDATION ===")
   print(f"Testing AI agents with {test_symbol}...")

   if not GROQ_API_KEY:
       print(f"⚠️ GROQ API key not available - AI agents will run in simulation mode")
       return

   try:
       async with AsyncClient(timeout=60.0) as client:
           # Prepare test data
           price_data = await fetch_short_term_price_data(test_symbol)

           # Test momentum agent
           momentum_agent = Agent(
               ACTIVE_GROQ_MODEL,
               system_prompt="You are a momentum trading expert. Analyze short-term momentum for swing trading."
           )

           test_context = {
               "current_price": price_data.current_price,
               "daily_momentum": price_data.daily_momentum,
               "week_to_date_return": price_data.week_to_date_return,
               "predicted_5day_return": price_data.predicted_5day_return
           }

           test_prompt = f"""
Test analysis for {test_symbol}. Provide JSON response with:
{{"score": 7.5, "confidence": 0.8, "recommendation": "buy_monday"}}
"""

           result = await run_short_term_ai_agent(
               "momentum", momentum_agent, test_context,
               test_prompt, test_symbol, client
           )

           if 'score' in result and isinstance(result['score'], (int, float)):
               print(f"✅ AI agent test successful")
               print(f"   Score: {result['score']:.1f}/10")
               print(f"   Recommendation: {result.get('recommendation', 'N/A')}")
               print(f"   Simulation: {result.get('simulation', False)}")
           else:
               print(f"❌ AI agent test failed - invalid response format")
               print(f"   Response: {result}")

   except Exception as e:
       print(f"❌ AI agent validation failed: {e}")

# === BACKTESTING SYSTEM ===

class BacktestEngine:
   """Engine for backtesting the AI system"""

   def __init__(self, start_date: str, end_date: str):
       self.start_date = start_date
       self.end_date = end_date
       self.results = []

   async def run_historical_backtest(self, weeks_to_test: int = 4) -> Dict[str, Any]:
       """Run historical backtest"""
       print(f"🔄 Starting backtest for {weeks_to_test} weeks...")

       backtest_results = []
       current_date = datetime.now() - timedelta(weeks=weeks_to_test)

       for week in range(weeks_to_test):
           week_start = current_date + timedelta(weeks=week)

           # Find Monday and Friday of that week
           days_since_monday = week_start.weekday()
           monday = week_start - timedelta(days=days_since_monday)
           friday = monday + timedelta(days=4)

           monday_str = monday.strftime('%Y-%m-%d')
           friday_str = friday.strftime('%Y-%m-%d')

           print(f"📅 Backtesting week {week+1}: {monday_str} to {friday_str}")

           try:
               # Test with top 3 tech stocks
               test_symbols = ["AAPL", "MSFT", "GOOGL"]

               week_performance = []
               for symbol in test_symbols:
                   ticker = yf.Ticker(symbol)
                   hist = ticker.history(start=monday_str, end=friday_str)

                   if not hist.empty and len(hist) >= 2:
                       start_price = hist['Open'].iloc[0]
                       end_price = hist['Close'].iloc[-1]
                       weekly_return = ((end_price - start_price) / start_price) * 100

                       week_performance.append({
                           'symbol': symbol,
                           'weekly_return': weekly_return,
                           'start_price': start_price,
                           'end_price': end_price
                       })

               if week_performance:
                   avg_weekly_return = np.mean([p['weekly_return'] for p in week_performance])
                   backtest_results.append({
                       'week': week + 1,
                       'start_date': monday_str,
                       'end_date': friday_str,
                       'avg_return': avg_weekly_return,
                       'individual_performance': week_performance
                   })

           except Exception as e:
               print(f"❌ Error in backtest week {week+1}: {e}")
               backtest_results.append({
                   'week': week + 1,
                   'start_date': monday_str,
                   'end_date': friday_str,
                   'avg_return': 0.0,
                   'error': str(e)
               })

       # Analyze backtest results
       successful_weeks = [r for r in backtest_results if 'error' not in r]
       if successful_weeks:
           all_returns = [r['avg_return'] for r in successful_weeks]

           backtest_summary = {
               'total_weeks_tested': weeks_to_test,
               'successful_weeks': len(successful_weeks),
               'success_rate': len(successful_weeks) / weeks_to_test * 100,
               'avg_weekly_return': round(np.mean(all_returns), 2),
               'volatility': round(np.std(all_returns), 2),
               'best_week': round(max(all_returns), 2),
               'worst_week': round(min(all_returns), 2),
               'win_rate': sum(1 for r in all_returns if r > 0) / len(all_returns) * 100,
               'detailed_results': backtest_results
           }
       else:
           backtest_summary = {
               'error': 'No successful backtest weeks',
               'total_weeks_tested': weeks_to_test
           }

       return backtest_summary

# === EXECUTION WRAPPER ===
if __name__ == "__main__":
   # Display configuration summary
   show_configuration_summary()

   # Check environment
   if IS_COLAB or "JPY_PARENT_PID" in os.environ or os.getenv('IPYTHON_kernel') == '1':
       try:
           import nest_asyncio
           nest_asyncio.apply()
           print("🔧 Applied nest_asyncio for Colab environment")
       except ImportError:
           print("⚠️ nest_asyncio not available")

       try:
           # Run main system
           asyncio.run(main_short_term_tech_analysis())
       except RuntimeError as e:
           if "cannot be nested" in str(e).lower() or "event loop is already running" in str(e).lower():
               print("🔄 Using event loop workaround for Colab")
               loop = asyncio.get_event_loop()
               loop.run_until_complete(main_short_term_tech_analysis())
           else:
               raise e
   else:
       # Run in normal environment
       asyncio.run(main_short_term_tech_analysis())

# === HELPER FUNCTIONS FOR MANUAL TESTING ===

async def run_single_stock_analysis(symbol: str):
   """Run analysis for a single stock"""
   print(f"🔍 === SINGLE STOCK ANALYSIS: {symbol} ===")

   start_time = time.time()

   try:
       async with AsyncClient(timeout=120.0) as client:
           # Data collection
           print(f"📊 Collecting data for {symbol}...")

           price_data = await fetch_short_term_price_data(symbol)
           deps = ShortTermAgentDeps(client, symbol)
           news_data = await fetch_news_catalyst_data(symbol, deps)
           technical_data = await calculate_technical_swing_signals(symbol, price_data)
           volume_data = await analyze_volume_liquidity(symbol)
           risk_data = await assess_short_term_risk(symbol, price_data)

           all_data = {
               'price_data': price_data.model_dump(),
               'technical_swing_data': technical_data.model_dump(),
               'volume_liquidity_data': volume_data.model_dump(),
               'short_term_risk_data': risk_data.model_dump()
           }
           timing_data = await optimize_entry_exit_timing(symbol, all_data)

           # Display results
           print(f"\n📈 === PRICE ANALYSIS ===")
           print(f"Current Price: ${price_data.current_price:.2f}")
           print(f"Predicted Friday: ${price_data.predicted_friday_price:.2f}")
           print(f"Expected 5-day Return: {price_data.predicted_5day_return:+.1f}%")
           print(f"Prediction Confidence: {price_data.prediction_confidence:.1%}")

           print(f"\n📰 === NEWS ANALYSIS ===")
           print(f"Catalyst Strength: {news_data.catalyst_strength}")
           print(f"Breaking News Count: {len(news_data.breaking_news)}")

           print(f"\n📊 === TECHNICAL ANALYSIS ===")
           print(f"Momentum Strength: {technical_data.momentum_strength}")
           print(f"Entry Level: ${technical_data.suggested_entry_level:.2f}")
           print(f"Exit Level: ${technical_data.suggested_exit_level:.2f}")
           print(f"Stop Loss: ${technical_data.stop_loss_level:.2f}")

           print(f"\n💧 === LIQUIDITY ANALYSIS ===")
           print(f"Liquidity Score: {volume_data.liquidity_score:.1f}/10")
           print(f"Average Volume: {volume_data.average_daily_volume/1_000_000:.1f}M")
           print(f"Execution Difficulty: {volume_data.execution_difficulty}")

           print(f"\n⚠️ === RISK ANALYSIS ===")
           print(f"5-day VaR: {risk_data.five_day_var:.1f}%")
           print(f"Risk-Reward Ratio: {risk_data.risk_reward_ratio:.2f}")
           print(f"Volatility vs Optimal: {risk_data.current_volatility_vs_optimal}")

           print(f"\n🎯 === TIMING ANALYSIS ===")
           print(f"Best Entry: {timing_data.best_entry_time}")
           print(f"Best Exit: {timing_data.best_exit_time}")
           print(f"Success Probability: {timing_data.success_probability:.1%}")

           elapsed = time.time() - start_time
           print(f"\n✅ Analysis completed in {elapsed:.1f} seconds")

   except Exception as e:
       print(f"❌ Error analyzing {symbol}: {e}")

def run_quick_analysis(symbol: str = "AAPL"):
   """Quick analysis wrapper"""
   if IS_COLAB:
       import nest_asyncio
       nest_asyncio.apply()

   asyncio.run(run_single_stock_analysis(symbol))

# === FINAL SYSTEM READY MESSAGE ===
print("\n✅ === SHORT-TERM TECH TRADING AI SYSTEM LOADED ===")
print("🚀 Ready for 5-day swing trading analysis")
print("🎯 Focused on Technology sector stocks")
print("🧠 Powered by 6 specialized AI agents")
print("📊 Optimized for Monday buy → Friday sell strategy")